In [ ]:
## This file implements neural networks for pyokpresabsSTCC_qual with four replicates.
## We compute the mean and standarad deviation of training and test accuracies.
## We also compute the mean and standard deviation of AUC ROC values for four replicates.

In [1]:
from numpy.random import seed
import numpy as np
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/pyokpresabsSTCC_qual.csv')
df.shape

(253, 18)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      0
1      0
2      0
3      0
4      0
      ..
248    0
249    0
250    0
251    0
252    0
Name: pheno, Length: 253, dtype: int64

In [5]:
df.head()

,id,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGTAATTTGAAAGTATCGTTTGATTATATAGATTGGAT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGTAATTTGAAAGTATCGTTTGATTATATAGATTGGATTAAATTAGGTTTTGGTCCATCAG,CCTTGTTGCGG,CCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTT,ST,CC,pheno
0,107,0,1,1,1,1,0,1,0,0,1,1,1,0,1,5,5,0
1,109,0,1,0,1,1,0,1,0,0,1,1,1,0,1,8,8,0
2,115,0,0,0,0,1,1,1,1,1,0,0,0,1,0,5,5,0
3,120335,0,1,0,1,1,0,1,0,0,1,1,1,0,1,5,5,0
4,120337,0,1,0,1,1,0,1,0,0,1,1,1,0,1,5,5,0


In [6]:
df['pheno'].value_counts()

0    217
1     32
2      4
Name: pheno, dtype: int64

In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(253, 17)

In [9]:
df_clean.head()

,TTTTCCCCCAT,TTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTTC,TGGGTCTGAC,TCCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGT,TATATAGACTG,TAGTCGCACT,TAAGAATAATATATTAAATATTTATTAACAAATTATAGATAAAATATGAATAATTAATTAATGGTATTTACATATTCATAACC,GGGCTGAGG,GAGCAACCTT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGTAATTTGAAAGTATCGTTTGATTATATAGATTGGAT,GAACCATGGACATCATGTGAATTTGATTTTACAAGAGAGGGTAATTTGAAAGTATCGTTTGATTATATAGATTGGATTAAATTAGGTTTTGGTCCATCAG,CCTTGTTGCGG,CCTGATGGACCAAAACCTAATTTAATCCAATCTATATAATCAAACGATACTTTCAAATTACCCTCTCTTGTAAAATCAAATTCACATGATGTCCATGGTT,ST,CC,pheno
0,0,1,1,1,1,0,1,0,0,1,1,1,0,1,5,5,0
1,0,1,0,1,1,0,1,0,0,1,1,1,0,1,8,8,0
2,0,0,0,0,1,1,1,1,1,0,0,0,1,0,5,5,0
3,0,1,0,1,1,0,1,0,0,1,1,1,0,1,5,5,0
4,0,1,0,1,1,0,1,0,0,1,1,1,0,1,5,5,0


In [10]:
X = df.loc[:, df.columns != 'pheno']
y = df['pheno']
print(X.shape, y.shape)

(253, 17) (253,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is  deprecated in versi

[(0, 217), (1, 217), (2, 217)]


/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is depreca

In [12]:
############# Fully-Connected Neural Network ################

In [13]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [14]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [15]:
dat = pd.DataFrame(X_test_over[:,0])
dat['test'] = y_test_over

In [16]:
dat

,0,test
0,CFBRSa07,0
1,CFBRSa66A,0
2,NRS112,1
3,NRS211,0
4,CFBRSa22,0
...,...,...
191,NRS148,2
192,NRS255,2
193,NRS205,2
194,NRS255,2


In [17]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [18]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [19]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [20]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 315us/step - loss: 11.1134 - accuracy: 0.4769 - val_loss: 5.7433 - val_accuracy: 0.3214
Epoch 2/1000
455/455 [==============================] - 0s 107us/step - loss: 3.8485 - accuracy: 0.4110 - val_loss: 1.7678 - val_accuracy: 0.3622
Epoch 3/1000
455/455 [==============================] - 0s 111us/step - loss: 1.3127 - accuracy: 0.4967 - val_loss: 1.2863 - val_accuracy: 0.5867
Epoch 4/1000
455/455 [==============================] - 0s 144us/step - loss: 1.0764 - accuracy: 0.5297 - val_loss: 0.9065 - val_accuracy: 0.5663
Epoch 5/1000
455/455 [==============================] - 0s 313us/step - loss: 0.9856 - accuracy: 0.5363 - val_loss: 1.4208 - val_accuracy: 0.6378
Epoch 6/1000
455/455 [==============================] - 0s 182us/step - loss: 0.9690 - accuracy: 0.6220 - val_loss: 0.7716 - val_accuracy: 0.5408
Epoch 7/1000
455/455 [==============================] - 0s 178us/step - loss:

Epoch 112/1000
455/455 [==============================] - 0s 130us/step - loss: 0.2796 - accuracy: 0.8989 - val_loss: 0.2547 - val_accuracy: 0.9133
Epoch 113/1000
455/455 [==============================] - 0s 130us/step - loss: 0.2640 - accuracy: 0.9011 - val_loss: 0.2452 - val_accuracy: 0.9082
Epoch 114/1000
455/455 [==============================] - 0s 118us/step - loss: 0.2599 - accuracy: 0.9077 - val_loss: 0.2914 - val_accuracy: 0.9082
Epoch 115/1000
455/455 [==============================] - 0s 179us/step - loss: 0.4477 - accuracy: 0.8593 - val_loss: 0.3970 - val_accuracy: 0.8724
Epoch 116/1000
455/455 [==============================] - 0s 120us/step - loss: 0.4646 - accuracy: 0.8440 - val_loss: 0.7138 - val_accuracy: 0.8571
Epoch 117/1000
455/455 [==============================] - 0s 127us/step - loss: 0.5550 - accuracy: 0.8352 - val_loss: 0.2979 - val_accuracy: 0.9184
Epoch 118/1000
455/455 [==============================] - 0s 112us/step - loss: 0.2762 - accuracy: 0.8989 - val_

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.115649). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


455/455 [==============================] - 0s 197us/step - loss: 0.2297 - accuracy: 0.9121 - val_loss: 0.2290 - val_accuracy: 0.9133
Epoch 162/1000
455/455 [==============================] - 0s 122us/step - loss: 0.2254 - accuracy: 0.9055 - val_loss: 0.2176 - val_accuracy: 0.9082
Epoch 163/1000
455/455 [==============================] - 0s 120us/step - loss: 0.2263 - accuracy: 0.9033 - val_loss: 0.2208 - val_accuracy: 0.9082
Epoch 164/1000
455/455 [==============================] - 0s 107us/step - loss: 0.2602 - accuracy: 0.8967 - val_loss: 0.2194 - val_accuracy: 0.9082
Epoch 165/1000
455/455 [==============================] - 0s 122us/step - loss: 0.2372 - accuracy: 0.9011 - val_loss: 0.2546 - val_accuracy: 0.9133
Epoch 166/1000
455/455 [==============================] - 0s 113us/step - loss: 0.2545 - accuracy: 0.8857 - val_loss: 0.2565 - val_accuracy: 0.9133
Epoch 167/1000
455/455 [==============================] - 0s 118us/step - loss: 0.2477 - accuracy: 0.9033 - val_loss: 0.2263 - 

455/455 [==============================] - 0s 134us/step - loss: 0.2327 - accuracy: 0.9055 - val_loss: 0.2311 - val_accuracy: 0.9133
Epoch 272/1000
455/455 [==============================] - 0s 172us/step - loss: 0.2161 - accuracy: 0.9099 - val_loss: 0.2131 - val_accuracy: 0.9082
Epoch 273/1000
455/455 [==============================] - 0s 155us/step - loss: 0.2115 - accuracy: 0.9011 - val_loss: 0.2191 - val_accuracy: 0.9133
Epoch 274/1000
455/455 [==============================] - 0s 131us/step - loss: 0.2119 - accuracy: 0.9143 - val_loss: 0.2091 - val_accuracy: 0.9082
Epoch 275/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2178 - accuracy: 0.8923 - val_loss: 0.2141 - val_accuracy: 0.9082
Epoch 276/1000
455/455 [==============================] - 0s 92us/step - loss: 0.2368 - accuracy: 0.8989 - val_loss: 0.2523 - val_accuracy: 0.9133
Epoch 277/1000
455/455 [==============================] - 0s 87us/step - loss: 0.2094 - accuracy: 0.9011 - val_loss: 0.2251 - val

...

455/455 [==============================] - 0s 90us/step - loss: 0.1979 - accuracy: 0.9121 - val_loss: 0.2183 - val_accuracy: 0.9082
Epoch 492/1000
455/455 [==============================] - 0s 88us/step - loss: 0.1958 - accuracy: 0.9099 - val_loss: 0.2189 - val_accuracy: 0.9133
Epoch 493/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2023 - accuracy: 0.9187 - val_loss: 0.2235 - val_accuracy: 0.9082
Epoch 494/1000
455/455 [==============================] - 0s 92us/step - loss: 0.1954 - accuracy: 0.9209 - val_loss: 0.2211 - val_accuracy: 0.9184
Epoch 495/1000
455/455 [==============================] - 0s 93us/step - loss: 0.1963 - accuracy: 0.9209 - val_loss: 0.2190 - val_accuracy: 0.9133
Epoch 496/1000
455/455 [==============================] - 0s 90us/step - loss: 0.1965 - accuracy: 0.9165 - val_loss: 0.2182 - val_accuracy: 0.9133
Epoch 497/1000
455/455 [==============================] - 0s 93us/step - loss: 0.2147 - accuracy: 0.9033 - val_loss: 0.2204 - val_acc

455/455 [==============================] - 0s 99us/step - loss: 0.1914 - accuracy: 0.9187 - val_loss: 0.2282 - val_accuracy: 0.9235
Epoch 602/1000
455/455 [==============================] - 0s 101us/step - loss: 0.2768 - accuracy: 0.8879 - val_loss: 0.7679 - val_accuracy: 0.8469
Epoch 603/1000
455/455 [==============================] - 0s 100us/step - loss: 0.3539 - accuracy: 0.8923 - val_loss: 0.2227 - val_accuracy: 0.9082
Epoch 604/1000
455/455 [==============================] - 0s 99us/step - loss: 0.2767 - accuracy: 0.8857 - val_loss: 0.2479 - val_accuracy: 0.9235
Epoch 605/1000
455/455 [==============================] - 0s 106us/step - loss: 0.1998 - accuracy: 0.9187 - val_loss: 0.2121 - val_accuracy: 0.9184
Epoch 606/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1893 - accuracy: 0.9231 - val_loss: 0.2111 - val_accuracy: 0.9184
Epoch 607/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1944 - accuracy: 0.9275 - val_loss: 0.2121 - va

455/455 [==============================] - 0s 103us/step - loss: 0.1977 - accuracy: 0.9275 - val_loss: 0.2310 - val_accuracy: 0.9235
Epoch 712/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1945 - accuracy: 0.9297 - val_loss: 0.2226 - val_accuracy: 0.9184
Epoch 713/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1903 - accuracy: 0.9319 - val_loss: 0.2273 - val_accuracy: 0.9286
Epoch 714/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1852 - accuracy: 0.9275 - val_loss: 0.2013 - val_accuracy: 0.9133
Epoch 715/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1842 - accuracy: 0.9165 - val_loss: 0.2059 - val_accuracy: 0.9235
Epoch 716/1000
455/455 [==============================] - 0s 111us/step - loss: 0.1815 - accuracy: 0.9341 - val_loss: 0.2094 - val_accuracy: 0.9235
Epoch 717/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1958 - accuracy: 0.9297 - val_loss: 0.2065 - 

455/455 [==============================] - 0s 97us/step - loss: 0.1736 - accuracy: 0.9297 - val_loss: 0.2044 - val_accuracy: 0.9235
Epoch 822/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1785 - accuracy: 0.9319 - val_loss: 0.2007 - val_accuracy: 0.9286
Epoch 823/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1783 - accuracy: 0.9275 - val_loss: 0.2060 - val_accuracy: 0.9235
Epoch 824/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1760 - accuracy: 0.9319 - val_loss: 0.2020 - val_accuracy: 0.9235
Epoch 825/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1795 - accuracy: 0.9319 - val_loss: 0.1953 - val_accuracy: 0.9184
Epoch 826/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1831 - accuracy: 0.9275 - val_loss: 0.2402 - val_accuracy: 0.9337
Epoch 827/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1927 - accuracy: 0.9121 - val_loss: 0.2023 - val

455/455 [==============================] - 0s 98us/step - loss: 0.1772 - accuracy: 0.9341 - val_loss: 0.2008 - val_accuracy: 0.9235
Epoch 932/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1812 - accuracy: 0.9231 - val_loss: 0.2079 - val_accuracy: 0.9235
Epoch 933/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1738 - accuracy: 0.9341 - val_loss: 0.1988 - val_accuracy: 0.9235
Epoch 934/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1729 - accuracy: 0.9341 - val_loss: 0.2056 - val_accuracy: 0.9235
Epoch 935/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1742 - accuracy: 0.9319 - val_loss: 0.1959 - val_accuracy: 0.9235
Epoch 936/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1690 - accuracy: 0.9319 - val_loss: 0.1983 - val_accuracy: 0.9235
Epoch 937/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1710 - accuracy: 0.9319 - val_loss: 0.2022 - va

In [29]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

196/196 [==============================] - 0s 55us/step
over-sampling test accuracy: 91.84%


In [21]:
pred = model1_over.predict_classes(X_test_over)
pred

array([0, 0, 1, 0, 0, 1, 0, 0, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 2, 2, 1, 2, 0, 0, 0, 0,
       0, 1, 0, 1, 1, 1, 2, 2, 2, 1, 1, 1, 0, 1, 1, 1, 2, 1, 0, 2, 0, 1,
       0, 1, 2, 1, 0, 1, 0, 2, 2, 0, 1, 2, 1, 1, 0, 0, 1, 1, 0, 1, 2, 1,
       0, 0, 0, 1, 0, 2, 2, 2, 2, 2, 1, 1, 2, 0, 0, 0, 1, 2, 1, 0, 2, 1,
       1, 1, 0, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 2, 1, 0, 1,
       0, 2, 2, 0, 2, 0, 0, 0, 2, 1, 0, 2, 1, 0, 1, 2, 2, 1, 0, 2, 2, 2,
       2, 2, 2, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 0, 2, 1, 2, 2, 2,
       1, 1, 2, 1, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 0, 2, 2, 2, 2, 2])

In [22]:
dat['pred'] = pred
dat

,0,test,pred
0,CFBRSa07,0,0
1,CFBRSa66A,0,0
2,NRS112,1,1
3,NRS211,0,0
4,CFBRSa22,0,0
...,...,...,...
191,NRS148,2,2
192,NRS255,2,2
193,NRS205,2,2
194,NRS255,2,2


In [23]:
proba1 = model1_over.predict_proba(X_test_over)
dat_proba1 = pd.DataFrame(proba1)

In [24]:
dat_proba1

,0,1,2
0,8.769647e-01,1.230353e-01,2.897799e-15
1,8.528891e-01,1.471109e-01,3.635687e-31
2,7.834299e-08,9.999999e-01,1.246762e-16
3,8.031634e-01,1.968366e-01,9.080670e-10
4,9.176830e-01,8.231699e-02,3.812492e-16
...,...,...,...
191,4.306421e-10,9.854259e-05,9.999014e-01
192,9.407130e-05,2.574446e-03,9.973315e-01
193,4.306421e-10,9.854259e-05,9.999014e-01
194,9.407130e-05,2.574446e-03,9.973315e-01


In [25]:
dat_proba1.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba1.csv", index = False,
         header=None)

In [27]:
dat.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/1pyoST.csv", index = False,
         header=None)

In [33]:
hist1_over = model1_over.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 196us/step - loss: 0.2292 - accuracy: 0.9187 - val_loss: 0.2452 - val_accuracy: 0.9235
Epoch 2/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1902 - accuracy: 0.9319 - val_loss: 0.2005 - val_accuracy: 0.9235
Epoch 3/1000
455/455 [==============================] - 0s 95us/step - loss: 0.2190 - accuracy: 0.9165 - val_loss: 0.1977 - val_accuracy: 0.9235
Epoch 4/1000
455/455 [==============================] - 0s 89us/step - loss: 0.2017 - accuracy: 0.9231 - val_loss: 0.2196 - val_accuracy: 0.9286
Epoch 5/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1994 - accuracy: 0.9297 - val_loss: 0.1975 - val_accuracy: 0.9235
Epoch 6/1000
455/455 [==============================] - 0s 91us/step - loss: 0.1959 - accuracy: 0.9297 - val_loss: 0.2066 - val_accuracy: 0.9286
Epoch 7/1000
455/455 [==============================] - 0s 112us/step - loss: 0.18

Epoch 112/1000
455/455 [==============================] - 0s 252us/step - loss: 0.1932 - accuracy: 0.9275 - val_loss: 0.2055 - val_accuracy: 0.9286
Epoch 113/1000
455/455 [==============================] - 0s 216us/step - loss: 0.1862 - accuracy: 0.9341 - val_loss: 0.1916 - val_accuracy: 0.9235
Epoch 114/1000
455/455 [==============================] - 0s 193us/step - loss: 0.1813 - accuracy: 0.9319 - val_loss: 0.1858 - val_accuracy: 0.9235
Epoch 115/1000
455/455 [==============================] - 0s 173us/step - loss: 0.1836 - accuracy: 0.9297 - val_loss: 0.1993 - val_accuracy: 0.9286
Epoch 116/1000
455/455 [==============================] - 0s 163us/step - loss: 0.1968 - accuracy: 0.9275 - val_loss: 0.1860 - val_accuracy: 0.9235
Epoch 117/1000
455/455 [==============================] - 0s 160us/step - loss: 0.1800 - accuracy: 0.9319 - val_loss: 0.1905 - val_accuracy: 0.9235
Epoch 118/1000
455/455 [==============================] - 0s 185us/step - loss: 0.1918 - accuracy: 0.9297 - val_

455/455 [==============================] - 0s 133us/step - loss: 0.1781 - accuracy: 0.9319 - val_loss: 0.1872 - val_accuracy: 0.9235
Epoch 223/1000
455/455 [==============================] - 0s 152us/step - loss: 0.1812 - accuracy: 0.9297 - val_loss: 0.1906 - val_accuracy: 0.9286
Epoch 224/1000
455/455 [==============================] - 0s 213us/step - loss: 0.1798 - accuracy: 0.9341 - val_loss: 0.1797 - val_accuracy: 0.9235
Epoch 225/1000
455/455 [==============================] - 0s 194us/step - loss: 0.1782 - accuracy: 0.9363 - val_loss: 0.1843 - val_accuracy: 0.9235
Epoch 226/1000
455/455 [==============================] - 0s 138us/step - loss: 0.1841 - accuracy: 0.9275 - val_loss: 0.1918 - val_accuracy: 0.9133
Epoch 227/1000
455/455 [==============================] - 0s 204us/step - loss: 0.1821 - accuracy: 0.9341 - val_loss: 0.1826 - val_accuracy: 0.9235
Epoch 228/1000
455/455 [==============================] - 0s 148us/step - loss: 0.1776 - accuracy: 0.9341 - val_loss: 0.1821 - 

455/455 [==============================] - 0s 100us/step - loss: 0.1777 - accuracy: 0.9275 - val_loss: 0.1839 - val_accuracy: 0.9286
Epoch 333/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1856 - accuracy: 0.9143 - val_loss: 0.1915 - val_accuracy: 0.9235
Epoch 334/1000
455/455 [==============================] - 0s 104us/step - loss: 0.1725 - accuracy: 0.9341 - val_loss: 0.1921 - val_accuracy: 0.9286
Epoch 335/1000
455/455 [==============================] - 0s 112us/step - loss: 0.1774 - accuracy: 0.9319 - val_loss: 0.1707 - val_accuracy: 0.9235
Epoch 336/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1778 - accuracy: 0.9319 - val_loss: 0.1820 - val_accuracy: 0.9235
Epoch 337/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1741 - accuracy: 0.9319 - val_loss: 0.1741 - val_accuracy: 0.9235
Epoch 338/1000
455/455 [==============================] - 0s 109us/step - loss: 0.1772 - accuracy: 0.9231 - val_loss: 0.1743 - v

455/455 [==============================] - 0s 138us/step - loss: 0.1653 - accuracy: 0.9297 - val_loss: 0.1966 - val_accuracy: 0.9286
Epoch 443/1000
455/455 [==============================] - 0s 105us/step - loss: 0.1708 - accuracy: 0.9297 - val_loss: 0.1715 - val_accuracy: 0.9235
Epoch 444/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1828 - accuracy: 0.9275 - val_loss: 0.1842 - val_accuracy: 0.9235
Epoch 445/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1788 - accuracy: 0.9297 - val_loss: 0.1999 - val_accuracy: 0.9286
Epoch 446/1000
455/455 [==============================] - 0s 220us/step - loss: 0.1900 - accuracy: 0.9319 - val_loss: 0.1739 - val_accuracy: 0.9235
Epoch 447/1000
455/455 [==============================] - 0s 112us/step - loss: 0.1770 - accuracy: 0.9231 - val_loss: 0.1902 - val_accuracy: 0.9286
Epoch 448/1000
455/455 [==============================] - 0s 93us/step - loss: 0.1748 - accuracy: 0.9319 - val_loss: 0.1714 - va

...

455/455 [==============================] - 0s 91us/step - loss: 0.1732 - accuracy: 0.9297 - val_loss: 0.1648 - val_accuracy: 0.9235
Epoch 664/1000
455/455 [==============================] - 0s 92us/step - loss: 0.1706 - accuracy: 0.9231 - val_loss: 0.1707 - val_accuracy: 0.9286
Epoch 665/1000
455/455 [==============================] - 0s 92us/step - loss: 0.2039 - accuracy: 0.9209 - val_loss: 0.1985 - val_accuracy: 0.9133
Epoch 666/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1696 - accuracy: 0.9297 - val_loss: 0.1831 - val_accuracy: 0.9286
Epoch 667/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1713 - accuracy: 0.9319 - val_loss: 0.1644 - val_accuracy: 0.9235
Epoch 668/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1747 - accuracy: 0.9341 - val_loss: 0.1689 - val_accuracy: 0.9286
Epoch 669/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1765 - accuracy: 0.9297 - val_loss: 0.1689 - val_

Epoch 719/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1798 - accuracy: 0.9231 - val_loss: 0.1707 - val_accuracy: 0.9286
Epoch 720/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1690 - accuracy: 0.9231 - val_loss: 0.1732 - val_accuracy: 0.9235
Epoch 721/1000
455/455 [==============================] - 0s 104us/step - loss: 0.1815 - accuracy: 0.9253 - val_loss: 0.1797 - val_accuracy: 0.9286
Epoch 722/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1720 - accuracy: 0.9253 - val_loss: 0.1793 - val_accuracy: 0.9286
Epoch 723/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1634 - accuracy: 0.9341 - val_loss: 0.2208 - val_accuracy: 0.9337
Epoch 724/1000
455/455 [==============================] - 0s 93us/step - loss: 0.1857 - accuracy: 0.9187 - val_loss: 0.1625 - val_accuracy: 0.9184
Epoch 725/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1727 - accuracy: 0.9297 - val_loss:

Epoch 830/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1610 - accuracy: 0.9231 - val_loss: 0.1758 - val_accuracy: 0.9286
Epoch 831/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1694 - accuracy: 0.9275 - val_loss: 0.1853 - val_accuracy: 0.9286
Epoch 832/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1655 - accuracy: 0.9297 - val_loss: 0.1581 - val_accuracy: 0.9235
Epoch 833/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1611 - accuracy: 0.9341 - val_loss: 0.1645 - val_accuracy: 0.9235
Epoch 834/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1589 - accuracy: 0.9341 - val_loss: 0.2160 - val_accuracy: 0.9337
Epoch 835/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1728 - accuracy: 0.9319 - val_loss: 0.2053 - val_accuracy: 0.9337
Epoch 836/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1979 - accuracy: 0.9319 - val_loss:

Epoch 941/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1575 - accuracy: 0.9341 - val_loss: 0.1627 - val_accuracy: 0.9337
Epoch 942/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1799 - accuracy: 0.9297 - val_loss: 0.1591 - val_accuracy: 0.9184
Epoch 943/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1736 - accuracy: 0.9319 - val_loss: 0.1621 - val_accuracy: 0.9337
Epoch 944/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1593 - accuracy: 0.9319 - val_loss: 0.2398 - val_accuracy: 0.9133
Epoch 945/1000
455/455 [==============================] - 0s 104us/step - loss: 0.1880 - accuracy: 0.9297 - val_loss: 0.1642 - val_accuracy: 0.9337
Epoch 946/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1566 - accuracy: 0.9297 - val_loss: 0.1896 - val_accuracy: 0.9337
Epoch 947/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1784 - accuracy: 0.9297 - val_loss: 

Epoch 997/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1667 - accuracy: 0.9363 - val_loss: 0.1634 - val_accuracy: 0.9337
Epoch 998/1000
455/455 [==============================] - 0s 93us/step - loss: 0.1681 - accuracy: 0.9297 - val_loss: 0.1839 - val_accuracy: 0.9235
Epoch 999/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1614 - accuracy: 0.9319 - val_loss: 0.2217 - val_accuracy: 0.9235
Epoch 1000/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1696 - accuracy: 0.9297 - val_loss: 0.1618 - val_accuracy: 0.9337


In [34]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over.history['accuracy'])*100))

over-sampling train accuracy: 92.93%


In [19]:
df_proba = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=0,
                        index_col=None)

In [20]:
df_proba

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,CFBRSa26,0,0,0.758914,0.241086,4.638713e-07
1,p002ykpresabs_qual,NRS109,2,2,0.005361,0.016236,9.784034e-01
2,p002ykpresabs_qual,NRS112,0,0,0.726623,0.273376,1.520979e-06
3,p002ykpresabs_qual,NRS216,1,1,0.138322,0.861665,1.334123e-05
4,p002ykpresabs_qual,NRS021,0,0,0.882176,0.117824,1.414530e-10
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS148,2,2,0.000007,0.000099,9.998934e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01
4281,pyopresabsSTCC_qual,NRS205,2,2,0.000011,0.000045,9.999435e-01
4282,pyopresabsSTCC_qual,NRS255,2,2,0.000257,0.002048,9.976944e-01


In [21]:
y_prob = df_proba[df_proba['phage']=='pyokpresabsSTCC_qual'].iloc[:,-3:]
y_prob = y_prob.to_numpy()
y_prob

array([[8.76964750e-01, 1.23035280e-01, 2.89779900e-15],
       [8.52889060e-01, 1.47110940e-01, 3.63568720e-31],
       [7.83429850e-08, 9.99999900e-01, 1.24676220e-16],
       [8.03163400e-01, 1.96836590e-01, 9.08067000e-10],
       [9.17683000e-01, 8.23169950e-02, 3.81249160e-16],
       [7.14591700e-02, 9.28207040e-01, 3.33824200e-04],
       [5.62550800e-01, 4.37449250e-01, 7.14945160e-10],
       [9.99639030e-01, 3.61012800e-04, 1.76575900e-14],
       [4.30642080e-10, 9.85425940e-05, 9.99901400e-01],
       [2.96929800e-07, 9.99999760e-01, 2.78088040e-19],
       [8.52889060e-01, 1.47110940e-01, 3.63568720e-31],
       [9.17683000e-01, 8.23169950e-02, 3.81249160e-16],
       [3.33123900e-07, 9.99999640e-01, 7.27607900e-15],
       [4.30642080e-10, 9.85425940e-05, 9.99901400e-01],
       [6.47706450e-01, 3.52293580e-01, 2.73130320e-11],
       [1.00000000e+00, 8.12506050e-12, 8.49675700e-34],
       [2.06158120e-01, 7.93841900e-01, 4.89789900e-34],
       [8.52889060e-01, 1.47110

In [22]:
## Retrieved from https://github.com/scikit-learn/scikit-learn/issues/3298
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

def rocauc_ovo(truth, pred, average="macro", multi_class="ovo"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   
    
    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [23]:
ovo1 = rocauc_ovo(y_test_over, y_prob, average="macro", multi_class="ovo")
ovo1

0.9848111176355451

In [24]:
def rocauc_ovr(truth, pred, average="macro", multi_class="ovr"):

    lb = LabelBinarizer()
    lb.fit(truth)

    truth = lb.transform(truth)   

    return roc_auc_score(truth, pred, average=average, multi_class=multi_class)

In [25]:
ovr1 = rocauc_ovr(y_test_over, y_prob, average="macro", multi_class="ovr")
ovr1

0.9848111176355451

In [26]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=234,
                                                    stratify=y_over)

In [27]:
dat2 = pd.DataFrame(X_test_over[:,0])
dat2['test'] = y_test_over

In [28]:
dat2

,0,test
0,BCH-SA-04,0
1,NRS110,1
2,NRS109,2
3,NRS183,1
4,BCH-SA-05,0
...,...,...
191,NRS112,1
192,SR1065,0
193,NRS203,0
194,CFBREBSa129,0


In [29]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [30]:
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [33]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 329us/step - loss: 19.4843 - accuracy: 0.2791 - val_loss: 10.5747 - val_accuracy: 0.3469
Epoch 2/1000
455/455 [==============================] - 0s 118us/step - loss: 7.0241 - accuracy: 0.3780 - val_loss: 2.3397 - val_accuracy: 0.4439
Epoch 3/1000
455/455 [==============================] - 0s 176us/step - loss: 1.8542 - accuracy: 0.3890 - val_loss: 1.7078 - val_accuracy: 0.3010
Epoch 4/1000
455/455 [==============================] - 0s 219us/step - loss: 1.4589 - accuracy: 0.4549 - val_loss: 1.1332 - val_accuracy: 0.3265
Epoch 5/1000
455/455 [==============================] - 0s 316us/step - loss: 1.1661 - accuracy: 0.4813 - val_loss: 0.8788 - val_accuracy: 0.5561
Epoch 6/1000
455/455 [==============================] - 0s 561us/step - loss: 1.0892 - accuracy: 0.5429 - val_loss: 1.0480 - val_accuracy: 0.3724
Epoch 7/1000
455/455 [==============================] - 0s 767us/step - loss

Epoch 112/1000
455/455 [==============================] - 0s 215us/step - loss: 0.2716 - accuracy: 0.8945 - val_loss: 0.3560 - val_accuracy: 0.8520
Epoch 113/1000
455/455 [==============================] - 0s 117us/step - loss: 0.2759 - accuracy: 0.8857 - val_loss: 0.3723 - val_accuracy: 0.8622
Epoch 114/1000
455/455 [==============================] - 0s 161us/step - loss: 0.2951 - accuracy: 0.8835 - val_loss: 0.3866 - val_accuracy: 0.8520
Epoch 115/1000
455/455 [==============================] - 0s 163us/step - loss: 0.3004 - accuracy: 0.8835 - val_loss: 0.3642 - val_accuracy: 0.8520
Epoch 116/1000
455/455 [==============================] - 0s 147us/step - loss: 0.2722 - accuracy: 0.8835 - val_loss: 0.3499 - val_accuracy: 0.8520
Epoch 117/1000
455/455 [==============================] - 0s 107us/step - loss: 0.2815 - accuracy: 0.8923 - val_loss: 0.3644 - val_accuracy: 0.8673
Epoch 118/1000
455/455 [==============================] - 0s 92us/step - loss: 0.3104 - accuracy: 0.8725 - val_l

Epoch 223/1000
455/455 [==============================] - 0s 92us/step - loss: 0.2315 - accuracy: 0.9033 - val_loss: 0.3351 - val_accuracy: 0.8673
Epoch 224/1000
455/455 [==============================] - 0s 91us/step - loss: 0.2344 - accuracy: 0.8923 - val_loss: 0.3299 - val_accuracy: 0.8724
Epoch 225/1000
455/455 [==============================] - 0s 106us/step - loss: 0.2329 - accuracy: 0.8879 - val_loss: 0.3576 - val_accuracy: 0.8673
Epoch 226/1000
455/455 [==============================] - 0s 112us/step - loss: 0.2336 - accuracy: 0.8989 - val_loss: 0.3684 - val_accuracy: 0.8673
Epoch 227/1000
455/455 [==============================] - 0s 125us/step - loss: 0.2275 - accuracy: 0.8989 - val_loss: 0.3826 - val_accuracy: 0.8673
Epoch 228/1000
455/455 [==============================] - 0s 103us/step - loss: 0.2309 - accuracy: 0.9165 - val_loss: 0.3310 - val_accuracy: 0.8673
Epoch 229/1000
455/455 [==============================] - 0s 104us/step - loss: 0.2270 - accuracy: 0.9033 - val_lo

455/455 [==============================] - 0s 95us/step - loss: 0.2077 - accuracy: 0.9121 - val_loss: 0.3372 - val_accuracy: 0.8673
Epoch 334/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2102 - accuracy: 0.9121 - val_loss: 0.3461 - val_accuracy: 0.8673
Epoch 335/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2147 - accuracy: 0.9077 - val_loss: 0.3564 - val_accuracy: 0.8673
Epoch 336/1000
455/455 [==============================] - 0s 100us/step - loss: 0.2181 - accuracy: 0.8989 - val_loss: 0.3549 - val_accuracy: 0.8827
Epoch 337/1000
455/455 [==============================] - 0s 105us/step - loss: 0.2184 - accuracy: 0.9055 - val_loss: 0.3556 - val_accuracy: 0.8673
Epoch 338/1000
455/455 [==============================] - 0s 102us/step - loss: 0.2200 - accuracy: 0.9011 - val_loss: 0.3610 - val_accuracy: 0.8724
Epoch 339/1000
455/455 [==============================] - 0s 99us/step - loss: 0.2146 - accuracy: 0.9121 - val_loss: 0.3399 - val_

455/455 [==============================] - 0s 96us/step - loss: 0.2227 - accuracy: 0.9077 - val_loss: 0.3811 - val_accuracy: 0.8673
Epoch 444/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2170 - accuracy: 0.9033 - val_loss: 0.3733 - val_accuracy: 0.8571
Epoch 445/1000
455/455 [==============================] - 0s 101us/step - loss: 0.2135 - accuracy: 0.9143 - val_loss: 0.3682 - val_accuracy: 0.8571
Epoch 446/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2092 - accuracy: 0.9099 - val_loss: 0.3707 - val_accuracy: 0.8571
Epoch 447/1000
455/455 [==============================] - 0s 111us/step - loss: 0.2157 - accuracy: 0.9077 - val_loss: 0.3676 - val_accuracy: 0.8571
Epoch 448/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2166 - accuracy: 0.8945 - val_loss: 0.4207 - val_accuracy: 0.8469
Epoch 449/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2188 - accuracy: 0.9077 - val_loss: 0.3708 - val_a

Epoch 499/1000
455/455 [==============================] - 0s 102us/step - loss: 0.2121 - accuracy: 0.9099 - val_loss: 0.3931 - val_accuracy: 0.8673
Epoch 500/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2391 - accuracy: 0.9055 - val_loss: 0.3892 - val_accuracy: 0.8673
Epoch 501/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2060 - accuracy: 0.9099 - val_loss: 0.4059 - val_accuracy: 0.8673
Epoch 502/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2104 - accuracy: 0.9099 - val_loss: 0.3746 - val_accuracy: 0.8673
Epoch 503/1000
455/455 [==============================] - 0s 106us/step - loss: 0.2086 - accuracy: 0.9099 - val_loss: 0.3844 - val_accuracy: 0.8673
Epoch 504/1000
455/455 [==============================] - 0s 126us/step - loss: 0.2112 - accuracy: 0.9033 - val_loss: 0.3848 - val_accuracy: 0.8622
Epoch 505/1000
455/455 [==============================] - 0s 107us/step - loss: 0.2107 - accuracy: 0.9077 - val_los

455/455 [==============================] - 0s 99us/step - loss: 0.1931 - accuracy: 0.9011 - val_loss: 0.5933 - val_accuracy: 0.8571
Epoch 610/1000
455/455 [==============================] - 0s 102us/step - loss: 0.2511 - accuracy: 0.9011 - val_loss: 0.4144 - val_accuracy: 0.8673
Epoch 611/1000
455/455 [==============================] - 0s 100us/step - loss: 0.2058 - accuracy: 0.8967 - val_loss: 0.4330 - val_accuracy: 0.8673
Epoch 612/1000
455/455 [==============================] - 0s 101us/step - loss: 0.2217 - accuracy: 0.9077 - val_loss: 0.4195 - val_accuracy: 0.8673
Epoch 613/1000
455/455 [==============================] - 0s 129us/step - loss: 0.1960 - accuracy: 0.8923 - val_loss: 0.3999 - val_accuracy: 0.8673
Epoch 614/1000
455/455 [==============================] - 0s 104us/step - loss: 0.2046 - accuracy: 0.9099 - val_loss: 0.3888 - val_accuracy: 0.8571
Epoch 615/1000
455/455 [==============================] - 0s 102us/step - loss: 0.2114 - accuracy: 0.8967 - val_loss: 0.4120 - v

Epoch 720/1000
455/455 [==============================] - 0s 100us/step - loss: 0.2133 - accuracy: 0.9143 - val_loss: 0.4438 - val_accuracy: 0.8571
Epoch 721/1000
455/455 [==============================] - 0s 99us/step - loss: 0.2016 - accuracy: 0.9055 - val_loss: 0.4212 - val_accuracy: 0.8724
Epoch 722/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1953 - accuracy: 0.9033 - val_loss: 0.4306 - val_accuracy: 0.8724
Epoch 723/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1980 - accuracy: 0.9099 - val_loss: 0.4361 - val_accuracy: 0.8724
Epoch 724/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2008 - accuracy: 0.9077 - val_loss: 0.4132 - val_accuracy: 0.8571
Epoch 725/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1923 - accuracy: 0.9055 - val_loss: 0.4155 - val_accuracy: 0.8571
Epoch 726/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1954 - accuracy: 0.9165 - val_loss:

455/455 [==============================] - 0s 109us/step - loss: 0.7165 - accuracy: 0.8484 - val_loss: 1.0945 - val_accuracy: 0.7959
Epoch 831/1000
455/455 [==============================] - 0s 101us/step - loss: 0.3465 - accuracy: 0.8901 - val_loss: 0.4276 - val_accuracy: 0.8673
Epoch 832/1000
455/455 [==============================] - 0s 107us/step - loss: 0.2004 - accuracy: 0.9011 - val_loss: 0.4358 - val_accuracy: 0.8724
Epoch 833/1000
455/455 [==============================] - 0s 100us/step - loss: 0.2499 - accuracy: 0.8923 - val_loss: 0.5108 - val_accuracy: 0.8827
Epoch 834/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2547 - accuracy: 0.8923 - val_loss: 0.4347 - val_accuracy: 0.8724
Epoch 835/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1935 - accuracy: 0.9099 - val_loss: 0.4350 - val_accuracy: 0.8724
Epoch 836/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1986 - accuracy: 0.9055 - val_loss: 0.4760 - va

455/455 [==============================] - 0s 97us/step - loss: 0.1965 - accuracy: 0.9055 - val_loss: 0.4693 - val_accuracy: 0.8724
Epoch 941/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1923 - accuracy: 0.9165 - val_loss: 0.4499 - val_accuracy: 0.8520
Epoch 942/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1888 - accuracy: 0.9143 - val_loss: 0.4459 - val_accuracy: 0.8827
Epoch 943/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1864 - accuracy: 0.9165 - val_loss: 0.4478 - val_accuracy: 0.8827
Epoch 944/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1939 - accuracy: 0.9077 - val_loss: 0.4685 - val_accuracy: 0.8724
Epoch 945/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1842 - accuracy: 0.9187 - val_loss: 0.4377 - val_accuracy: 0.8724
Epoch 946/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1911 - accuracy: 0.9165 - val_loss: 0.4473 - val

In [50]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

196/196 [==============================] - 0s 63us/step
over-sampling test accuracy: 89.80%


In [35]:
pred2 = model1_over2.predict_classes(X_test_over)
pred2

array([0, 1, 2, 0, 0, 0, 2, 2, 2, 2, 1, 0, 0, 2, 0, 0, 2, 1, 2, 1, 2, 0,
       2, 1, 0, 1, 0, 0, 2, 2, 2, 1, 1, 0, 0, 2, 0, 2, 1, 0, 1, 1, 1, 2,
       1, 2, 0, 1, 2, 1, 2, 0, 1, 0, 0, 1, 2, 1, 1, 1, 2, 0, 1, 1, 1, 0,
       0, 2, 1, 2, 1, 2, 2, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 2, 0, 1, 1, 1,
       2, 1, 1, 0, 1, 1, 0, 2, 1, 2, 1, 1, 2, 0, 2, 1, 0, 0, 0, 1, 2, 2,
       0, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 1, 0, 1, 2, 0, 1, 1, 2, 2,
       0, 2, 1, 2, 2, 1, 1, 2, 1, 0, 1, 2, 2, 1, 1, 0, 0, 2, 2, 2, 1, 1,
       0, 2, 1, 2, 1, 0, 2, 1, 2, 1, 0, 1, 2, 1, 2, 0, 1, 0, 1, 2, 0, 2,
       1, 0, 0, 0, 0, 1, 2, 2, 1, 0, 2, 1, 2, 0, 2, 1, 0, 0, 0, 0])

In [36]:
dat2['pred'] = pred2
dat2

,0,test,pred
0,BCH-SA-04,0,0
1,NRS110,1,1
2,NRS109,2,2
3,NRS183,1,0
4,BCH-SA-05,0,0
...,...,...,...
191,NRS112,1,1
192,SR1065,0,0
193,NRS203,0,0
194,CFBREBSa129,0,0


In [37]:
proba2 = model1_over2.predict_proba(X_test_over)
dat_proba2 = pd.DataFrame(proba2)

In [38]:
dat_proba2

,0,1,2
0,9.999977e-01,0.000002,3.515766e-22
1,1.033491e-04,0.999872,2.419857e-05
2,1.951965e-09,0.000106,9.998940e-01
3,6.812023e-01,0.317383,1.414624e-03
4,8.766014e-01,0.123399,3.313500e-08
...,...,...,...
191,1.044573e-10,0.999555,4.447276e-04
192,9.793976e-01,0.020602,5.908110e-12
193,9.999974e-01,0.000003,4.468452e-13
194,8.632075e-01,0.134663,2.129702e-03


In [39]:
dat_proba2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba2.csv", index = False,
         header=None)

In [40]:
dat2.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/2pyoST.csv", index = False,
         header=None)

In [54]:
hist1_over2 = model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 220us/step - loss: 0.1859 - accuracy: 0.9165 - val_loss: 0.4384 - val_accuracy: 0.9031
Epoch 2/1000
455/455 [==============================] - 0s 194us/step - loss: 0.1696 - accuracy: 0.9319 - val_loss: 0.4386 - val_accuracy: 0.9031
Epoch 3/1000
455/455 [==============================] - 0s 166us/step - loss: 0.1657 - accuracy: 0.9275 - val_loss: 0.4362 - val_accuracy: 0.9031
Epoch 4/1000
455/455 [==============================] - 0s 187us/step - loss: 0.1717 - accuracy: 0.9297 - val_loss: 0.4491 - val_accuracy: 0.9031
Epoch 5/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1798 - accuracy: 0.9275 - val_loss: 0.4623 - val_accuracy: 0.9031
Epoch 6/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1828 - accuracy: 0.9077 - val_loss: 0.4547 - val_accuracy: 0.9031
Epoch 7/1000
455/455 [==============================] - 0s 108us/step - loss: 

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.141721). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


455/455 [==============================] - 0s 977us/step - loss: 0.1605 - accuracy: 0.9341 - val_loss: 0.4781 - val_accuracy: 0.8724
Epoch 27/1000
455/455 [==============================] - 0s 497us/step - loss: 0.1884 - accuracy: 0.9275 - val_loss: 0.4592 - val_accuracy: 0.9031
Epoch 28/1000
455/455 [==============================] - 0s 275us/step - loss: 0.1697 - accuracy: 0.9253 - val_loss: 0.4599 - val_accuracy: 0.9031
Epoch 29/1000
455/455 [==============================] - 0s 171us/step - loss: 0.1667 - accuracy: 0.9253 - val_loss: 0.4443 - val_accuracy: 0.9031
Epoch 30/1000
455/455 [==============================] - 0s 187us/step - loss: 0.1756 - accuracy: 0.9253 - val_loss: 0.6604 - val_accuracy: 0.8776
Epoch 31/1000
455/455 [==============================] - 0s 167us/step - loss: 0.4723 - accuracy: 0.8813 - val_loss: 0.4980 - val_accuracy: 0.8724
Epoch 32/1000
455/455 [==============================] - 0s 166us/step - loss: 0.4613 - accuracy: 0.8725 - val_loss: 0.4776 - val_ac

Epoch 82/1000
455/455 [==============================] - 0s 206us/step - loss: 0.1649 - accuracy: 0.9319 - val_loss: 0.4522 - val_accuracy: 0.9031
Epoch 83/1000
455/455 [==============================] - 0s 197us/step - loss: 0.1622 - accuracy: 0.9297 - val_loss: 0.4547 - val_accuracy: 0.9031
Epoch 84/1000
455/455 [==============================] - 0s 204us/step - loss: 0.1632 - accuracy: 0.9319 - val_loss: 0.4670 - val_accuracy: 0.9031
Epoch 85/1000
455/455 [==============================] - 0s 235us/step - loss: 0.1658 - accuracy: 0.9319 - val_loss: 0.4793 - val_accuracy: 0.9031
Epoch 86/1000
455/455 [==============================] - 0s 203us/step - loss: 0.1704 - accuracy: 0.9231 - val_loss: 0.4611 - val_accuracy: 0.9031
Epoch 87/1000
455/455 [==============================] - 0s 188us/step - loss: 0.1661 - accuracy: 0.9231 - val_loss: 0.4560 - val_accuracy: 0.9031
Epoch 88/1000
455/455 [==============================] - 0s 179us/step - loss: 0.1633 - accuracy: 0.9275 - val_loss: 0

455/455 [==============================] - 0s 115us/step - loss: 0.1636 - accuracy: 0.9319 - val_loss: 0.4655 - val_accuracy: 0.9031
Epoch 193/1000
455/455 [==============================] - 0s 122us/step - loss: 0.1660 - accuracy: 0.9341 - val_loss: 0.4627 - val_accuracy: 0.9031
Epoch 194/1000
455/455 [==============================] - 0s 131us/step - loss: 0.1600 - accuracy: 0.9231 - val_loss: 0.4639 - val_accuracy: 0.9031
Epoch 195/1000
455/455 [==============================] - 0s 124us/step - loss: 0.1640 - accuracy: 0.9319 - val_loss: 0.4810 - val_accuracy: 0.9031
Epoch 196/1000
455/455 [==============================] - 0s 123us/step - loss: 0.1638 - accuracy: 0.9231 - val_loss: 0.4770 - val_accuracy: 0.9031
Epoch 197/1000
455/455 [==============================] - 0s 123us/step - loss: 0.1641 - accuracy: 0.9341 - val_loss: 0.4804 - val_accuracy: 0.9031
Epoch 198/1000
455/455 [==============================] - 0s 166us/step - loss: 0.1603 - accuracy: 0.9253 - val_loss: 0.4627 - 

...

Epoch 579/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1495 - accuracy: 0.9363 - val_loss: 0.5361 - val_accuracy: 0.9031
Epoch 580/1000
455/455 [==============================] - 0s 94us/step - loss: 0.1484 - accuracy: 0.9341 - val_loss: 0.5372 - val_accuracy: 0.9031
Epoch 581/1000
455/455 [==============================] - 0s 105us/step - loss: 0.1638 - accuracy: 0.9319 - val_loss: 0.5456 - val_accuracy: 0.9031
Epoch 582/1000
455/455 [==============================] - 0s 109us/step - loss: 0.1499 - accuracy: 0.9341 - val_loss: 0.5601 - val_accuracy: 0.9031
Epoch 583/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1467 - accuracy: 0.9385 - val_loss: 0.5388 - val_accuracy: 0.9031
Epoch 584/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1533 - accuracy: 0.9385 - val_loss: 0.5386 - val_accuracy: 0.9031
Epoch 585/1000
455/455 [==============================] - 0s 106us/step - loss: 0.1498 - accuracy: 0.9385 - val_los

Epoch 690/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1477 - accuracy: 0.9429 - val_loss: 0.5834 - val_accuracy: 0.9031
Epoch 691/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1647 - accuracy: 0.9253 - val_loss: 0.5639 - val_accuracy: 0.9031
Epoch 692/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1537 - accuracy: 0.9363 - val_loss: 0.5612 - val_accuracy: 0.9031
Epoch 693/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1505 - accuracy: 0.9341 - val_loss: 0.5991 - val_accuracy: 0.9031
Epoch 694/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1563 - accuracy: 0.9319 - val_loss: 0.5527 - val_accuracy: 0.9031
Epoch 695/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1514 - accuracy: 0.9363 - val_loss: 0.5442 - val_accuracy: 0.9031
Epoch 696/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1493 - accuracy: 0.9385 - val_loss

455/455 [==============================] - 0s 96us/step - loss: 0.1473 - accuracy: 0.9363 - val_loss: 0.5709 - val_accuracy: 0.9031
Epoch 801/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1458 - accuracy: 0.9385 - val_loss: 0.5706 - val_accuracy: 0.9031
Epoch 802/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1483 - accuracy: 0.9297 - val_loss: 0.5711 - val_accuracy: 0.9031
Epoch 803/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1486 - accuracy: 0.9363 - val_loss: 0.5739 - val_accuracy: 0.9031
Epoch 804/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1470 - accuracy: 0.9341 - val_loss: 0.5645 - val_accuracy: 0.9031
Epoch 805/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1530 - accuracy: 0.9319 - val_loss: 0.5696 - val_accuracy: 0.9031
Epoch 806/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1686 - accuracy: 0.9341 - val_loss: 0.5993 - val_

Epoch 856/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1491 - accuracy: 0.9385 - val_loss: 0.6055 - val_accuracy: 0.9031
Epoch 857/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1535 - accuracy: 0.9385 - val_loss: 0.5983 - val_accuracy: 0.9031
Epoch 858/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1665 - accuracy: 0.9319 - val_loss: 0.5695 - val_accuracy: 0.9031
Epoch 859/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1504 - accuracy: 0.9363 - val_loss: 0.5783 - val_accuracy: 0.9031
Epoch 860/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1495 - accuracy: 0.9341 - val_loss: 0.6039 - val_accuracy: 0.9031
Epoch 861/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1674 - accuracy: 0.9319 - val_loss: 0.5698 - val_accuracy: 0.9031
Epoch 862/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1498 - accuracy: 0.9319 - val_loss: 

Epoch 967/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1725 - accuracy: 0.9297 - val_loss: 0.6501 - val_accuracy: 0.8980
Epoch 968/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1978 - accuracy: 0.9297 - val_loss: 0.5716 - val_accuracy: 0.9031
Epoch 969/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1587 - accuracy: 0.9341 - val_loss: 0.5769 - val_accuracy: 0.8980
Epoch 970/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1482 - accuracy: 0.9363 - val_loss: 0.5868 - val_accuracy: 0.8980
Epoch 971/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1488 - accuracy: 0.9319 - val_loss: 0.5849 - val_accuracy: 0.9031
Epoch 972/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1599 - accuracy: 0.9253 - val_loss: 0.6076 - val_accuracy: 0.9031
Epoch 973/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1474 - accuracy: 0.9385 - val_loss: 

In [55]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over2.history['accuracy'])*100))

over-sampling train accuracy: 93.19%


In [31]:
df_proba2 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=1,
                        index_col=None)

In [32]:
df_proba2

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS148,2,2,0.000056,1.748042e-03,9.981960e-01
1,p002ykpresabs_qual,BCH-SA-03,1,0,0.712007,2.879924e-01,9.646217e-07
2,p002ykpresabs_qual,NRS218,1,1,0.006222,9.937732e-01,4.482882e-06
3,p002ykpresabs_qual,NRS036,0,0,0.882617,1.173831e-01,2.310933e-10
4,p002ykpresabs_qual,NRS386,1,0,0.571179,4.288184e-01,2.444667e-06
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS112,1,1,0.001860,9.979747e-01,1.653396e-04
4280,pyopresabsSTCC_qual,SR1065,0,0,0.982940,1.705227e-02,7.349168e-06
4281,pyopresabsSTCC_qual,NRS203,0,0,0.997093,1.962516e-03,9.441347e-04
4282,pyopresabsSTCC_qual,CFBREBSa129,0,0,1.000000,3.031141e-13,3.208205e-09


In [33]:
y_prob2 = df_proba2[df_proba2['phage']=='pyokpresabsSTCC_qual'].iloc[:,-3:]
y_prob2 = y_prob2.to_numpy()
y_prob2

array([[9.99997740e-01, 2.30272580e-06, 3.51576570e-22],
       [1.03349150e-04, 9.99872450e-01, 2.41985730e-05],
       [1.95196500e-09, 1.05957190e-04, 9.99894000e-01],
       [6.81202300e-01, 3.17383080e-01, 1.41462370e-03],
       [8.76601400e-01, 1.23398630e-01, 3.31349950e-08],
       [9.94888100e-01, 5.11189100e-03, 1.99745190e-10],
       [3.91925530e-05, 2.87736500e-03, 9.97083500e-01],
       [2.65715370e-10, 2.99384160e-08, 1.00000000e+00],
       [1.95196500e-09, 1.05957190e-04, 9.99894000e-01],
       [2.65715370e-10, 2.99384160e-08, 1.00000000e+00],
       [1.38363460e-03, 9.98560250e-01, 5.60354130e-05],
       [9.62238130e-01, 3.77608000e-02, 1.17010730e-06],
       [6.96110370e-01, 3.03889660e-01, 1.66666370e-09],
       [1.95196500e-09, 1.05957190e-04, 9.99894000e-01],
       [9.29665900e-01, 7.03340400e-02, 5.23632400e-08],
       [9.74624630e-01, 2.53172350e-02, 5.81011630e-05],
       [2.65715370e-10, 2.99384160e-08, 1.00000000e+00],
       [1.38363460e-03, 9.98560

In [34]:
ovo2 = rocauc_ovo(y_test_over, y_prob2, average="macro", multi_class="ovo")
ovo2

0.9616266896801248

In [35]:
ovr2 = rocauc_ovr(y_test_over, y_prob2, average="macro", multi_class="ovr")
ovr2

0.9616266896801248

In [36]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=345,
                                                    stratify=y_over)

In [37]:
dat3 = pd.DataFrame(X_test_over[:,0])
dat3['test'] = y_test_over

In [38]:
dat3

,0,test
0,NRS168,1
1,NRS383,1
2,NRS148,2
3,NRS109,2
4,NRS213,0
...,...,...
191,NRS255,2
192,NRS255,2
193,NRS266,1
194,NRS001,1


In [39]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [45]:
model1_over3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [46]:
model1_over3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [47]:
model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 245us/step - loss: 23.7110 - accuracy: 0.4967 - val_loss: 23.9599 - val_accuracy: 0.4898
Epoch 2/1000
455/455 [==============================] - 0s 82us/step - loss: 16.3984 - accuracy: 0.5451 - val_loss: 15.0462 - val_accuracy: 0.5000
Epoch 3/1000
455/455 [==============================] - 0s 46us/step - loss: 9.9917 - accuracy: 0.4923 - val_loss: 6.3333 - val_accuracy: 0.5408
Epoch 4/1000
455/455 [==============================] - 0s 59us/step - loss: 4.1562 - accuracy: 0.4879 - val_loss: 4.2030 - val_accuracy: 0.4133
Epoch 5/1000
455/455 [==============================] - 0s 110us/step - loss: 3.5407 - accuracy: 0.4505 - val_loss: 3.0796 - val_accuracy: 0.6173
Epoch 6/1000
455/455 [==============================] - 0s 89us/step - loss: 2.9134 - accuracy: 0.5692 - val_loss: 2.1385 - val_accuracy: 0.4439
Epoch 7/1000
455/455 [==============================] - 0s 88us/step - loss: 2

Epoch 57/1000
455/455 [==============================] - 0s 40us/step - loss: 0.5562 - accuracy: 0.8198 - val_loss: 0.5933 - val_accuracy: 0.8163
Epoch 58/1000
455/455 [==============================] - 0s 72us/step - loss: 0.5333 - accuracy: 0.8176 - val_loss: 0.6114 - val_accuracy: 0.8061
Epoch 59/1000
455/455 [==============================] - 0s 173us/step - loss: 0.5224 - accuracy: 0.8242 - val_loss: 0.5833 - val_accuracy: 0.8316
Epoch 60/1000
455/455 [==============================] - 0s 242us/step - loss: 0.5254 - accuracy: 0.8242 - val_loss: 0.5732 - val_accuracy: 0.8316
Epoch 61/1000
455/455 [==============================] - 0s 135us/step - loss: 0.5263 - accuracy: 0.8242 - val_loss: 0.5750 - val_accuracy: 0.8316
Epoch 62/1000
455/455 [==============================] - 0s 66us/step - loss: 0.5278 - accuracy: 0.8154 - val_loss: 0.6126 - val_accuracy: 0.8622
Epoch 63/1000
455/455 [==============================] - 0s 74us/step - loss: 0.5290 - accuracy: 0.8418 - val_loss: 0.675

Epoch 113/1000
455/455 [==============================] - 0s 56us/step - loss: 0.3920 - accuracy: 0.8615 - val_loss: 0.4380 - val_accuracy: 0.8367
Epoch 114/1000
455/455 [==============================] - 0s 77us/step - loss: 0.3905 - accuracy: 0.8593 - val_loss: 0.8440 - val_accuracy: 0.6684
Epoch 115/1000
455/455 [==============================] - 0s 41us/step - loss: 1.4203 - accuracy: 0.6703 - val_loss: 1.4747 - val_accuracy: 0.7551
Epoch 116/1000
455/455 [==============================] - 0s 39us/step - loss: 1.2066 - accuracy: 0.7648 - val_loss: 1.8224 - val_accuracy: 0.8010
Epoch 117/1000
455/455 [==============================] - 0s 50us/step - loss: 0.6839 - accuracy: 0.8154 - val_loss: 0.7224 - val_accuracy: 0.6327
Epoch 118/1000
455/455 [==============================] - 0s 38us/step - loss: 0.5991 - accuracy: 0.7363 - val_loss: 0.6103 - val_accuracy: 0.8163
Epoch 119/1000
455/455 [==============================] - 0s 36us/step - loss: 0.4659 - accuracy: 0.8088 - val_loss: 0

Epoch 169/1000
455/455 [==============================] - 0s 42us/step - loss: 0.3096 - accuracy: 0.8681 - val_loss: 0.3873 - val_accuracy: 0.8622
Epoch 170/1000
455/455 [==============================] - 0s 37us/step - loss: 0.3248 - accuracy: 0.8747 - val_loss: 0.3908 - val_accuracy: 0.8469
Epoch 171/1000
455/455 [==============================] - 0s 49us/step - loss: 0.3162 - accuracy: 0.8791 - val_loss: 0.3912 - val_accuracy: 0.8520
Epoch 172/1000
455/455 [==============================] - 0s 36us/step - loss: 0.3094 - accuracy: 0.8769 - val_loss: 0.3771 - val_accuracy: 0.8622
Epoch 173/1000
455/455 [==============================] - 0s 43us/step - loss: 0.3053 - accuracy: 0.8791 - val_loss: 0.3863 - val_accuracy: 0.8520
Epoch 174/1000
455/455 [==============================] - 0s 38us/step - loss: 0.3071 - accuracy: 0.8769 - val_loss: 0.3813 - val_accuracy: 0.8622
Epoch 175/1000
455/455 [==============================] - 0s 44us/step - loss: 0.3036 - accuracy: 0.8747 - val_loss: 0

Epoch 225/1000
455/455 [==============================] - 0s 52us/step - loss: 0.2704 - accuracy: 0.8901 - val_loss: 0.3570 - val_accuracy: 0.8673
Epoch 226/1000
455/455 [==============================] - 0s 44us/step - loss: 0.2720 - accuracy: 0.8857 - val_loss: 0.3488 - val_accuracy: 0.8673
Epoch 227/1000
455/455 [==============================] - 0s 52us/step - loss: 0.2701 - accuracy: 0.8989 - val_loss: 0.3623 - val_accuracy: 0.8673
Epoch 228/1000
455/455 [==============================] - 0s 51us/step - loss: 0.2714 - accuracy: 0.8945 - val_loss: 0.3583 - val_accuracy: 0.8622
Epoch 229/1000
455/455 [==============================] - 0s 122us/step - loss: 0.2746 - accuracy: 0.8923 - val_loss: 0.3495 - val_accuracy: 0.8673
Epoch 230/1000
455/455 [==============================] - 0s 121us/step - loss: 0.2654 - accuracy: 0.8989 - val_loss: 0.3539 - val_accuracy: 0.8622
Epoch 231/1000
455/455 [==============================] - 0s 72us/step - loss: 0.2643 - accuracy: 0.8989 - val_loss:

...

Epoch 672/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1963 - accuracy: 0.9275 - val_loss: 0.3305 - val_accuracy: 0.8571
Epoch 673/1000
455/455 [==============================] - 0s 44us/step - loss: 0.1929 - accuracy: 0.9253 - val_loss: 0.3194 - val_accuracy: 0.8469
Epoch 674/1000
455/455 [==============================] - 0s 46us/step - loss: 0.2091 - accuracy: 0.9253 - val_loss: 0.3553 - val_accuracy: 0.8622
Epoch 675/1000
455/455 [==============================] - 0s 48us/step - loss: 0.1921 - accuracy: 0.9341 - val_loss: 0.3200 - val_accuracy: 0.8571
Epoch 676/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1864 - accuracy: 0.9319 - val_loss: 0.3387 - val_accuracy: 0.8673
Epoch 677/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1953 - accuracy: 0.9275 - val_loss: 0.3122 - val_accuracy: 0.8469
Epoch 678/1000
455/455 [==============================] - 0s 49us/step - loss: 0.1920 - accuracy: 0.9231 - val_loss: 0

Epoch 728/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1868 - accuracy: 0.9253 - val_loss: 0.3284 - val_accuracy: 0.8520
Epoch 729/1000
455/455 [==============================] - 0s 39us/step - loss: 0.1848 - accuracy: 0.9231 - val_loss: 0.3263 - val_accuracy: 0.8571
Epoch 730/1000
455/455 [==============================] - 0s 49us/step - loss: 0.1838 - accuracy: 0.9253 - val_loss: 0.3295 - val_accuracy: 0.8673
Epoch 731/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1834 - accuracy: 0.9275 - val_loss: 0.3221 - val_accuracy: 0.8673
Epoch 732/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1894 - accuracy: 0.9297 - val_loss: 0.3245 - val_accuracy: 0.8571
Epoch 733/1000
455/455 [==============================] - 0s 44us/step - loss: 0.1873 - accuracy: 0.9275 - val_loss: 0.3458 - val_accuracy: 0.8520
Epoch 734/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1975 - accuracy: 0.9231 - val_loss: 0

Epoch 839/1000
455/455 [==============================] - 0s 48us/step - loss: 0.1803 - accuracy: 0.9341 - val_loss: 0.3361 - val_accuracy: 0.8571
Epoch 840/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1786 - accuracy: 0.9297 - val_loss: 0.3236 - val_accuracy: 0.8673
Epoch 841/1000
455/455 [==============================] - 0s 49us/step - loss: 0.1893 - accuracy: 0.9319 - val_loss: 0.3281 - val_accuracy: 0.8571
Epoch 842/1000
455/455 [==============================] - 0s 51us/step - loss: 0.1927 - accuracy: 0.9209 - val_loss: 0.3218 - val_accuracy: 0.8673
Epoch 843/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1773 - accuracy: 0.9319 - val_loss: 0.3438 - val_accuracy: 0.8724
Epoch 844/1000
455/455 [==============================] - 0s 47us/step - loss: 0.1799 - accuracy: 0.9363 - val_loss: 0.3208 - val_accuracy: 0.8520
Epoch 845/1000
455/455 [==============================] - 0s 90us/step - loss: 0.1916 - accuracy: 0.9253 - val_loss: 0

Epoch 895/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1837 - accuracy: 0.9231 - val_loss: 0.3106 - val_accuracy: 0.8469
Epoch 896/1000
455/455 [==============================] - 0s 44us/step - loss: 0.1874 - accuracy: 0.9297 - val_loss: 0.3426 - val_accuracy: 0.8622
Epoch 897/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1786 - accuracy: 0.9363 - val_loss: 0.3121 - val_accuracy: 0.8622
Epoch 898/1000
455/455 [==============================] - 0s 50us/step - loss: 0.1792 - accuracy: 0.9297 - val_loss: 0.3454 - val_accuracy: 0.8673
Epoch 899/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1793 - accuracy: 0.9363 - val_loss: 0.3222 - val_accuracy: 0.8571
Epoch 900/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1947 - accuracy: 0.9297 - val_loss: 0.3971 - val_accuracy: 0.8571
Epoch 901/1000
455/455 [==============================] - 0s 45us/step - loss: 0.2160 - accuracy: 0.9187 - val_loss: 0

Epoch 951/1000
455/455 [==============================] - 0s 45us/step - loss: 0.3557 - accuracy: 0.9011 - val_loss: 0.6344 - val_accuracy: 0.8673
Epoch 952/1000
455/455 [==============================] - 0s 49us/step - loss: 0.3428 - accuracy: 0.8681 - val_loss: 0.7449 - val_accuracy: 0.8418
Epoch 953/1000
455/455 [==============================] - 0s 47us/step - loss: 0.2740 - accuracy: 0.9209 - val_loss: 0.3081 - val_accuracy: 0.8622
Epoch 954/1000
455/455 [==============================] - 0s 47us/step - loss: 0.2122 - accuracy: 0.9275 - val_loss: 0.3472 - val_accuracy: 0.8622
Epoch 955/1000
455/455 [==============================] - 0s 47us/step - loss: 0.1848 - accuracy: 0.9275 - val_loss: 0.3176 - val_accuracy: 0.8776
Epoch 956/1000
455/455 [==============================] - 0s 46us/step - loss: 0.2011 - accuracy: 0.9319 - val_loss: 0.3586 - val_accuracy: 0.8827
Epoch 957/1000
455/455 [==============================] - 0s 44us/step - loss: 0.1882 - accuracy: 0.9165 - val_loss: 0

In [75]:
acc_test_over3 = model1_over3.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over3*100))

196/196 [==============================] - 0s 64us/step
over-sampling test accuracy: 87.76%


In [48]:
pred3 = model1_over3.predict_classes(X_test_over)
pred3

array([1, 1, 2, 2, 0, 0, 0, 2, 1, 0, 0, 2, 1, 1, 1, 1, 0, 0, 2, 2, 1, 1,
       0, 2, 0, 1, 2, 1, 1, 0, 0, 2, 1, 2, 1, 1, 1, 2, 1, 1, 0, 2, 1, 2,
       0, 1, 2, 1, 1, 2, 0, 2, 1, 1, 0, 2, 0, 0, 2, 1, 2, 1, 0, 0, 2, 1,
       1, 0, 2, 0, 1, 0, 1, 0, 0, 2, 0, 1, 0, 0, 2, 0, 0, 2, 0, 2, 0, 1,
       2, 1, 2, 1, 0, 2, 0, 1, 2, 2, 1, 2, 2, 1, 2, 2, 0, 0, 2, 1, 2, 2,
       1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0,
       1, 2, 2, 1, 2, 2, 2, 0, 1, 0, 0, 2, 1, 1, 1, 2, 0, 2, 2, 2, 2, 1,
       0, 1, 2, 0, 1, 1, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 1, 1, 2, 0, 1, 2,
       2, 0, 2, 1, 1, 2, 0, 2, 1, 1, 1, 1, 0, 0, 2, 2, 2, 1, 1, 1])

In [49]:
dat3['pred'] = pred3
dat3

,0,test,pred
0,NRS168,1,1
1,NRS383,1,1
2,NRS148,2,2
3,NRS109,2,2
4,NRS213,0,0
...,...,...,...
191,NRS255,2,2
192,NRS255,2,2
193,NRS266,1,1
194,NRS001,1,1


In [50]:
proba3 = model1_over3.predict_proba(X_test_over)
dat_proba3 = pd.DataFrame(proba3)

In [51]:
dat_proba3

,0,1,2
0,7.290077e-05,0.999901,2.562677e-05
1,3.031491e-04,0.999697,1.020154e-14
2,6.348556e-05,0.001806,9.981303e-01
3,5.384807e-07,0.000350,9.996489e-01
4,7.222769e-01,0.277717,6.008372e-06
...,...,...,...
191,1.051729e-03,0.006413,9.925351e-01
192,1.051729e-03,0.006413,9.925351e-01
193,1.802318e-02,0.981923,5.351173e-05
194,1.943948e-01,0.805605,1.856581e-11


In [52]:
dat_proba3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba3.csv", index = False,
         header=None)

In [53]:
dat3.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/3pyoST.csv", index = False,
         header=None)

In [79]:
hist1_over3 = model1_over3.fit(X_train_over, y_train_over,
          batch_size=64, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 53us/step - loss: 0.2030 - accuracy: 0.9165 - val_loss: 0.3236 - val_accuracy: 0.8622
Epoch 2/1000
455/455 [==============================] - 0s 78us/step - loss: 0.1927 - accuracy: 0.9209 - val_loss: 0.3317 - val_accuracy: 0.8776
Epoch 3/1000
455/455 [==============================] - 0s 76us/step - loss: 0.1897 - accuracy: 0.9165 - val_loss: 0.3291 - val_accuracy: 0.8776
Epoch 4/1000
455/455 [==============================] - 0s 61us/step - loss: 0.1906 - accuracy: 0.9165 - val_loss: 0.3316 - val_accuracy: 0.8776
Epoch 5/1000
455/455 [==============================] - 0s 48us/step - loss: 0.1889 - accuracy: 0.9275 - val_loss: 0.3293 - val_accuracy: 0.8776
Epoch 6/1000
455/455 [==============================] - 0s 52us/step - loss: 0.1906 - accuracy: 0.9231 - val_loss: 0.3339 - val_accuracy: 0.8673
Epoch 7/1000
455/455 [==============================] - 0s 52us/step - loss: 0.1883 

Epoch 57/1000
455/455 [==============================] - 0s 72us/step - loss: 0.1864 - accuracy: 0.9165 - val_loss: 0.3753 - val_accuracy: 0.8673
Epoch 58/1000
455/455 [==============================] - 0s 65us/step - loss: 0.2108 - accuracy: 0.9187 - val_loss: 0.3214 - val_accuracy: 0.8673
Epoch 59/1000
455/455 [==============================] - 0s 86us/step - loss: 0.2135 - accuracy: 0.9231 - val_loss: 0.3374 - val_accuracy: 0.8724
Epoch 60/1000
455/455 [==============================] - 0s 135us/step - loss: 0.2012 - accuracy: 0.9121 - val_loss: 0.3448 - val_accuracy: 0.8724
Epoch 61/1000
455/455 [==============================] - 0s 307us/step - loss: 0.2037 - accuracy: 0.9209 - val_loss: 0.3160 - val_accuracy: 0.8673
Epoch 62/1000
455/455 [==============================] - 0s 67us/step - loss: 0.1866 - accuracy: 0.9319 - val_loss: 0.4071 - val_accuracy: 0.8622
Epoch 63/1000
455/455 [==============================] - 0s 72us/step - loss: 0.2183 - accuracy: 0.9187 - val_loss: 0.3136

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.109797). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


455/455 [==============================] - 0s 84us/step - loss: 0.1927 - accuracy: 0.9231 - val_loss: 0.3402 - val_accuracy: 0.8724
Epoch 72/1000
455/455 [==============================] - 0s 79us/step - loss: 0.1993 - accuracy: 0.9165 - val_loss: 0.3254 - val_accuracy: 0.8571
Epoch 73/1000
455/455 [==============================] - 0s 65us/step - loss: 0.2215 - accuracy: 0.8989 - val_loss: 0.3090 - val_accuracy: 0.8520
Epoch 74/1000
455/455 [==============================] - 0s 59us/step - loss: 0.2692 - accuracy: 0.8879 - val_loss: 0.3508 - val_accuracy: 0.8724
Epoch 75/1000
455/455 [==============================] - 0s 57us/step - loss: 0.1999 - accuracy: 0.9143 - val_loss: 0.3935 - val_accuracy: 0.8878
Epoch 76/1000
455/455 [==============================] - 0s 55us/step - loss: 0.1991 - accuracy: 0.9187 - val_loss: 0.3232 - val_accuracy: 0.8673
Epoch 77/1000
455/455 [==============================] - 0s 60us/step - loss: 0.1890 - accuracy: 0.9187 - val_loss: 0.3620 - val_accuracy:

Epoch 127/1000
455/455 [==============================] - 0s 78us/step - loss: 0.1932 - accuracy: 0.9143 - val_loss: 0.3126 - val_accuracy: 0.8520
Epoch 128/1000
455/455 [==============================] - 0s 69us/step - loss: 0.1870 - accuracy: 0.9143 - val_loss: 0.3103 - val_accuracy: 0.8776
Epoch 129/1000
455/455 [==============================] - 0s 82us/step - loss: 0.1883 - accuracy: 0.9319 - val_loss: 0.3297 - val_accuracy: 0.8673
Epoch 130/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1901 - accuracy: 0.9187 - val_loss: 0.2993 - val_accuracy: 0.8520
Epoch 131/1000
455/455 [==============================] - 0s 108us/step - loss: 0.1921 - accuracy: 0.9165 - val_loss: 0.3408 - val_accuracy: 0.8724
Epoch 132/1000
455/455 [==============================] - 0s 82us/step - loss: 0.1928 - accuracy: 0.9231 - val_loss: 0.3258 - val_accuracy: 0.8724
Epoch 133/1000
455/455 [==============================] - 0s 64us/step - loss: 0.1872 - accuracy: 0.9143 - val_loss:

...

Epoch 572/1000
455/455 [==============================] - 0s 42us/step - loss: 0.1854 - accuracy: 0.9187 - val_loss: 0.3131 - val_accuracy: 0.8929
Epoch 573/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1811 - accuracy: 0.9121 - val_loss: 0.3293 - val_accuracy: 0.8776
Epoch 574/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1805 - accuracy: 0.9253 - val_loss: 0.3196 - val_accuracy: 0.8878
Epoch 575/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1865 - accuracy: 0.9231 - val_loss: 0.3279 - val_accuracy: 0.8776
Epoch 576/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1842 - accuracy: 0.9187 - val_loss: 0.3090 - val_accuracy: 0.8520
Epoch 577/1000
455/455 [==============================] - 0s 50us/step - loss: 0.1809 - accuracy: 0.9121 - val_loss: 0.3199 - val_accuracy: 0.8776
Epoch 578/1000
455/455 [==============================] - 0s 46us/step - loss: 0.1788 - accuracy: 0.9253 - val_loss: 0

Epoch 628/1000
455/455 [==============================] - 0s 52us/step - loss: 0.2542 - accuracy: 0.8945 - val_loss: 0.2991 - val_accuracy: 0.8571
Epoch 629/1000
455/455 [==============================] - 0s 51us/step - loss: 0.1939 - accuracy: 0.9121 - val_loss: 0.3953 - val_accuracy: 0.8929
Epoch 630/1000
455/455 [==============================] - 0s 53us/step - loss: 0.1866 - accuracy: 0.9231 - val_loss: 0.3099 - val_accuracy: 0.8622
Epoch 631/1000
455/455 [==============================] - 0s 45us/step - loss: 0.2064 - accuracy: 0.9209 - val_loss: 0.3863 - val_accuracy: 0.8827
Epoch 632/1000
455/455 [==============================] - 0s 56us/step - loss: 0.1968 - accuracy: 0.9253 - val_loss: 0.3050 - val_accuracy: 0.8622
Epoch 633/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1930 - accuracy: 0.9253 - val_loss: 0.3441 - val_accuracy: 0.8980
Epoch 634/1000
455/455 [==============================] - 0s 61us/step - loss: 0.1859 - accuracy: 0.9209 - val_loss: 0

Epoch 739/1000
455/455 [==============================] - 0s 47us/step - loss: 0.2178 - accuracy: 0.9099 - val_loss: 0.3063 - val_accuracy: 0.8418
Epoch 740/1000
455/455 [==============================] - 0s 55us/step - loss: 0.2268 - accuracy: 0.9099 - val_loss: 0.3194 - val_accuracy: 0.8571
Epoch 741/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1795 - accuracy: 0.9165 - val_loss: 0.3925 - val_accuracy: 0.8571
Epoch 742/1000
455/455 [==============================] - 0s 51us/step - loss: 0.1884 - accuracy: 0.9209 - val_loss: 0.3138 - val_accuracy: 0.8622
Epoch 743/1000
455/455 [==============================] - 0s 45us/step - loss: 0.1951 - accuracy: 0.9121 - val_loss: 0.3721 - val_accuracy: 0.8776
Epoch 744/1000
455/455 [==============================] - 0s 46us/step - loss: 0.3125 - accuracy: 0.9121 - val_loss: 0.3272 - val_accuracy: 0.8469
Epoch 745/1000
455/455 [==============================] - 0s 49us/step - loss: 0.2253 - accuracy: 0.9077 - val_loss: 0

Epoch 850/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1877 - accuracy: 0.9231 - val_loss: 0.3319 - val_accuracy: 0.8776
Epoch 851/1000
455/455 [==============================] - 0s 63us/step - loss: 0.1798 - accuracy: 0.9143 - val_loss: 0.3116 - val_accuracy: 0.8622
Epoch 852/1000
455/455 [==============================] - 0s 54us/step - loss: 0.1873 - accuracy: 0.9165 - val_loss: 0.3110 - val_accuracy: 0.8724
Epoch 853/1000
455/455 [==============================] - 0s 55us/step - loss: 0.1868 - accuracy: 0.9209 - val_loss: 0.3142 - val_accuracy: 0.8673
Epoch 854/1000
455/455 [==============================] - 0s 49us/step - loss: 0.2000 - accuracy: 0.9209 - val_loss: 0.3326 - val_accuracy: 0.8673
Epoch 855/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1797 - accuracy: 0.9275 - val_loss: 0.3548 - val_accuracy: 0.8776
Epoch 856/1000
455/455 [==============================] - 0s 109us/step - loss: 0.1911 - accuracy: 0.9209 - val_loss:

Epoch 961/1000
455/455 [==============================] - 0s 52us/step - loss: 0.1842 - accuracy: 0.9187 - val_loss: 0.3493 - val_accuracy: 0.8827
Epoch 962/1000
455/455 [==============================] - 0s 36us/step - loss: 0.1887 - accuracy: 0.9165 - val_loss: 0.3267 - val_accuracy: 0.8673
Epoch 963/1000
455/455 [==============================] - 0s 40us/step - loss: 0.1805 - accuracy: 0.9231 - val_loss: 0.3265 - val_accuracy: 0.8673
Epoch 964/1000
455/455 [==============================] - 0s 35us/step - loss: 0.1937 - accuracy: 0.9209 - val_loss: 0.3218 - val_accuracy: 0.8673
Epoch 965/1000
455/455 [==============================] - 0s 35us/step - loss: 0.1813 - accuracy: 0.9231 - val_loss: 0.3460 - val_accuracy: 0.8673
Epoch 966/1000
455/455 [==============================] - 0s 34us/step - loss: 0.1799 - accuracy: 0.9319 - val_loss: 0.3359 - val_accuracy: 0.8776
Epoch 967/1000
455/455 [==============================] - 0s 35us/step - loss: 0.2083 - accuracy: 0.9231 - val_loss: 0

In [80]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over3.history['accuracy'])*100))

over-sampling train accuracy: 91.75%


In [40]:
df_proba3 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=2,
                        index_col=None)

In [41]:
df_proba3

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
1,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
2,p002ykpresabs_qual,NRS222,0,0,0.851725,0.148269,5.980786e-06
3,p002ykpresabs_qual,NRS109,2,2,0.004477,0.013518,9.820048e-01
4,p002ykpresabs_qual,GA50245,0,0,0.812055,0.187945,1.161034e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4280,pyopresabsSTCC_qual,NRS255,2,2,0.000633,0.000928,9.984396e-01
4281,pyopresabsSTCC_qual,NRS266,1,1,0.025932,0.974061,7.323514e-06
4282,pyopresabsSTCC_qual,NRS001,1,1,0.000597,0.999403,3.675362e-10


In [42]:
y_prob3 = df_proba3[df_proba3['phage']=='pyokpresabsSTCC_qual'].iloc[:,-3:]
y_prob3 = y_prob3.to_numpy()
y_prob3

array([[7.29007660e-05, 9.99901400e-01, 2.56267700e-05],
       [3.03149080e-04, 9.99696850e-01, 1.02015380e-14],
       [6.34855600e-05, 1.80626790e-03, 9.98130260e-01],
       [5.38480660e-07, 3.50494870e-04, 9.99648900e-01],
       [7.22276900e-01, 2.77717050e-01, 6.00837200e-06],
       [9.31639200e-01, 6.83607600e-02, 1.24983120e-10],
       [8.76215640e-01, 1.23770200e-01, 1.41630360e-05],
       [5.38480660e-07, 3.50494870e-04, 9.99648900e-01],
       [1.94394840e-01, 8.05605200e-01, 1.85658100e-11],
       [9.31639200e-01, 6.83607600e-02, 1.24983120e-10],
       [5.21436450e-01, 4.78545800e-01, 1.77767470e-05],
       [4.32009960e-02, 7.43097000e-03, 9.49368000e-01],
       [1.19894100e-01, 8.79528340e-01, 5.77577400e-04],
       [2.75911720e-01, 7.23173140e-01, 9.15089200e-04],
       [1.57515100e-02, 9.84248460e-01, 7.65452840e-11],
       [7.29007660e-05, 9.99901400e-01, 2.56267700e-05],
       [9.74025100e-01, 2.59748660e-02, 1.42114880e-13],
       [9.99998570e-01, 1.45026

In [43]:
ovo3 = rocauc_ovo(y_test_over, y_prob3, average="macro", multi_class="ovo")
ovo3

0.964836562928166

In [44]:
ovr3 = rocauc_ovr(y_test_over, y_prob3, average="macro", multi_class="ovr")
ovr3

0.964836562928166

In [45]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=456,
                                                    stratify=y_over)

In [46]:
dat4 = pd.DataFrame(X_test_over[:,0])
dat4['test'] = y_test_over

In [47]:
dat4

,0,test
0,NRS178,1
1,NRS109,2
2,NRS073,1
3,CFBREBSa119,0
4,NRS109,2
...,...,...
191,NRS236,1
192,NRS029,0
193,NRS148,2
194,CFBRSa28,0


In [48]:
X_train_over = X_train_over[:,1:]
X_test_over = X_test_over[:,1:]

In [49]:
model1_over4 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [59]:
model1_over4.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [60]:
model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 468us/step - loss: 20.3301 - accuracy: 0.3187 - val_loss: 5.6662 - val_accuracy: 0.3265
Epoch 2/1000
455/455 [==============================] - 0s 189us/step - loss: 3.5564 - accuracy: 0.4308 - val_loss: 1.8215 - val_accuracy: 0.4796
Epoch 3/1000
455/455 [==============================] - 0s 198us/step - loss: 1.5707 - accuracy: 0.4396 - val_loss: 1.0144 - val_accuracy: 0.5408
Epoch 4/1000
455/455 [==============================] - 0s 217us/step - loss: 0.9649 - accuracy: 0.5473 - val_loss: 0.8548 - val_accuracy: 0.6888
Epoch 5/1000
455/455 [==============================] - 0s 208us/step - loss: 0.9022 - accuracy: 0.6220 - val_loss: 0.8776 - val_accuracy: 0.6582
Epoch 6/1000
455/455 [==============================] - 0s 208us/step - loss: 0.8781 - accuracy: 0.6505 - val_loss: 0.7115 - val_accuracy: 0.7143
Epoch 7/1000
455/455 [==============================] - 0s 132us/step - loss:

Epoch 112/1000
455/455 [==============================] - 0s 104us/step - loss: 0.2477 - accuracy: 0.8945 - val_loss: 0.2707 - val_accuracy: 0.8827
Epoch 113/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2429 - accuracy: 0.9121 - val_loss: 0.2376 - val_accuracy: 0.9082
Epoch 114/1000
455/455 [==============================] - 0s 101us/step - loss: 0.2432 - accuracy: 0.8989 - val_loss: 0.2419 - val_accuracy: 0.8980
Epoch 115/1000
455/455 [==============================] - 0s 115us/step - loss: 0.2415 - accuracy: 0.9033 - val_loss: 0.2315 - val_accuracy: 0.8980
Epoch 116/1000
455/455 [==============================] - 0s 99us/step - loss: 0.2435 - accuracy: 0.9033 - val_loss: 0.2390 - val_accuracy: 0.8980
Epoch 117/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2391 - accuracy: 0.9011 - val_loss: 0.3176 - val_accuracy: 0.8673
Epoch 118/1000
455/455 [==============================] - 0s 95us/step - loss: 0.2680 - accuracy: 0.9011 - val_loss

455/455 [==============================] - 0s 336us/step - loss: 0.2662 - accuracy: 0.8857 - val_loss: 0.2361 - val_accuracy: 0.9031
Epoch 223/1000
455/455 [==============================] - 0s 312us/step - loss: 0.2049 - accuracy: 0.9187 - val_loss: 0.2134 - val_accuracy: 0.9031
Epoch 224/1000
455/455 [==============================] - 0s 250us/step - loss: 0.2017 - accuracy: 0.9121 - val_loss: 0.2072 - val_accuracy: 0.9031
Epoch 225/1000
455/455 [==============================] - 0s 162us/step - loss: 0.2108 - accuracy: 0.9143 - val_loss: 0.2256 - val_accuracy: 0.9031
Epoch 226/1000
455/455 [==============================] - 0s 119us/step - loss: 0.2006 - accuracy: 0.9121 - val_loss: 0.2147 - val_accuracy: 0.9031
Epoch 227/1000
455/455 [==============================] - 0s 276us/step - loss: 0.2042 - accuracy: 0.9077 - val_loss: 0.2105 - val_accuracy: 0.9133
Epoch 228/1000
455/455 [==============================] - 0s 345us/step - loss: 0.2130 - accuracy: 0.9077 - val_loss: 0.2048 - 

Epoch 333/1000
455/455 [==============================] - 0s 97us/step - loss: 0.2088 - accuracy: 0.9099 - val_loss: 0.2283 - val_accuracy: 0.9031
Epoch 334/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1871 - accuracy: 0.9209 - val_loss: 0.2202 - val_accuracy: 0.9031
Epoch 335/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1951 - accuracy: 0.9143 - val_loss: 0.2406 - val_accuracy: 0.9031
Epoch 336/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1916 - accuracy: 0.9165 - val_loss: 0.2155 - val_accuracy: 0.9031
Epoch 337/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1884 - accuracy: 0.9209 - val_loss: 0.2300 - val_accuracy: 0.9031
Epoch 338/1000
455/455 [==============================] - 0s 98us/step - loss: 0.2117 - accuracy: 0.9099 - val_loss: 0.2636 - val_accuracy: 0.9031
Epoch 339/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1994 - accuracy: 0.9165 - val_loss: 

455/455 [==============================] - 0s 100us/step - loss: 0.1876 - accuracy: 0.9209 - val_loss: 0.2097 - val_accuracy: 0.9133
Epoch 444/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1851 - accuracy: 0.9341 - val_loss: 0.2061 - val_accuracy: 0.9082
Epoch 445/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1805 - accuracy: 0.9363 - val_loss: 0.2172 - val_accuracy: 0.9133
Epoch 446/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1806 - accuracy: 0.9275 - val_loss: 0.2177 - val_accuracy: 0.9082
Epoch 447/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1881 - accuracy: 0.9253 - val_loss: 0.2069 - val_accuracy: 0.9082
Epoch 448/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1885 - accuracy: 0.9297 - val_loss: 0.2032 - val_accuracy: 0.9133
Epoch 449/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1814 - accuracy: 0.9231 - val_loss: 0.2099 - val_

Epoch 554/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1762 - accuracy: 0.9319 - val_loss: 0.2324 - val_accuracy: 0.9082
Epoch 555/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1781 - accuracy: 0.9319 - val_loss: 0.2054 - val_accuracy: 0.9133
Epoch 556/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1823 - accuracy: 0.9253 - val_loss: 0.2126 - val_accuracy: 0.9133
Epoch 557/1000
455/455 [==============================] - 0s 107us/step - loss: 0.1865 - accuracy: 0.9341 - val_loss: 0.2212 - val_accuracy: 0.9133
Epoch 558/1000
455/455 [==============================] - 0s 101us/step - loss: 0.1807 - accuracy: 0.9363 - val_loss: 0.2104 - val_accuracy: 0.9133
Epoch 559/1000
455/455 [==============================] - 0s 111us/step - loss: 0.1806 - accuracy: 0.9253 - val_loss: 0.2016 - val_accuracy: 0.9133
Epoch 560/1000
455/455 [==============================] - 0s 108us/step - loss: 0.1780 - accuracy: 0.9253 - val_l

455/455 [==============================] - 0s 128us/step - loss: 0.1794 - accuracy: 0.9319 - val_loss: 0.2159 - val_accuracy: 0.9133
Epoch 665/1000
455/455 [==============================] - 0s 118us/step - loss: 0.1772 - accuracy: 0.9341 - val_loss: 0.2116 - val_accuracy: 0.9133
Epoch 666/1000
455/455 [==============================] - 0s 120us/step - loss: 0.1886 - accuracy: 0.9363 - val_loss: 0.2356 - val_accuracy: 0.9133
Epoch 667/1000
455/455 [==============================] - 0s 124us/step - loss: 0.1776 - accuracy: 0.9253 - val_loss: 0.2070 - val_accuracy: 0.9184
Epoch 668/1000
455/455 [==============================] - 0s 124us/step - loss: 0.2037 - accuracy: 0.9187 - val_loss: 0.2015 - val_accuracy: 0.9133
Epoch 669/1000
455/455 [==============================] - 0s 121us/step - loss: 0.1831 - accuracy: 0.9275 - val_loss: 0.2160 - val_accuracy: 0.9184
Epoch 670/1000
455/455 [==============================] - 0s 119us/step - loss: 0.1839 - accuracy: 0.9275 - val_loss: 0.2045 - 

455/455 [==============================] - 0s 139us/step - loss: 0.1794 - accuracy: 0.9341 - val_loss: 0.2019 - val_accuracy: 0.9286
Epoch 775/1000
455/455 [==============================] - 0s 144us/step - loss: 0.2139 - accuracy: 0.8989 - val_loss: 0.2497 - val_accuracy: 0.9184
Epoch 776/1000
455/455 [==============================] - 0s 140us/step - loss: 0.5271 - accuracy: 0.8835 - val_loss: 0.2862 - val_accuracy: 0.9184
Epoch 777/1000
455/455 [==============================] - 0s 139us/step - loss: 0.5850 - accuracy: 0.8659 - val_loss: 0.2550 - val_accuracy: 0.8980
Epoch 778/1000
455/455 [==============================] - 0s 141us/step - loss: 0.3491 - accuracy: 0.8813 - val_loss: 0.2136 - val_accuracy: 0.9184
Epoch 779/1000
455/455 [==============================] - 0s 138us/step - loss: 0.1984 - accuracy: 0.9297 - val_loss: 0.2277 - val_accuracy: 0.9184
Epoch 780/1000
455/455 [==============================] - 0s 169us/step - loss: 0.1743 - accuracy: 0.9341 - val_loss: 0.2172 - 

455/455 [==============================] - 0s 135us/step - loss: 0.1754 - accuracy: 0.9363 - val_loss: 0.2095 - val_accuracy: 0.9184
Epoch 885/1000
455/455 [==============================] - 0s 129us/step - loss: 0.2034 - accuracy: 0.9231 - val_loss: 0.2066 - val_accuracy: 0.9286
Epoch 886/1000
455/455 [==============================] - 0s 143us/step - loss: 0.1929 - accuracy: 0.9275 - val_loss: 0.2102 - val_accuracy: 0.9184
Epoch 887/1000
455/455 [==============================] - 0s 141us/step - loss: 0.1726 - accuracy: 0.9341 - val_loss: 0.2098 - val_accuracy: 0.9133
Epoch 888/1000
455/455 [==============================] - 0s 139us/step - loss: 0.1830 - accuracy: 0.9341 - val_loss: 0.2088 - val_accuracy: 0.9133
Epoch 889/1000
455/455 [==============================] - 0s 142us/step - loss: 0.1779 - accuracy: 0.9341 - val_loss: 0.2179 - val_accuracy: 0.9133
Epoch 890/1000
455/455 [==============================] - 0s 160us/step - loss: 0.1760 - accuracy: 0.9319 - val_loss: 0.2059 - 

455/455 [==============================] - 0s 133us/step - loss: 0.1688 - accuracy: 0.9341 - val_loss: 0.2224 - val_accuracy: 0.9184
Epoch 995/1000
455/455 [==============================] - 0s 124us/step - loss: 0.1783 - accuracy: 0.9297 - val_loss: 0.2136 - val_accuracy: 0.9184
Epoch 996/1000
455/455 [==============================] - 0s 119us/step - loss: 0.1749 - accuracy: 0.9363 - val_loss: 0.2087 - val_accuracy: 0.9133
Epoch 997/1000
455/455 [==============================] - 0s 111us/step - loss: 0.1782 - accuracy: 0.9231 - val_loss: 0.2142 - val_accuracy: 0.9184
Epoch 998/1000
455/455 [==============================] - 0s 116us/step - loss: 0.1693 - accuracy: 0.9363 - val_loss: 0.2537 - val_accuracy: 0.9133
Epoch 999/1000
455/455 [==============================] - 0s 110us/step - loss: 0.1676 - accuracy: 0.9341 - val_loss: 0.2014 - val_accuracy: 0.9133
Epoch 1000/1000
455/455 [==============================] - 0s 109us/step - loss: 0.1734 - accuracy: 0.9341 - val_loss: 0.2254 -

In [96]:
acc_test_over4 = model1_over4.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over4*100))

196/196 [==============================] - 0s 52us/step
over-sampling test accuracy: 90.82%


In [61]:
pred4 = model1_over4.predict_classes(X_test_over)
pred4

array([1, 2, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 1, 2, 0, 0, 0, 2, 2, 1, 2, 1,
       1, 1, 1, 0, 2, 2, 1, 1, 2, 1, 1, 2, 2, 0, 0, 0, 1, 1, 1, 2, 1, 0,
       1, 1, 2, 2, 0, 0, 2, 0, 2, 2, 1, 0, 0, 0, 0, 1, 1, 1, 2, 1, 2, 0,
       0, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 2, 1, 2, 0, 0, 1, 0,
       0, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 0, 2, 2, 1, 1, 1, 2, 0, 1,
       2, 2, 2, 0, 2, 2, 2, 1, 1, 0, 0, 2, 0, 1, 0, 1, 2, 2, 2, 0, 0, 0,
       2, 1, 2, 0, 0, 0, 1, 2, 2, 1, 0, 2, 0, 0, 1, 0, 0, 1, 1, 2, 2, 2,
       1, 1, 1, 2, 2, 0, 0, 2, 1, 0, 0, 2, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 2, 1, 1, 1, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2, 1, 1, 2, 0, 2])

In [62]:
dat4['pred'] = pred4
dat4

,0,test,pred
0,NRS178,1,1
1,NRS109,2,2
2,NRS073,1,0
3,CFBREBSa119,0,0
4,NRS109,2,2
...,...,...,...
191,NRS236,1,1
192,NRS029,0,1
193,NRS148,2,2
194,CFBRSa28,0,0


In [63]:
proba4 = model1_over4.predict_proba(X_test_over)
dat_proba4 = pd.DataFrame(proba4)

In [64]:
dat_proba4

,0,1,2
0,4.670745e-07,9.999995e-01,1.172018e-11
1,2.928581e-10,1.421466e-05,9.999858e-01
2,8.752239e-01,1.247761e-01,7.607542e-09
3,9.992100e-01,7.899318e-04,3.151931e-19
4,2.928581e-10,1.421466e-05,9.999858e-01
...,...,...,...
191,1.044402e-04,9.998951e-01,4.247138e-07
192,2.787349e-01,7.212651e-01,2.023308e-08
193,9.903677e-05,1.057225e-05,9.998903e-01
194,1.000000e+00,9.156101e-11,2.589021e-35


In [65]:
dat_proba4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/proba4.csv", index = False,
         header=None)

In [66]:
dat4.to_csv("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/4pyoST.csv", index = False,
         header=None)

In [100]:
hist1_over4 = model1_over4.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 165us/step - loss: 0.1671 - accuracy: 0.9341 - val_loss: 0.2603 - val_accuracy: 0.9133
Epoch 2/1000
455/455 [==============================] - 0s 117us/step - loss: 0.1624 - accuracy: 0.9319 - val_loss: 0.2605 - val_accuracy: 0.9133
Epoch 3/1000
455/455 [==============================] - 0s 118us/step - loss: 0.1585 - accuracy: 0.9341 - val_loss: 0.2911 - val_accuracy: 0.9133
Epoch 4/1000
455/455 [==============================] - 0s 119us/step - loss: 0.1652 - accuracy: 0.9363 - val_loss: 0.2536 - val_accuracy: 0.9082
Epoch 5/1000
455/455 [==============================] - 0s 134us/step - loss: 0.1701 - accuracy: 0.9363 - val_loss: 0.2452 - val_accuracy: 0.9082
Epoch 6/1000
455/455 [==============================] - 0s 133us/step - loss: 0.1658 - accuracy: 0.9363 - val_loss: 0.2563 - val_accuracy: 0.9082
Epoch 7/1000
455/455 [==============================] - 0s 136us/step - loss: 

Epoch 112/1000
455/455 [==============================] - 0s 407us/step - loss: 0.2723 - accuracy: 0.9077 - val_loss: 0.2703 - val_accuracy: 0.9133
Epoch 113/1000
455/455 [==============================] - 0s 262us/step - loss: 0.1704 - accuracy: 0.9297 - val_loss: 0.2803 - val_accuracy: 0.9082
Epoch 114/1000
455/455 [==============================] - 0s 219us/step - loss: 0.1637 - accuracy: 0.9341 - val_loss: 0.2792 - val_accuracy: 0.9082
Epoch 115/1000
455/455 [==============================] - 0s 204us/step - loss: 0.1680 - accuracy: 0.9341 - val_loss: 0.2626 - val_accuracy: 0.9082
Epoch 116/1000
455/455 [==============================] - 0s 191us/step - loss: 0.1694 - accuracy: 0.9341 - val_loss: 0.2913 - val_accuracy: 0.9133
Epoch 117/1000
455/455 [==============================] - 0s 186us/step - loss: 0.1673 - accuracy: 0.9319 - val_loss: 0.2604 - val_accuracy: 0.9133
Epoch 118/1000
455/455 [==============================] - 0s 178us/step - loss: 0.1604 - accuracy: 0.9385 - val_

455/455 [==============================] - 0s 126us/step - loss: 0.1673 - accuracy: 0.9363 - val_loss: 0.2541 - val_accuracy: 0.9082
Epoch 223/1000
455/455 [==============================] - 0s 112us/step - loss: 0.1597 - accuracy: 0.9363 - val_loss: 0.2609 - val_accuracy: 0.9133
Epoch 224/1000
455/455 [==============================] - 0s 109us/step - loss: 0.1617 - accuracy: 0.9363 - val_loss: 0.2998 - val_accuracy: 0.9133
Epoch 225/1000
455/455 [==============================] - 0s 121us/step - loss: 0.1682 - accuracy: 0.9341 - val_loss: 0.2643 - val_accuracy: 0.9082
Epoch 226/1000
455/455 [==============================] - 0s 179us/step - loss: 0.1721 - accuracy: 0.9363 - val_loss: 0.2576 - val_accuracy: 0.9082
Epoch 227/1000
455/455 [==============================] - 0s 114us/step - loss: 0.1653 - accuracy: 0.9319 - val_loss: 0.2609 - val_accuracy: 0.9082
Epoch 228/1000
455/455 [==============================] - 0s 105us/step - loss: 0.1578 - accuracy: 0.9407 - val_loss: 0.2659 - 

455/455 [==============================] - 0s 135us/step - loss: 0.1668 - accuracy: 0.9231 - val_loss: 0.2680 - val_accuracy: 0.9082
Epoch 333/1000
455/455 [==============================] - 0s 129us/step - loss: 0.1613 - accuracy: 0.9341 - val_loss: 0.2550 - val_accuracy: 0.9133
Epoch 334/1000
455/455 [==============================] - 0s 125us/step - loss: 0.1595 - accuracy: 0.9429 - val_loss: 0.2531 - val_accuracy: 0.9133
Epoch 335/1000
455/455 [==============================] - 0s 119us/step - loss: 0.1583 - accuracy: 0.9363 - val_loss: 0.2727 - val_accuracy: 0.9082
Epoch 336/1000
455/455 [==============================] - 0s 129us/step - loss: 0.1599 - accuracy: 0.9385 - val_loss: 0.2583 - val_accuracy: 0.9082
Epoch 337/1000
455/455 [==============================] - 0s 118us/step - loss: 0.1573 - accuracy: 0.9385 - val_loss: 0.2583 - val_accuracy: 0.9133
Epoch 338/1000
455/455 [==============================] - 0s 112us/step - loss: 0.1562 - accuracy: 0.9363 - val_loss: 0.2804 - 

455/455 [==============================] - 0s 123us/step - loss: 0.1578 - accuracy: 0.9385 - val_loss: 0.2582 - val_accuracy: 0.9133
Epoch 443/1000
455/455 [==============================] - 0s 126us/step - loss: 0.1553 - accuracy: 0.9363 - val_loss: 0.2647 - val_accuracy: 0.9082
Epoch 444/1000
455/455 [==============================] - 0s 133us/step - loss: 0.1592 - accuracy: 0.9363 - val_loss: 0.2569 - val_accuracy: 0.9133
Epoch 445/1000
455/455 [==============================] - 0s 175us/step - loss: 0.1530 - accuracy: 0.9363 - val_loss: 0.2714 - val_accuracy: 0.9133
Epoch 446/1000
455/455 [==============================] - 0s 176us/step - loss: 0.1550 - accuracy: 0.9363 - val_loss: 0.2667 - val_accuracy: 0.9133
Epoch 447/1000
455/455 [==============================] - 0s 130us/step - loss: 0.1575 - accuracy: 0.9363 - val_loss: 0.2836 - val_accuracy: 0.9133
Epoch 448/1000
455/455 [==============================] - 0s 126us/step - loss: 0.1627 - accuracy: 0.9319 - val_loss: 0.2499 - 

455/455 [==============================] - 0s 185us/step - loss: 0.1580 - accuracy: 0.9363 - val_loss: 0.2605 - val_accuracy: 0.9133
Epoch 553/1000
455/455 [==============================] - 0s 215us/step - loss: 0.1617 - accuracy: 0.9385 - val_loss: 0.2721 - val_accuracy: 0.9133
Epoch 554/1000
455/455 [==============================] - 0s 136us/step - loss: 0.1573 - accuracy: 0.9407 - val_loss: 0.2622 - val_accuracy: 0.9133
Epoch 555/1000
455/455 [==============================] - 0s 188us/step - loss: 0.1589 - accuracy: 0.9319 - val_loss: 0.2945 - val_accuracy: 0.9184
Epoch 556/1000
455/455 [==============================] - 0s 183us/step - loss: 0.1838 - accuracy: 0.9341 - val_loss: 0.4009 - val_accuracy: 0.8724
Epoch 557/1000
455/455 [==============================] - 0s 189us/step - loss: 0.1685 - accuracy: 0.9297 - val_loss: 0.2678 - val_accuracy: 0.9133
Epoch 558/1000
455/455 [==============================] - 0s 206us/step - loss: 0.1541 - accuracy: 0.9363 - val_loss: 0.2558 - 

455/455 [==============================] - 0s 174us/step - loss: 0.1548 - accuracy: 0.9407 - val_loss: 0.2613 - val_accuracy: 0.9082
Epoch 663/1000
455/455 [==============================] - 0s 135us/step - loss: 0.1541 - accuracy: 0.9407 - val_loss: 0.2710 - val_accuracy: 0.9133
Epoch 664/1000
455/455 [==============================] - 0s 130us/step - loss: 0.1574 - accuracy: 0.9319 - val_loss: 0.2502 - val_accuracy: 0.9133
Epoch 665/1000
455/455 [==============================] - 0s 135us/step - loss: 0.1546 - accuracy: 0.9385 - val_loss: 0.2610 - val_accuracy: 0.9082
Epoch 666/1000
455/455 [==============================] - 0s 133us/step - loss: 0.1595 - accuracy: 0.9319 - val_loss: 0.2659 - val_accuracy: 0.9133
Epoch 667/1000
455/455 [==============================] - 0s 127us/step - loss: 0.1530 - accuracy: 0.9363 - val_loss: 0.2659 - val_accuracy: 0.9133
Epoch 668/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1625 - accuracy: 0.9341 - val_loss: 0.2482 - 

Epoch 773/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1554 - accuracy: 0.9363 - val_loss: 0.2711 - val_accuracy: 0.9184
Epoch 774/1000
455/455 [==============================] - 0s 103us/step - loss: 0.1603 - accuracy: 0.9363 - val_loss: 0.2721 - val_accuracy: 0.9184
Epoch 775/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1543 - accuracy: 0.9407 - val_loss: 0.2536 - val_accuracy: 0.9082
Epoch 776/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1539 - accuracy: 0.9363 - val_loss: 0.2681 - val_accuracy: 0.9184
Epoch 777/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1569 - accuracy: 0.9385 - val_loss: 0.2496 - val_accuracy: 0.9133
Epoch 778/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1541 - accuracy: 0.9385 - val_loss: 0.2599 - val_accuracy: 0.9184
Epoch 779/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1581 - accuracy: 0.9297 - val_loss

Epoch 884/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1521 - accuracy: 0.9407 - val_loss: 0.2503 - val_accuracy: 0.9184
Epoch 885/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1545 - accuracy: 0.9363 - val_loss: 0.2516 - val_accuracy: 0.9184
Epoch 886/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1626 - accuracy: 0.9385 - val_loss: 0.2666 - val_accuracy: 0.9184
Epoch 887/1000
455/455 [==============================] - 0s 95us/step - loss: 0.1577 - accuracy: 0.9341 - val_loss: 0.2795 - val_accuracy: 0.9184
Epoch 888/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1571 - accuracy: 0.9407 - val_loss: 0.2434 - val_accuracy: 0.9184
Epoch 889/1000
455/455 [==============================] - 0s 97us/step - loss: 0.1526 - accuracy: 0.9385 - val_loss: 0.2654 - val_accuracy: 0.9184
Epoch 890/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1470 - accuracy: 0.9385 - val_loss: 

Epoch 940/1000
455/455 [==============================] - 0s 99us/step - loss: 0.1506 - accuracy: 0.9319 - val_loss: 0.2558 - val_accuracy: 0.9184
Epoch 941/1000
455/455 [==============================] - 0s 100us/step - loss: 0.1579 - accuracy: 0.9341 - val_loss: 0.2764 - val_accuracy: 0.9133
Epoch 942/1000
455/455 [==============================] - 0s 98us/step - loss: 0.1511 - accuracy: 0.9363 - val_loss: 0.2581 - val_accuracy: 0.9133
Epoch 943/1000
455/455 [==============================] - 0s 102us/step - loss: 0.1529 - accuracy: 0.9363 - val_loss: 0.2851 - val_accuracy: 0.9184
Epoch 944/1000
455/455 [==============================] - 0s 96us/step - loss: 0.2875 - accuracy: 0.9165 - val_loss: 0.2731 - val_accuracy: 0.9235
Epoch 945/1000
455/455 [==============================] - 0s 96us/step - loss: 0.1578 - accuracy: 0.9319 - val_loss: 0.2634 - val_accuracy: 0.9133
Epoch 946/1000
455/455 [==============================] - 0s 121us/step - loss: 0.1487 - accuracy: 0.9407 - val_loss

In [101]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over4.history['accuracy'])*100))

over-sampling train accuracy: 93.39%


In [50]:
df_proba4 = pd.read_excel("/Users/Rebecca/Desktop/Claudia/neural network/new_phage_qual/dataset/NN_over_2.xlsx",
                        sheet_name=3,
                        index_col=None)

In [51]:
df_proba4

,phage,strain,phenotype,prediction,0,1,2
0,p002ykpresabs_qual,NRS110,1,1,0.000003,0.999997,5.870196e-13
1,p002ykpresabs_qual,NRS216,1,1,0.039254,0.960745,9.078969e-07
2,p002ykpresabs_qual,NRS386,1,1,0.326752,0.673248,1.061032e-07
3,p002ykpresabs_qual,CFBRSa25,0,0,0.611084,0.388916,7.664974e-07
4,p002ykpresabs_qual,BCH-SA-03,1,0,0.611084,0.388916,7.664974e-07
...,...,...,...,...,...,...,...
4279,pyopresabsSTCC_qual,NRS236,1,1,0.000052,0.999768,1.803156e-04
4280,pyopresabsSTCC_qual,NRS029,0,1,0.322350,0.677496,1.533154e-04
4281,pyopresabsSTCC_qual,NRS148,2,2,0.000006,0.000026,9.999682e-01
4282,pyopresabsSTCC_qual,CFBRSa28,0,0,0.999288,0.000176,5.361527e-04


In [52]:
y_prob4 = df_proba4[df_proba4['phage']=='pyokpresabsSTCC_qual'].iloc[:,-3:]
y_prob4 = y_prob4.to_numpy()
y_prob4

array([[4.67074530e-07, 9.99999500e-01, 1.17201830e-11],
       [2.92858100e-10, 1.42146610e-05, 9.99985800e-01],
       [8.75223900e-01, 1.24776095e-01, 7.60754200e-09],
       [9.99210000e-01, 7.89931800e-04, 3.15193100e-19],
       [2.92858100e-10, 1.42146610e-05, 9.99985800e-01],
       [7.51580200e-02, 9.19317250e-01, 5.52470560e-03],
       [3.03495270e-04, 9.99695660e-01, 8.74896900e-07],
       [2.92858100e-10, 1.42146610e-05, 9.99985800e-01],
       [9.90367700e-05, 1.05722465e-05, 9.99890300e-01],
       [1.00000000e+00, 1.58114330e-24, 0.00000000e+00],
       [2.92858100e-10, 1.42146610e-05, 9.99985800e-01],
       [7.51580200e-02, 9.19317250e-01, 5.52470560e-03],
       [2.77676250e-01, 7.22323700e-01, 6.55355150e-28],
       [9.90367700e-05, 1.05722465e-05, 9.99890300e-01],
       [8.86884150e-01, 1.13115800e-01, 1.29184800e-21],
       [8.96951000e-01, 1.03048930e-01, 2.05142640e-08],
       [8.86884150e-01, 1.13115800e-01, 1.29184800e-21],
       [9.90367700e-05, 1.05722

In [53]:
ovo4 = rocauc_ovo(y_test_over, y_prob4, average="macro", multi_class="ovo")
ovo4

0.9762890205638297

In [54]:
ovr4 = rocauc_ovr(y_test_over, y_prob4, average="macro", multi_class="ovr")
ovr4

0.9762890205638297

In [55]:
ovos = [ovo1, ovo2, ovo3, ovo4]
np.mean(ovos)

0.9718908477019164

In [56]:
np.std(ovos)

0.009238414004126684

In [57]:
ovrs = [ovr1, ovr2, ovr3, ovr4]
np.mean(ovrs)

0.9718908477019164

In [58]:
np.std(ovrs)

0.009238414004126684

In [102]:
accs = [acc_test_over, acc_test_over2, acc_test_over3, acc_test_over4]

In [103]:
mean = np.mean(accs)
print('over-sampling test accuracy mean: %.2f%%' % (mean*100))

over-sampling test accuracy mean: 90.05%


In [104]:
std = np.std(accs)
print('over-sampling test accuracy standard deviation:', std)

over-sampling test accuracy standard deviation: 0.015092033066563634


In [105]:
accs_train = [np.mean(hist1_over.history['accuracy']), np.mean(hist1_over2.history['accuracy']), np.mean(hist1_over3.history['accuracy']),
             np.mean(hist1_over4.history['accuracy'])]

In [106]:
mean_train = np.mean(accs_train)
print('over-sampling train accuracy mean: %.2f%%' % (mean_train*100))

over-sampling train accuracy mean: 92.81%


In [107]:
std_train = np.std(accs_train)
print('over-sampling train accuracy standard deviation:', std_train)

over-sampling train accuracy standard deviation: 0.0063417354


In [59]:
#### add regularizer and dropout
model1_over5 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [60]:
model1_over5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [61]:
model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 966us/step - loss: 28.7861 - accuracy: 0.3736 - val_loss: 26.4031 - val_accuracy: 0.4949
Epoch 2/1000
455/455 [==============================] - 0s 112us/step - loss: 27.9320 - accuracy: 0.4440 - val_loss: 24.7947 - val_accuracy: 0.4286
Epoch 3/1000
455/455 [==============================] - 0s 101us/step - loss: 26.2801 - accuracy: 0.4484 - val_loss: 23.1199 - val_accuracy: 0.4643
Epoch 4/1000
455/455 [==============================] - 0s 79us/step - loss: 24.5841 - accuracy: 0.4286 - val_loss: 20.8934 - val_accuracy: 0.5408
Epoch 5/1000
455/455 [==============================] - 0s 88us/step - loss: 23.2883 - accuracy: 0.4088 - val_loss: 20.1763 - val_accuracy: 0.5255
Epoch 6/1000
455/455 [==============================] - 0s 84us/step - loss: 21.6469 - accuracy: 0.5275 - val_loss: 17.9040 - val_accuracy: 0.5255
Epoch 7/1000
455/455 [==============================] - 0s 80us/step 

455/455 [==============================] - 0s 147us/step - loss: 2.9152 - accuracy: 0.6879 - val_loss: 0.7042 - val_accuracy: 0.8418
Epoch 112/1000
455/455 [==============================] - 0s 110us/step - loss: 3.2429 - accuracy: 0.6615 - val_loss: 0.7471 - val_accuracy: 0.7908
Epoch 113/1000
455/455 [==============================] - 0s 102us/step - loss: 3.0488 - accuracy: 0.6791 - val_loss: 0.7845 - val_accuracy: 0.7908
Epoch 114/1000
455/455 [==============================] - 0s 137us/step - loss: 2.7841 - accuracy: 0.6857 - val_loss: 0.7464 - val_accuracy: 0.7908
Epoch 115/1000
455/455 [==============================] - 0s 144us/step - loss: 2.9210 - accuracy: 0.6725 - val_loss: 0.7307 - val_accuracy: 0.8316
Epoch 116/1000
455/455 [==============================] - 0s 111us/step - loss: 3.8911 - accuracy: 0.5846 - val_loss: 0.9988 - val_accuracy: 0.8367
Epoch 117/1000
455/455 [==============================] - 0s 127us/step - loss: 3.2209 - accuracy: 0.6879 - val_loss: 0.9008 - 

Epoch 222/1000
455/455 [==============================] - 0s 100us/step - loss: 1.9433 - accuracy: 0.7429 - val_loss: 0.8464 - val_accuracy: 0.8622
Epoch 223/1000
455/455 [==============================] - 0s 91us/step - loss: 2.2698 - accuracy: 0.7165 - val_loss: 0.8682 - val_accuracy: 0.8622
Epoch 224/1000
455/455 [==============================] - 0s 95us/step - loss: 2.2553 - accuracy: 0.7165 - val_loss: 0.9278 - val_accuracy: 0.8163
Epoch 225/1000
455/455 [==============================] - 0s 82us/step - loss: 2.0360 - accuracy: 0.6813 - val_loss: 0.8990 - val_accuracy: 0.8622
Epoch 226/1000
455/455 [==============================] - 0s 78us/step - loss: 2.1350 - accuracy: 0.6967 - val_loss: 0.8837 - val_accuracy: 0.8622
Epoch 227/1000
455/455 [==============================] - 0s 76us/step - loss: 2.4932 - accuracy: 0.6791 - val_loss: 0.9193 - val_accuracy: 0.8112
Epoch 228/1000
455/455 [==============================] - 0s 72us/step - loss: 2.3321 - accuracy: 0.6725 - val_loss: 

Epoch 333/1000
455/455 [==============================] - 0s 74us/step - loss: 1.8270 - accuracy: 0.7033 - val_loss: 0.8168 - val_accuracy: 0.8724
Epoch 334/1000
455/455 [==============================] - 0s 79us/step - loss: 1.7002 - accuracy: 0.7341 - val_loss: 0.8700 - val_accuracy: 0.8776
Epoch 335/1000
455/455 [==============================] - 0s 77us/step - loss: 1.7297 - accuracy: 0.7363 - val_loss: 0.9112 - val_accuracy: 0.8622
Epoch 336/1000
455/455 [==============================] - 0s 64us/step - loss: 2.1172 - accuracy: 0.6967 - val_loss: 0.8834 - val_accuracy: 0.8776
Epoch 337/1000
455/455 [==============================] - 0s 104us/step - loss: 2.0605 - accuracy: 0.6967 - val_loss: 0.8595 - val_accuracy: 0.8622
Epoch 338/1000
455/455 [==============================] - 0s 144us/step - loss: 2.0129 - accuracy: 0.7143 - val_loss: 0.8493 - val_accuracy: 0.8622
Epoch 339/1000
455/455 [==============================] - 0s 118us/step - loss: 1.5237 - accuracy: 0.7099 - val_loss

Epoch 444/1000
455/455 [==============================] - 0s 226us/step - loss: 1.7146 - accuracy: 0.7319 - val_loss: 0.8336 - val_accuracy: 0.8724
Epoch 445/1000
455/455 [==============================] - 0s 140us/step - loss: 1.4013 - accuracy: 0.7560 - val_loss: 0.8273 - val_accuracy: 0.8673
Epoch 446/1000
455/455 [==============================] - 0s 131us/step - loss: 1.5359 - accuracy: 0.7209 - val_loss: 0.8070 - val_accuracy: 0.8776
Epoch 447/1000
455/455 [==============================] - 0s 121us/step - loss: 1.4423 - accuracy: 0.7275 - val_loss: 0.8301 - val_accuracy: 0.8724
Epoch 448/1000
455/455 [==============================] - 0s 122us/step - loss: 1.6451 - accuracy: 0.7385 - val_loss: 0.8050 - val_accuracy: 0.8776
Epoch 449/1000
455/455 [==============================] - 0s 115us/step - loss: 1.4163 - accuracy: 0.7582 - val_loss: 0.8399 - val_accuracy: 0.8776
Epoch 450/1000
455/455 [==============================] - 0s 146us/step - loss: 1.6076 - accuracy: 0.7319 - val_

455/455 [==============================] - 0s 78us/step - loss: 1.4321 - accuracy: 0.7253 - val_loss: 0.8039 - val_accuracy: 0.8827
Epoch 555/1000
455/455 [==============================] - 0s 85us/step - loss: 1.5229 - accuracy: 0.7429 - val_loss: 0.8084 - val_accuracy: 0.8827
Epoch 556/1000
455/455 [==============================] - 0s 106us/step - loss: 1.5885 - accuracy: 0.7187 - val_loss: 0.8018 - val_accuracy: 0.8776
Epoch 557/1000
455/455 [==============================] - 0s 99us/step - loss: 1.2475 - accuracy: 0.7407 - val_loss: 0.8207 - val_accuracy: 0.8724
Epoch 558/1000
455/455 [==============================] - 0s 179us/step - loss: 1.5498 - accuracy: 0.7363 - val_loss: 0.8211 - val_accuracy: 0.8776
Epoch 559/1000
455/455 [==============================] - 0s 168us/step - loss: 1.6134 - accuracy: 0.6857 - val_loss: 0.8163 - val_accuracy: 0.8827
Epoch 560/1000
455/455 [==============================] - 0s 107us/step - loss: 1.4140 - accuracy: 0.7209 - val_loss: 0.7799 - val

455/455 [==============================] - 0s 124us/step - loss: 1.4730 - accuracy: 0.7429 - val_loss: 0.7834 - val_accuracy: 0.8827
Epoch 665/1000
455/455 [==============================] - 0s 128us/step - loss: 1.4916 - accuracy: 0.7648 - val_loss: 0.7976 - val_accuracy: 0.8827
Epoch 666/1000
455/455 [==============================] - 0s 144us/step - loss: 1.2012 - accuracy: 0.7451 - val_loss: 0.7865 - val_accuracy: 0.8827
Epoch 667/1000
455/455 [==============================] - 0s 134us/step - loss: 1.2763 - accuracy: 0.7516 - val_loss: 0.7829 - val_accuracy: 0.8827
Epoch 668/1000
455/455 [==============================] - 0s 133us/step - loss: 1.4628 - accuracy: 0.7538 - val_loss: 0.7824 - val_accuracy: 0.8827
Epoch 669/1000
455/455 [==============================] - 0s 137us/step - loss: 1.4423 - accuracy: 0.7385 - val_loss: 0.7307 - val_accuracy: 0.8980
Epoch 670/1000
455/455 [==============================] - 0s 141us/step - loss: 1.5536 - accuracy: 0.7143 - val_loss: 0.7697 - 

Epoch 775/1000
455/455 [==============================] - 0s 113us/step - loss: 1.1261 - accuracy: 0.7626 - val_loss: 0.7984 - val_accuracy: 0.8827
Epoch 776/1000
455/455 [==============================] - 0s 122us/step - loss: 1.3090 - accuracy: 0.7385 - val_loss: 0.9219 - val_accuracy: 0.8776
Epoch 777/1000
455/455 [==============================] - 0s 149us/step - loss: 1.1464 - accuracy: 0.7604 - val_loss: 0.8344 - val_accuracy: 0.8878
Epoch 778/1000
455/455 [==============================] - 0s 98us/step - loss: 1.3436 - accuracy: 0.7341 - val_loss: 0.7949 - val_accuracy: 0.8827
Epoch 779/1000
455/455 [==============================] - 0s 85us/step - loss: 1.1985 - accuracy: 0.7363 - val_loss: 0.8162 - val_accuracy: 0.8827
Epoch 780/1000
455/455 [==============================] - 0s 75us/step - loss: 1.4705 - accuracy: 0.7231 - val_loss: 0.8086 - val_accuracy: 0.8878
Epoch 781/1000
455/455 [==============================] - 0s 75us/step - loss: 1.1554 - accuracy: 0.7604 - val_loss

455/455 [==============================] - 0s 90us/step - loss: 1.4393 - accuracy: 0.7165 - val_loss: 0.8068 - val_accuracy: 0.8980
Epoch 886/1000
455/455 [==============================] - 0s 80us/step - loss: 1.2416 - accuracy: 0.7473 - val_loss: 0.8390 - val_accuracy: 0.8827
Epoch 887/1000
455/455 [==============================] - 0s 77us/step - loss: 1.2678 - accuracy: 0.7758 - val_loss: 0.7789 - val_accuracy: 0.8980
Epoch 888/1000
455/455 [==============================] - 0s 94us/step - loss: 1.1323 - accuracy: 0.7890 - val_loss: 0.7641 - val_accuracy: 0.8980
Epoch 889/1000
455/455 [==============================] - 0s 108us/step - loss: 1.1927 - accuracy: 0.7516 - val_loss: 0.7750 - val_accuracy: 0.8980
Epoch 890/1000
455/455 [==============================] - 0s 117us/step - loss: 1.3344 - accuracy: 0.7077 - val_loss: 0.8226 - val_accuracy: 0.8776
Epoch 891/1000
455/455 [==============================] - 0s 115us/step - loss: 1.4726 - accuracy: 0.7407 - val_loss: 0.7942 - val_

Epoch 996/1000
455/455 [==============================] - 0s 224us/step - loss: 1.2736 - accuracy: 0.7582 - val_loss: 0.8498 - val_accuracy: 0.9031
Epoch 997/1000
455/455 [==============================] - 0s 132us/step - loss: 1.3941 - accuracy: 0.7385 - val_loss: 0.7404 - val_accuracy: 0.9031
Epoch 998/1000
455/455 [==============================] - 0s 110us/step - loss: 1.1257 - accuracy: 0.7780 - val_loss: 0.7797 - val_accuracy: 0.9031
Epoch 999/1000
455/455 [==============================] - 0s 93us/step - loss: 1.0769 - accuracy: 0.7473 - val_loss: 0.8101 - val_accuracy: 0.8929
Epoch 1000/1000
455/455 [==============================] - 0s 115us/step - loss: 0.9244 - accuracy: 0.7626 - val_loss: 0.8265 - val_accuracy: 0.8980


In [62]:
acc_test_over5 = model1_over5.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over5*100))

196/196 [==============================] - 0s 60us/step
over-sampling test accuracy: 89.80%


In [63]:
hist1_over5 = model1_over5.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 98us/step - loss: 1.2596 - accuracy: 0.7341 - val_loss: 0.7828 - val_accuracy: 0.8980
Epoch 2/1000
455/455 [==============================] - 0s 89us/step - loss: 1.0582 - accuracy: 0.7626 - val_loss: 0.7910 - val_accuracy: 0.8929
Epoch 3/1000
455/455 [==============================] - 0s 100us/step - loss: 1.3298 - accuracy: 0.7297 - val_loss: 0.8313 - val_accuracy: 0.8929
Epoch 4/1000
455/455 [==============================] - 0s 122us/step - loss: 1.1171 - accuracy: 0.7802 - val_loss: 0.8669 - val_accuracy: 0.8827
Epoch 5/1000
455/455 [==============================] - 0s 125us/step - loss: 0.9433 - accuracy: 0.7560 - val_loss: 0.8593 - val_accuracy: 0.8980
Epoch 6/1000
455/455 [==============================] - 0s 95us/step - loss: 1.2245 - accuracy: 0.7560 - val_loss: 0.8803 - val_accuracy: 0.8827
Epoch 7/1000
455/455 [==============================] - 0s 81us/step - loss: 1.15

Epoch 57/1000
455/455 [==============================] - 0s 76us/step - loss: 1.0340 - accuracy: 0.7758 - val_loss: 0.8662 - val_accuracy: 0.8980
Epoch 58/1000
455/455 [==============================] - 0s 78us/step - loss: 1.1679 - accuracy: 0.7341 - val_loss: 0.7647 - val_accuracy: 0.8980
Epoch 59/1000
455/455 [==============================] - 0s 88us/step - loss: 1.1466 - accuracy: 0.7582 - val_loss: 0.7653 - val_accuracy: 0.8980
Epoch 60/1000
455/455 [==============================] - 0s 81us/step - loss: 1.0893 - accuracy: 0.7846 - val_loss: 0.7494 - val_accuracy: 0.8980
Epoch 61/1000
455/455 [==============================] - 0s 72us/step - loss: 1.2633 - accuracy: 0.7714 - val_loss: 0.8004 - val_accuracy: 0.8980
Epoch 62/1000
455/455 [==============================] - 0s 89us/step - loss: 1.2067 - accuracy: 0.7648 - val_loss: 0.8658 - val_accuracy: 0.8827
Epoch 63/1000
455/455 [==============================] - 0s 86us/step - loss: 1.1778 - accuracy: 0.7956 - val_loss: 0.8266 -

Epoch 113/1000
455/455 [==============================] - 0s 96us/step - loss: 1.2272 - accuracy: 0.7758 - val_loss: 0.8034 - val_accuracy: 0.9031
Epoch 114/1000
455/455 [==============================] - 0s 89us/step - loss: 1.0028 - accuracy: 0.7736 - val_loss: 0.8080 - val_accuracy: 0.8980
Epoch 115/1000
455/455 [==============================] - 0s 158us/step - loss: 1.3945 - accuracy: 0.7626 - val_loss: 0.8363 - val_accuracy: 0.8980
Epoch 116/1000
455/455 [==============================] - 0s 195us/step - loss: 1.3920 - accuracy: 0.7341 - val_loss: 0.8482 - val_accuracy: 0.8980
Epoch 117/1000
455/455 [==============================] - 0s 64us/step - loss: 1.1987 - accuracy: 0.7626 - val_loss: 0.8155 - val_accuracy: 0.8980
Epoch 118/1000
455/455 [==============================] - 0s 65us/step - loss: 1.1625 - accuracy: 0.7451 - val_loss: 0.7895 - val_accuracy: 0.8980
Epoch 119/1000
455/455 [==============================] - 0s 55us/step - loss: 1.1973 - accuracy: 0.7516 - val_loss:

Epoch 224/1000
455/455 [==============================] - 0s 94us/step - loss: 1.1778 - accuracy: 0.7802 - val_loss: 0.8225 - val_accuracy: 0.9031
Epoch 225/1000
455/455 [==============================] - 0s 82us/step - loss: 1.2013 - accuracy: 0.7385 - val_loss: 0.9258 - val_accuracy: 0.9031
Epoch 226/1000
455/455 [==============================] - 0s 91us/step - loss: 1.2130 - accuracy: 0.7538 - val_loss: 0.8958 - val_accuracy: 0.8980
Epoch 227/1000
455/455 [==============================] - 0s 103us/step - loss: 1.4339 - accuracy: 0.7275 - val_loss: 0.8063 - val_accuracy: 0.8980
Epoch 228/1000
455/455 [==============================] - 0s 106us/step - loss: 1.3199 - accuracy: 0.7736 - val_loss: 0.7866 - val_accuracy: 0.8980
Epoch 229/1000
455/455 [==============================] - 0s 92us/step - loss: 1.1226 - accuracy: 0.7824 - val_loss: 0.8661 - val_accuracy: 0.8980
Epoch 230/1000
455/455 [==============================] - 0s 91us/step - loss: 1.2236 - accuracy: 0.7736 - val_loss:

Epoch 335/1000
455/455 [==============================] - 0s 108us/step - loss: 1.1556 - accuracy: 0.7670 - val_loss: 0.7574 - val_accuracy: 0.8980
Epoch 336/1000
455/455 [==============================] - 0s 111us/step - loss: 1.1810 - accuracy: 0.7582 - val_loss: 0.7833 - val_accuracy: 0.8980
Epoch 337/1000
455/455 [==============================] - 0s 80us/step - loss: 0.9608 - accuracy: 0.7780 - val_loss: 0.8681 - val_accuracy: 0.9031
Epoch 338/1000
455/455 [==============================] - 0s 114us/step - loss: 1.2759 - accuracy: 0.7758 - val_loss: 0.8405 - val_accuracy: 0.8980
Epoch 339/1000
455/455 [==============================] - 0s 91us/step - loss: 0.8732 - accuracy: 0.7846 - val_loss: 0.7810 - val_accuracy: 0.8980
Epoch 340/1000
455/455 [==============================] - 0s 77us/step - loss: 0.8576 - accuracy: 0.8110 - val_loss: 0.7368 - val_accuracy: 0.8980
Epoch 341/1000
455/455 [==============================] - 0s 69us/step - loss: 1.1484 - accuracy: 0.7538 - val_loss

...

455/455 [==============================] - 0s 149us/step - loss: 1.1039 - accuracy: 0.7758 - val_loss: 0.8107 - val_accuracy: 0.9031
Epoch 667/1000
455/455 [==============================] - 0s 149us/step - loss: 1.1225 - accuracy: 0.7626 - val_loss: 0.8997 - val_accuracy: 0.9031
Epoch 668/1000
455/455 [==============================] - 0s 168us/step - loss: 1.2276 - accuracy: 0.7560 - val_loss: 0.9643 - val_accuracy: 0.9031
Epoch 669/1000
455/455 [==============================] - 0s 147us/step - loss: 1.0952 - accuracy: 0.7692 - val_loss: 0.9339 - val_accuracy: 0.9031
Epoch 670/1000
455/455 [==============================] - 0s 144us/step - loss: 1.1456 - accuracy: 0.7714 - val_loss: 0.8691 - val_accuracy: 0.9031
Epoch 671/1000
455/455 [==============================] - 0s 151us/step - loss: 1.2864 - accuracy: 0.7604 - val_loss: 0.8193 - val_accuracy: 0.9031
Epoch 672/1000
455/455 [==============================] - 0s 136us/step - loss: 0.9695 - accuracy: 0.7604 - val_loss: 0.7681 - 

Epoch 777/1000
455/455 [==============================] - 0s 85us/step - loss: 0.8717 - accuracy: 0.7692 - val_loss: 0.7516 - val_accuracy: 0.8929
Epoch 778/1000
455/455 [==============================] - 0s 76us/step - loss: 1.1629 - accuracy: 0.7495 - val_loss: 0.8030 - val_accuracy: 0.9031
Epoch 779/1000
455/455 [==============================] - 0s 72us/step - loss: 1.2561 - accuracy: 0.7121 - val_loss: 0.8536 - val_accuracy: 0.9031
Epoch 780/1000
455/455 [==============================] - 0s 72us/step - loss: 1.2678 - accuracy: 0.7692 - val_loss: 0.8105 - val_accuracy: 0.8980
Epoch 781/1000
455/455 [==============================] - 0s 78us/step - loss: 1.3352 - accuracy: 0.7275 - val_loss: 0.7340 - val_accuracy: 0.9031
Epoch 782/1000
455/455 [==============================] - 0s 73us/step - loss: 1.0962 - accuracy: 0.7429 - val_loss: 0.8203 - val_accuracy: 0.9082
Epoch 783/1000
455/455 [==============================] - 0s 74us/step - loss: 1.0161 - accuracy: 0.7692 - val_loss: 1

Epoch 833/1000
455/455 [==============================] - 0s 79us/step - loss: 0.9476 - accuracy: 0.7516 - val_loss: 0.7902 - val_accuracy: 0.9031
Epoch 834/1000
455/455 [==============================] - 0s 80us/step - loss: 0.8947 - accuracy: 0.7802 - val_loss: 0.9280 - val_accuracy: 0.9031
Epoch 835/1000
455/455 [==============================] - 0s 73us/step - loss: 1.0406 - accuracy: 0.7604 - val_loss: 1.0012 - val_accuracy: 0.8980
Epoch 836/1000
455/455 [==============================] - 0s 83us/step - loss: 1.1069 - accuracy: 0.7758 - val_loss: 0.9156 - val_accuracy: 0.8980
Epoch 837/1000
455/455 [==============================] - 0s 76us/step - loss: 1.2307 - accuracy: 0.7231 - val_loss: 0.8183 - val_accuracy: 0.8980
Epoch 838/1000
455/455 [==============================] - 0s 86us/step - loss: 1.3614 - accuracy: 0.7209 - val_loss: 0.7275 - val_accuracy: 0.8980
Epoch 839/1000
455/455 [==============================] - 0s 76us/step - loss: 1.1762 - accuracy: 0.7780 - val_loss: 0

455/455 [==============================] - 0s 64us/step - loss: 1.0616 - accuracy: 0.7648 - val_loss: 0.8409 - val_accuracy: 0.8980
Epoch 944/1000
455/455 [==============================] - 0s 62us/step - loss: 1.1233 - accuracy: 0.7187 - val_loss: 0.8855 - val_accuracy: 0.9031
Epoch 945/1000
455/455 [==============================] - 0s 61us/step - loss: 1.1170 - accuracy: 0.7516 - val_loss: 0.8555 - val_accuracy: 0.8980
Epoch 946/1000
455/455 [==============================] - 0s 65us/step - loss: 1.2898 - accuracy: 0.7363 - val_loss: 0.7931 - val_accuracy: 0.9031
Epoch 947/1000
455/455 [==============================] - 0s 65us/step - loss: 1.2069 - accuracy: 0.7451 - val_loss: 0.7646 - val_accuracy: 0.9031
Epoch 948/1000
455/455 [==============================] - 0s 77us/step - loss: 1.1694 - accuracy: 0.7692 - val_loss: 0.7386 - val_accuracy: 0.9031
Epoch 949/1000
455/455 [==============================] - 0s 86us/step - loss: 0.8897 - accuracy: 0.7824 - val_loss: 0.7787 - val_acc

Epoch 999/1000
455/455 [==============================] - 0s 65us/step - loss: 1.1487 - accuracy: 0.7846 - val_loss: 0.8777 - val_accuracy: 0.9031
Epoch 1000/1000
455/455 [==============================] - 0s 70us/step - loss: 1.1255 - accuracy: 0.7451 - val_loss: 0.9091 - val_accuracy: 0.9031


In [64]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over5.history['accuracy'])*100))

over-sampling train accuracy: 75.96%


In [65]:
model1_over6 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [78]:
model1_over6.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [79]:
model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 520us/step - loss: 38.5452 - accuracy: 0.3846 - val_loss: 28.9891 - val_accuracy: 0.4133
Epoch 2/1000
455/455 [==============================] - 0s 113us/step - loss: 36.6061 - accuracy: 0.3495 - val_loss: 27.2941 - val_accuracy: 0.4439
Epoch 3/1000
455/455 [==============================] - 0s 110us/step - loss: 34.9940 - accuracy: 0.4176 - val_loss: 26.5359 - val_accuracy: 0.4133
Epoch 4/1000
455/455 [==============================] - 0s 126us/step - loss: 33.5346 - accuracy: 0.4396 - val_loss: 25.1226 - val_accuracy: 0.5153
Epoch 5/1000
455/455 [==============================] - 0s 101us/step - loss: 31.9236 - accuracy: 0.4330 - val_loss: 23.3065 - val_accuracy: 0.4694
Epoch 6/1000
455/455 [==============================] - 0s 95us/step - loss: 30.0906 - accuracy: 0.4615 - val_loss: 22.1477 - val_accuracy: 0.5000
Epoch 7/1000
455/455 [==============================] - 0s 109us/st

455/455 [==============================] - 0s 77us/step - loss: 3.4077 - accuracy: 0.6505 - val_loss: 0.9521 - val_accuracy: 0.8061
Epoch 57/1000
455/455 [==============================] - 0s 86us/step - loss: 3.4507 - accuracy: 0.6418 - val_loss: 0.8661 - val_accuracy: 0.7296
Epoch 58/1000
455/455 [==============================] - 0s 95us/step - loss: 3.6808 - accuracy: 0.6264 - val_loss: 0.8301 - val_accuracy: 0.8061
Epoch 59/1000
455/455 [==============================] - 0s 85us/step - loss: 3.3854 - accuracy: 0.6549 - val_loss: 0.7563 - val_accuracy: 0.7347
Epoch 60/1000
455/455 [==============================] - 0s 109us/step - loss: 3.2596 - accuracy: 0.6242 - val_loss: 0.7200 - val_accuracy: 0.7347
Epoch 61/1000
455/455 [==============================] - 0s 98us/step - loss: 3.6504 - accuracy: 0.5736 - val_loss: 0.7222 - val_accuracy: 0.7653
Epoch 62/1000
455/455 [==============================] - 0s 106us/step - loss: 3.9278 - accuracy: 0.5934 - val_loss: 0.7613 - val_accurac

455/455 [==============================] - 0s 68us/step - loss: 3.2456 - accuracy: 0.6352 - val_loss: 0.8553 - val_accuracy: 0.8061
Epoch 113/1000
455/455 [==============================] - 0s 57us/step - loss: 3.5778 - accuracy: 0.6396 - val_loss: 0.8484 - val_accuracy: 0.8112
Epoch 114/1000
455/455 [==============================] - 0s 63us/step - loss: 2.6299 - accuracy: 0.6813 - val_loss: 0.8146 - val_accuracy: 0.8163
Epoch 115/1000
455/455 [==============================] - 0s 62us/step - loss: 2.7318 - accuracy: 0.6835 - val_loss: 0.8038 - val_accuracy: 0.8163
Epoch 116/1000
455/455 [==============================] - 0s 60us/step - loss: 3.5372 - accuracy: 0.6220 - val_loss: 0.8199 - val_accuracy: 0.7857
Epoch 117/1000
455/455 [==============================] - 0s 65us/step - loss: 3.1996 - accuracy: 0.6044 - val_loss: 0.7875 - val_accuracy: 0.8010
Epoch 118/1000
455/455 [==============================] - 0s 63us/step - loss: 2.9167 - accuracy: 0.6352 - val_loss: 0.8102 - val_acc

Epoch 168/1000
455/455 [==============================] - 0s 110us/step - loss: 2.7912 - accuracy: 0.6945 - val_loss: 0.6889 - val_accuracy: 0.8265
Epoch 169/1000
455/455 [==============================] - 0s 95us/step - loss: 3.0414 - accuracy: 0.6615 - val_loss: 0.7903 - val_accuracy: 0.8214
Epoch 170/1000
455/455 [==============================] - 0s 93us/step - loss: 2.9984 - accuracy: 0.6703 - val_loss: 0.7944 - val_accuracy: 0.8163
Epoch 171/1000
455/455 [==============================] - 0s 100us/step - loss: 2.4536 - accuracy: 0.7055 - val_loss: 0.7317 - val_accuracy: 0.8163
Epoch 172/1000
455/455 [==============================] - 0s 98us/step - loss: 2.5232 - accuracy: 0.6923 - val_loss: 0.8097 - val_accuracy: 0.8061
Epoch 173/1000
455/455 [==============================] - 0s 131us/step - loss: 2.6407 - accuracy: 0.6967 - val_loss: 0.6711 - val_accuracy: 0.8724
Epoch 174/1000
455/455 [==============================] - 0s 90us/step - loss: 2.4774 - accuracy: 0.7143 - val_loss

Epoch 279/1000
455/455 [==============================] - 0s 77us/step - loss: 2.0093 - accuracy: 0.7187 - val_loss: 0.7900 - val_accuracy: 0.8929
Epoch 280/1000
455/455 [==============================] - 0s 71us/step - loss: 1.8155 - accuracy: 0.7209 - val_loss: 0.8112 - val_accuracy: 0.8878
Epoch 281/1000
455/455 [==============================] - 0s 68us/step - loss: 2.1622 - accuracy: 0.7341 - val_loss: 0.7727 - val_accuracy: 0.8929
Epoch 282/1000
455/455 [==============================] - 0s 61us/step - loss: 1.9456 - accuracy: 0.7516 - val_loss: 0.8205 - val_accuracy: 0.8571
Epoch 283/1000
455/455 [==============================] - 0s 60us/step - loss: 2.4437 - accuracy: 0.6923 - val_loss: 0.7834 - val_accuracy: 0.8673
Epoch 284/1000
455/455 [==============================] - 0s 61us/step - loss: 2.0173 - accuracy: 0.6945 - val_loss: 0.8341 - val_accuracy: 0.8571
Epoch 285/1000
455/455 [==============================] - 0s 62us/step - loss: 2.3101 - accuracy: 0.7033 - val_loss: 0

...

Epoch 670/1000
455/455 [==============================] - 0s 92us/step - loss: 1.3742 - accuracy: 0.7648 - val_loss: 0.5884 - val_accuracy: 0.9133
Epoch 671/1000
455/455 [==============================] - 0s 63us/step - loss: 1.2395 - accuracy: 0.7604 - val_loss: 0.5833 - val_accuracy: 0.9133
Epoch 672/1000
455/455 [==============================] - 0s 68us/step - loss: 1.5679 - accuracy: 0.7538 - val_loss: 0.5872 - val_accuracy: 0.9133
Epoch 673/1000
455/455 [==============================] - 0s 75us/step - loss: 1.5712 - accuracy: 0.7077 - val_loss: 0.6331 - val_accuracy: 0.9184
Epoch 674/1000
455/455 [==============================] - 0s 101us/step - loss: 1.4672 - accuracy: 0.7582 - val_loss: 0.5871 - val_accuracy: 0.9133
Epoch 675/1000
455/455 [==============================] - 0s 111us/step - loss: 1.3495 - accuracy: 0.7670 - val_loss: 0.5676 - val_accuracy: 0.9184
Epoch 676/1000
455/455 [==============================] - 0s 107us/step - loss: 1.2973 - accuracy: 0.7538 - val_loss

Epoch 726/1000
455/455 [==============================] - 0s 82us/step - loss: 1.3580 - accuracy: 0.7582 - val_loss: 0.6157 - val_accuracy: 0.9184
Epoch 727/1000
455/455 [==============================] - 0s 83us/step - loss: 1.6822 - accuracy: 0.7165 - val_loss: 0.5800 - val_accuracy: 0.9133
Epoch 728/1000
455/455 [==============================] - 0s 71us/step - loss: 1.6796 - accuracy: 0.7407 - val_loss: 0.5986 - val_accuracy: 0.9184
Epoch 729/1000
455/455 [==============================] - 0s 69us/step - loss: 1.5324 - accuracy: 0.7495 - val_loss: 0.5913 - val_accuracy: 0.9184
Epoch 730/1000
455/455 [==============================] - 0s 78us/step - loss: 1.4011 - accuracy: 0.7736 - val_loss: 0.5645 - val_accuracy: 0.9184
Epoch 731/1000
455/455 [==============================] - 0s 80us/step - loss: 1.4923 - accuracy: 0.7516 - val_loss: 0.5627 - val_accuracy: 0.9184
Epoch 732/1000
455/455 [==============================] - 0s 76us/step - loss: 1.2422 - accuracy: 0.7670 - val_loss: 0

Epoch 782/1000
455/455 [==============================] - 0s 66us/step - loss: 1.0889 - accuracy: 0.8110 - val_loss: 0.5886 - val_accuracy: 0.9235
Epoch 783/1000
455/455 [==============================] - 0s 63us/step - loss: 1.3505 - accuracy: 0.7736 - val_loss: 0.6025 - val_accuracy: 0.9235
Epoch 784/1000
455/455 [==============================] - 0s 63us/step - loss: 1.3841 - accuracy: 0.7736 - val_loss: 0.5797 - val_accuracy: 0.9235
Epoch 785/1000
455/455 [==============================] - 0s 62us/step - loss: 1.7864 - accuracy: 0.7187 - val_loss: 0.5743 - val_accuracy: 0.9235
Epoch 786/1000
455/455 [==============================] - 0s 68us/step - loss: 1.2867 - accuracy: 0.7846 - val_loss: 0.5894 - val_accuracy: 0.9133
Epoch 787/1000
455/455 [==============================] - 0s 70us/step - loss: 1.4253 - accuracy: 0.7516 - val_loss: 0.6049 - val_accuracy: 0.9235
Epoch 788/1000
455/455 [==============================] - 0s 73us/step - loss: 1.4662 - accuracy: 0.7429 - val_loss: 0

Epoch 838/1000
455/455 [==============================] - 0s 78us/step - loss: 1.6068 - accuracy: 0.7407 - val_loss: 0.5617 - val_accuracy: 0.9184
Epoch 839/1000
455/455 [==============================] - 0s 78us/step - loss: 1.7314 - accuracy: 0.7055 - val_loss: 0.5746 - val_accuracy: 0.9184
Epoch 840/1000
455/455 [==============================] - 0s 85us/step - loss: 1.3067 - accuracy: 0.7670 - val_loss: 0.5671 - val_accuracy: 0.9184
Epoch 841/1000
455/455 [==============================] - 0s 79us/step - loss: 1.4201 - accuracy: 0.7495 - val_loss: 0.5616 - val_accuracy: 0.9184
Epoch 842/1000
455/455 [==============================] - 0s 69us/step - loss: 1.4100 - accuracy: 0.7604 - val_loss: 0.5566 - val_accuracy: 0.9184
Epoch 843/1000
455/455 [==============================] - 0s 64us/step - loss: 1.2176 - accuracy: 0.7692 - val_loss: 0.5557 - val_accuracy: 0.9184
Epoch 844/1000
455/455 [==============================] - 0s 69us/step - loss: 1.4407 - accuracy: 0.7560 - val_loss: 0

Epoch 894/1000
455/455 [==============================] - 0s 65us/step - loss: 1.5023 - accuracy: 0.7560 - val_loss: 0.5675 - val_accuracy: 0.9184
Epoch 895/1000
455/455 [==============================] - 0s 70us/step - loss: 1.1239 - accuracy: 0.7758 - val_loss: 0.5453 - val_accuracy: 0.9184
Epoch 896/1000
455/455 [==============================] - 0s 81us/step - loss: 1.2745 - accuracy: 0.7582 - val_loss: 0.5677 - val_accuracy: 0.9235
Epoch 897/1000
455/455 [==============================] - 0s 78us/step - loss: 1.3259 - accuracy: 0.7758 - val_loss: 0.5802 - val_accuracy: 0.9031
Epoch 898/1000
455/455 [==============================] - 0s 93us/step - loss: 1.5736 - accuracy: 0.7407 - val_loss: 0.5738 - val_accuracy: 0.9184
Epoch 899/1000
455/455 [==============================] - 0s 95us/step - loss: 1.2983 - accuracy: 0.7692 - val_loss: 0.5830 - val_accuracy: 0.9184
Epoch 900/1000
455/455 [==============================] - 0s 92us/step - loss: 1.4088 - accuracy: 0.7407 - val_loss: 0

In [80]:
acc_test_over6 = model1_over6.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over6*100))

196/196 [==============================] - 0s 108us/step
over-sampling test accuracy: 92.35%


In [81]:
hist1_over6 = model1_over6.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 92us/step - loss: 1.3813 - accuracy: 0.7868 - val_loss: 0.5808 - val_accuracy: 0.9133
Epoch 2/1000
455/455 [==============================] - 0s 67us/step - loss: 1.3382 - accuracy: 0.7780 - val_loss: 0.6167 - val_accuracy: 0.9286
Epoch 3/1000
455/455 [==============================] - 0s 60us/step - loss: 1.2819 - accuracy: 0.7516 - val_loss: 0.5572 - val_accuracy: 0.9184
Epoch 4/1000
455/455 [==============================] - 0s 59us/step - loss: 1.3173 - accuracy: 0.7626 - val_loss: 0.5850 - val_accuracy: 0.9133
Epoch 5/1000
455/455 [==============================] - 0s 59us/step - loss: 1.3946 - accuracy: 0.7538 - val_loss: 0.5870 - val_accuracy: 0.9184
Epoch 6/1000
455/455 [==============================] - 0s 60us/step - loss: 1.3694 - accuracy: 0.7451 - val_loss: 0.5623 - val_accuracy: 0.9235
Epoch 7/1000
455/455 [==============================] - 0s 60us/step - loss: 1.6684 

Epoch 57/1000
455/455 [==============================] - 0s 87us/step - loss: 1.1413 - accuracy: 0.7824 - val_loss: 0.6430 - val_accuracy: 0.9235
Epoch 58/1000
455/455 [==============================] - 0s 70us/step - loss: 1.4808 - accuracy: 0.7231 - val_loss: 0.6151 - val_accuracy: 0.9235
Epoch 59/1000
455/455 [==============================] - 0s 69us/step - loss: 1.2931 - accuracy: 0.7780 - val_loss: 0.5852 - val_accuracy: 0.9184
Epoch 60/1000
455/455 [==============================] - 0s 65us/step - loss: 1.2457 - accuracy: 0.7648 - val_loss: 0.5909 - val_accuracy: 0.9235
Epoch 61/1000
455/455 [==============================] - 0s 63us/step - loss: 1.4541 - accuracy: 0.7648 - val_loss: 0.5756 - val_accuracy: 0.9184
Epoch 62/1000
455/455 [==============================] - 0s 68us/step - loss: 1.2973 - accuracy: 0.7407 - val_loss: 0.5620 - val_accuracy: 0.9133
Epoch 63/1000
455/455 [==============================] - 0s 81us/step - loss: 1.3723 - accuracy: 0.7802 - val_loss: 0.5573 -

Epoch 113/1000
455/455 [==============================] - 0s 66us/step - loss: 1.2715 - accuracy: 0.7582 - val_loss: 0.5678 - val_accuracy: 0.9184
Epoch 114/1000
455/455 [==============================] - 0s 76us/step - loss: 1.3707 - accuracy: 0.7714 - val_loss: 0.5744 - val_accuracy: 0.9184
Epoch 115/1000
455/455 [==============================] - 0s 86us/step - loss: 1.4017 - accuracy: 0.7802 - val_loss: 0.5674 - val_accuracy: 0.9133
Epoch 116/1000
455/455 [==============================] - 0s 81us/step - loss: 1.0436 - accuracy: 0.7912 - val_loss: 0.5618 - val_accuracy: 0.9133
Epoch 117/1000
455/455 [==============================] - 0s 92us/step - loss: 1.3542 - accuracy: 0.7121 - val_loss: 0.5967 - val_accuracy: 0.9184
Epoch 118/1000
455/455 [==============================] - 0s 96us/step - loss: 1.2963 - accuracy: 0.7604 - val_loss: 0.6088 - val_accuracy: 0.9184
Epoch 119/1000
455/455 [==============================] - 0s 93us/step - loss: 1.3270 - accuracy: 0.7648 - val_loss: 0

Epoch 169/1000
455/455 [==============================] - 0s 66us/step - loss: 1.3246 - accuracy: 0.7451 - val_loss: 0.5705 - val_accuracy: 0.9235
Epoch 170/1000
455/455 [==============================] - 0s 62us/step - loss: 1.1983 - accuracy: 0.7604 - val_loss: 0.6597 - val_accuracy: 0.9235
Epoch 171/1000
455/455 [==============================] - 0s 62us/step - loss: 1.5019 - accuracy: 0.7604 - val_loss: 0.5644 - val_accuracy: 0.9184
Epoch 172/1000
455/455 [==============================] - 0s 75us/step - loss: 1.2050 - accuracy: 0.7780 - val_loss: 0.5752 - val_accuracy: 0.9184
Epoch 173/1000
455/455 [==============================] - 0s 147us/step - loss: 0.9960 - accuracy: 0.7780 - val_loss: 0.5773 - val_accuracy: 0.9235
Epoch 174/1000
455/455 [==============================] - 0s 208us/step - loss: 1.5490 - accuracy: 0.7253 - val_loss: 0.5628 - val_accuracy: 0.9184
Epoch 175/1000
455/455 [==============================] - 0s 220us/step - loss: 1.1266 - accuracy: 0.7868 - val_loss

Epoch 280/1000
455/455 [==============================] - 0s 94us/step - loss: 1.2190 - accuracy: 0.7495 - val_loss: 0.5634 - val_accuracy: 0.9184
Epoch 281/1000
455/455 [==============================] - 0s 82us/step - loss: 1.3359 - accuracy: 0.7538 - val_loss: 0.6123 - val_accuracy: 0.9184
Epoch 282/1000
455/455 [==============================] - 0s 73us/step - loss: 1.5152 - accuracy: 0.7429 - val_loss: 0.6302 - val_accuracy: 0.9184
Epoch 283/1000
455/455 [==============================] - 0s 68us/step - loss: 1.3878 - accuracy: 0.7451 - val_loss: 0.6962 - val_accuracy: 0.9235
Epoch 284/1000
455/455 [==============================] - 0s 64us/step - loss: 1.2986 - accuracy: 0.7648 - val_loss: 0.6439 - val_accuracy: 0.9235
Epoch 285/1000
455/455 [==============================] - 0s 69us/step - loss: 1.0017 - accuracy: 0.7758 - val_loss: 0.5754 - val_accuracy: 0.9184
Epoch 286/1000
455/455 [==============================] - 0s 63us/step - loss: 1.3070 - accuracy: 0.7495 - val_loss: 0

...

Epoch 726/1000
455/455 [==============================] - 0s 68us/step - loss: 1.3301 - accuracy: 0.7626 - val_loss: 0.6564 - val_accuracy: 0.9235
Epoch 727/1000
455/455 [==============================] - 0s 66us/step - loss: 1.5301 - accuracy: 0.7275 - val_loss: 0.5757 - val_accuracy: 0.9235
Epoch 728/1000
455/455 [==============================] - 0s 63us/step - loss: 1.1461 - accuracy: 0.7495 - val_loss: 0.6200 - val_accuracy: 0.9235
Epoch 729/1000
455/455 [==============================] - 0s 91us/step - loss: 1.2050 - accuracy: 0.7692 - val_loss: 0.5748 - val_accuracy: 0.9286
Epoch 730/1000
455/455 [==============================] - 0s 81us/step - loss: 1.1219 - accuracy: 0.7868 - val_loss: 0.5653 - val_accuracy: 0.9235
Epoch 731/1000
455/455 [==============================] - 0s 109us/step - loss: 1.1703 - accuracy: 0.7516 - val_loss: 0.6004 - val_accuracy: 0.9235
Epoch 732/1000
455/455 [==============================] - 0s 77us/step - loss: 1.1918 - accuracy: 0.7626 - val_loss: 

Epoch 782/1000
455/455 [==============================] - 0s 88us/step - loss: 1.1546 - accuracy: 0.7626 - val_loss: 0.6467 - val_accuracy: 0.9184
Epoch 783/1000
455/455 [==============================] - 0s 84us/step - loss: 1.1213 - accuracy: 0.7604 - val_loss: 0.6641 - val_accuracy: 0.9184
Epoch 784/1000
455/455 [==============================] - 0s 79us/step - loss: 1.2026 - accuracy: 0.7758 - val_loss: 0.6339 - val_accuracy: 0.9235
Epoch 785/1000
455/455 [==============================] - 0s 81us/step - loss: 1.1661 - accuracy: 0.7802 - val_loss: 0.6779 - val_accuracy: 0.9286
Epoch 786/1000
455/455 [==============================] - 0s 91us/step - loss: 0.9696 - accuracy: 0.8022 - val_loss: 0.5718 - val_accuracy: 0.9184
Epoch 787/1000
455/455 [==============================] - 0s 74us/step - loss: 1.1957 - accuracy: 0.7495 - val_loss: 0.5507 - val_accuracy: 0.9184
Epoch 788/1000
455/455 [==============================] - 0s 68us/step - loss: 1.1095 - accuracy: 0.7802 - val_loss: 0

Epoch 838/1000
455/455 [==============================] - 0s 79us/step - loss: 1.5051 - accuracy: 0.7363 - val_loss: 0.6927 - val_accuracy: 0.9235
Epoch 839/1000
455/455 [==============================] - 0s 72us/step - loss: 1.2616 - accuracy: 0.7429 - val_loss: 0.6121 - val_accuracy: 0.9235
Epoch 840/1000
455/455 [==============================] - 0s 64us/step - loss: 1.1060 - accuracy: 0.7648 - val_loss: 0.6384 - val_accuracy: 0.9184
Epoch 841/1000
455/455 [==============================] - 0s 60us/step - loss: 1.1688 - accuracy: 0.7714 - val_loss: 0.6235 - val_accuracy: 0.9235
Epoch 842/1000
455/455 [==============================] - 0s 65us/step - loss: 1.2633 - accuracy: 0.7670 - val_loss: 0.6010 - val_accuracy: 0.9235
Epoch 843/1000
455/455 [==============================] - 0s 68us/step - loss: 1.3898 - accuracy: 0.7231 - val_loss: 0.6917 - val_accuracy: 0.9235
Epoch 844/1000
455/455 [==============================] - 0s 85us/step - loss: 1.4400 - accuracy: 0.7626 - val_loss: 0

Epoch 894/1000
455/455 [==============================] - 0s 80us/step - loss: 0.8765 - accuracy: 0.7846 - val_loss: 0.6219 - val_accuracy: 0.9235
Epoch 895/1000
455/455 [==============================] - 0s 65us/step - loss: 1.1883 - accuracy: 0.7714 - val_loss: 0.6075 - val_accuracy: 0.9235
Epoch 896/1000
455/455 [==============================] - 0s 68us/step - loss: 1.1397 - accuracy: 0.7582 - val_loss: 0.6085 - val_accuracy: 0.9235
Epoch 897/1000
455/455 [==============================] - 0s 71us/step - loss: 1.0680 - accuracy: 0.7714 - val_loss: 0.5879 - val_accuracy: 0.9235
Epoch 898/1000
455/455 [==============================] - 0s 80us/step - loss: 0.9015 - accuracy: 0.8000 - val_loss: 0.5661 - val_accuracy: 0.9184
Epoch 899/1000
455/455 [==============================] - 0s 79us/step - loss: 1.0425 - accuracy: 0.7626 - val_loss: 0.5993 - val_accuracy: 0.9184
Epoch 900/1000
455/455 [==============================] - 0s 83us/step - loss: 1.3835 - accuracy: 0.7363 - val_loss: 0

Epoch 950/1000
455/455 [==============================] - 0s 63us/step - loss: 1.0617 - accuracy: 0.7714 - val_loss: 0.6309 - val_accuracy: 0.9184
Epoch 951/1000
455/455 [==============================] - 0s 60us/step - loss: 1.0630 - accuracy: 0.7648 - val_loss: 0.6111 - val_accuracy: 0.9184
Epoch 952/1000
455/455 [==============================] - 0s 105us/step - loss: 1.3391 - accuracy: 0.7560 - val_loss: 0.6078 - val_accuracy: 0.9235
Epoch 953/1000
455/455 [==============================] - 0s 94us/step - loss: 1.0458 - accuracy: 0.7626 - val_loss: 0.5922 - val_accuracy: 0.9184
Epoch 954/1000
455/455 [==============================] - 0s 70us/step - loss: 1.3707 - accuracy: 0.7407 - val_loss: 0.7286 - val_accuracy: 0.9133
Epoch 955/1000
455/455 [==============================] - 0s 66us/step - loss: 1.3438 - accuracy: 0.7231 - val_loss: 0.6625 - val_accuracy: 0.9235
Epoch 956/1000
455/455 [==============================] - 0s 107us/step - loss: 1.1234 - accuracy: 0.7714 - val_loss:

In [82]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over6.history['accuracy'])*100))

over-sampling train accuracy: 75.74%


In [83]:
model1_over7 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [84]:
model1_over7.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [85]:
model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 501us/step - loss: 20.4718 - accuracy: 0.2308 - val_loss: 15.8158 - val_accuracy: 0.2194
Epoch 2/1000
455/455 [==============================] - 0s 112us/step - loss: 18.5883 - accuracy: 0.2352 - val_loss: 13.8802 - val_accuracy: 0.1735
Epoch 3/1000
455/455 [==============================] - 0s 120us/step - loss: 17.6948 - accuracy: 0.2132 - val_loss: 12.2731 - val_accuracy: 0.2602
Epoch 4/1000
455/455 [==============================] - 0s 111us/step - loss: 14.7428 - accuracy: 0.4110 - val_loss: 10.4366 - val_accuracy: 0.5051
Epoch 5/1000
455/455 [==============================] - 0s 91us/step - loss: 13.8834 - accuracy: 0.4088 - val_loss: 9.6587 - val_accuracy: 0.4490
Epoch 6/1000
455/455 [==============================] - 0s 79us/step - loss: 13.5637 - accuracy: 0.4220 - val_loss: 8.9929 - val_accuracy: 0.4541
Epoch 7/1000
455/455 [==============================] - 0s 75us/step -

455/455 [==============================] - 0s 72us/step - loss: 3.1676 - accuracy: 0.6374 - val_loss: 1.0166 - val_accuracy: 0.7755
Epoch 113/1000
455/455 [==============================] - 0s 68us/step - loss: 3.0835 - accuracy: 0.6484 - val_loss: 0.9654 - val_accuracy: 0.7908
Epoch 114/1000
455/455 [==============================] - 0s 71us/step - loss: 3.0243 - accuracy: 0.6396 - val_loss: 0.7503 - val_accuracy: 0.8214
Epoch 115/1000
455/455 [==============================] - 0s 65us/step - loss: 2.8621 - accuracy: 0.6615 - val_loss: 0.6910 - val_accuracy: 0.8469
Epoch 116/1000
455/455 [==============================] - 0s 62us/step - loss: 2.8063 - accuracy: 0.6703 - val_loss: 0.7140 - val_accuracy: 0.8163
Epoch 117/1000
455/455 [==============================] - 0s 70us/step - loss: 2.9604 - accuracy: 0.6615 - val_loss: 0.7175 - val_accuracy: 0.8265
Epoch 118/1000
455/455 [==============================] - 0s 73us/step - loss: 3.4590 - accuracy: 0.6242 - val_loss: 0.7655 - val_acc

Epoch 168/1000
455/455 [==============================] - 0s 68us/step - loss: 2.7081 - accuracy: 0.6571 - val_loss: 0.7403 - val_accuracy: 0.8265
Epoch 169/1000
455/455 [==============================] - 0s 105us/step - loss: 2.1799 - accuracy: 0.7099 - val_loss: 0.7050 - val_accuracy: 0.8520
Epoch 170/1000
455/455 [==============================] - 0s 113us/step - loss: 2.5169 - accuracy: 0.6945 - val_loss: 0.7954 - val_accuracy: 0.8214
Epoch 171/1000
455/455 [==============================] - 0s 78us/step - loss: 2.5590 - accuracy: 0.6571 - val_loss: 0.7573 - val_accuracy: 0.8418
Epoch 172/1000
455/455 [==============================] - 0s 68us/step - loss: 2.7373 - accuracy: 0.6571 - val_loss: 0.7747 - val_accuracy: 0.8367
Epoch 173/1000
455/455 [==============================] - 0s 63us/step - loss: 2.8224 - accuracy: 0.6396 - val_loss: 0.8188 - val_accuracy: 0.8571
Epoch 174/1000
455/455 [==============================] - 0s 66us/step - loss: 2.2234 - accuracy: 0.6703 - val_loss:

455/455 [==============================] - 0s 73us/step - loss: 1.9282 - accuracy: 0.7077 - val_loss: 0.6069 - val_accuracy: 0.9082
Epoch 279/1000
455/455 [==============================] - 0s 67us/step - loss: 2.0331 - accuracy: 0.7165 - val_loss: 0.6297 - val_accuracy: 0.9082
Epoch 280/1000
455/455 [==============================] - 0s 72us/step - loss: 2.1002 - accuracy: 0.7055 - val_loss: 0.6332 - val_accuracy: 0.9133
Epoch 281/1000
455/455 [==============================] - 0s 68us/step - loss: 2.0969 - accuracy: 0.7297 - val_loss: 0.6416 - val_accuracy: 0.9184
Epoch 282/1000
455/455 [==============================] - 0s 59us/step - loss: 2.3670 - accuracy: 0.7099 - val_loss: 0.6211 - val_accuracy: 0.9133
Epoch 283/1000
455/455 [==============================] - 0s 69us/step - loss: 2.0556 - accuracy: 0.7055 - val_loss: 0.6355 - val_accuracy: 0.9082
Epoch 284/1000
455/455 [==============================] - 0s 68us/step - loss: 1.9489 - accuracy: 0.7275 - val_loss: 0.6356 - val_acc

Epoch 334/1000
455/455 [==============================] - 0s 69us/step - loss: 1.5981 - accuracy: 0.7429 - val_loss: 0.6254 - val_accuracy: 0.9184
Epoch 335/1000
455/455 [==============================] - 0s 82us/step - loss: 1.7607 - accuracy: 0.7077 - val_loss: 0.6405 - val_accuracy: 0.9184
Epoch 336/1000
455/455 [==============================] - 0s 91us/step - loss: 2.0273 - accuracy: 0.7253 - val_loss: 0.6235 - val_accuracy: 0.9133
Epoch 337/1000
455/455 [==============================] - 0s 93us/step - loss: 1.9297 - accuracy: 0.7187 - val_loss: 0.6245 - val_accuracy: 0.9184
Epoch 338/1000
455/455 [==============================] - 0s 107us/step - loss: 1.9098 - accuracy: 0.7077 - val_loss: 0.6356 - val_accuracy: 0.9184
Epoch 339/1000
455/455 [==============================] - 0s 94us/step - loss: 1.7924 - accuracy: 0.7582 - val_loss: 0.6116 - val_accuracy: 0.9082
Epoch 340/1000
455/455 [==============================] - 0s 100us/step - loss: 1.7993 - accuracy: 0.7011 - val_loss:

...

Epoch 669/1000
455/455 [==============================] - 0s 63us/step - loss: 1.5120 - accuracy: 0.7407 - val_loss: 0.5980 - val_accuracy: 0.9286
Epoch 670/1000
455/455 [==============================] - 0s 94us/step - loss: 1.2948 - accuracy: 0.7341 - val_loss: 0.5762 - val_accuracy: 0.9235
Epoch 671/1000
455/455 [==============================] - 0s 98us/step - loss: 1.6775 - accuracy: 0.7209 - val_loss: 0.5890 - val_accuracy: 0.9337
Epoch 672/1000
455/455 [==============================] - 0s 75us/step - loss: 1.5070 - accuracy: 0.7363 - val_loss: 0.5824 - val_accuracy: 0.9235
Epoch 673/1000
455/455 [==============================] - 0s 71us/step - loss: 1.4594 - accuracy: 0.7275 - val_loss: 0.5853 - val_accuracy: 0.9337
Epoch 674/1000
455/455 [==============================] - 0s 136us/step - loss: 1.5559 - accuracy: 0.7451 - val_loss: 0.6075 - val_accuracy: 0.9337
Epoch 675/1000
455/455 [==============================] - 0s 137us/step - loss: 1.2050 - accuracy: 0.7516 - val_loss:

Epoch 725/1000
455/455 [==============================] - 0s 71us/step - loss: 1.4843 - accuracy: 0.7429 - val_loss: 0.5763 - val_accuracy: 0.9235
Epoch 726/1000
455/455 [==============================] - 0s 71us/step - loss: 1.4697 - accuracy: 0.7363 - val_loss: 0.5804 - val_accuracy: 0.9286
Epoch 727/1000
455/455 [==============================] - 0s 71us/step - loss: 1.0933 - accuracy: 0.7758 - val_loss: 0.5603 - val_accuracy: 0.9184
Epoch 728/1000
455/455 [==============================] - 0s 73us/step - loss: 1.5716 - accuracy: 0.7385 - val_loss: 0.5811 - val_accuracy: 0.9184
Epoch 729/1000
455/455 [==============================] - 0s 75us/step - loss: 1.6167 - accuracy: 0.7231 - val_loss: 0.5977 - val_accuracy: 0.9286
Epoch 730/1000
455/455 [==============================] - 0s 76us/step - loss: 1.5875 - accuracy: 0.7077 - val_loss: 0.5866 - val_accuracy: 0.9184
Epoch 731/1000
455/455 [==============================] - 0s 81us/step - loss: 1.1448 - accuracy: 0.7802 - val_loss: 0

Epoch 781/1000
455/455 [==============================] - 0s 109us/step - loss: 1.2865 - accuracy: 0.7516 - val_loss: 0.5586 - val_accuracy: 0.9184
Epoch 782/1000
455/455 [==============================] - 0s 103us/step - loss: 1.4637 - accuracy: 0.7363 - val_loss: 0.5543 - val_accuracy: 0.9184
Epoch 783/1000
455/455 [==============================] - 0s 77us/step - loss: 1.1868 - accuracy: 0.7473 - val_loss: 0.5758 - val_accuracy: 0.9133
Epoch 784/1000
455/455 [==============================] - 0s 80us/step - loss: 1.3637 - accuracy: 0.7670 - val_loss: 0.5416 - val_accuracy: 0.9184
Epoch 785/1000
455/455 [==============================] - 0s 100us/step - loss: 1.3909 - accuracy: 0.7516 - val_loss: 0.5718 - val_accuracy: 0.9286
Epoch 786/1000
455/455 [==============================] - 0s 91us/step - loss: 1.1236 - accuracy: 0.7385 - val_loss: 0.5750 - val_accuracy: 0.9184
Epoch 787/1000
455/455 [==============================] - 0s 88us/step - loss: 1.0718 - accuracy: 0.7912 - val_loss

Epoch 892/1000
455/455 [==============================] - 0s 63us/step - loss: 1.3318 - accuracy: 0.7626 - val_loss: 0.5570 - val_accuracy: 0.9184
Epoch 893/1000
455/455 [==============================] - 0s 61us/step - loss: 1.4150 - accuracy: 0.7758 - val_loss: 0.5576 - val_accuracy: 0.9184
Epoch 894/1000
455/455 [==============================] - 0s 89us/step - loss: 1.0968 - accuracy: 0.7890 - val_loss: 0.5667 - val_accuracy: 0.9184
Epoch 895/1000
455/455 [==============================] - 0s 106us/step - loss: 1.4141 - accuracy: 0.7604 - val_loss: 0.5607 - val_accuracy: 0.9184
Epoch 896/1000
455/455 [==============================] - 0s 76us/step - loss: 1.5377 - accuracy: 0.7626 - val_loss: 0.5580 - val_accuracy: 0.9184
Epoch 897/1000
455/455 [==============================] - 0s 66us/step - loss: 1.2717 - accuracy: 0.7604 - val_loss: 0.5526 - val_accuracy: 0.9184
Epoch 898/1000
455/455 [==============================] - 0s 64us/step - loss: 1.3459 - accuracy: 0.7560 - val_loss: 

Epoch 948/1000
455/455 [==============================] - 0s 89us/step - loss: 1.2974 - accuracy: 0.7319 - val_loss: 0.5201 - val_accuracy: 0.9235
Epoch 949/1000
455/455 [==============================] - 0s 86us/step - loss: 1.3552 - accuracy: 0.7385 - val_loss: 0.5286 - val_accuracy: 0.9235
Epoch 950/1000
455/455 [==============================] - 0s 90us/step - loss: 1.5131 - accuracy: 0.7363 - val_loss: 0.5765 - val_accuracy: 0.9286
Epoch 951/1000
455/455 [==============================] - 0s 83us/step - loss: 1.3427 - accuracy: 0.7187 - val_loss: 0.5697 - val_accuracy: 0.9184
Epoch 952/1000
455/455 [==============================] - 0s 76us/step - loss: 1.3198 - accuracy: 0.7692 - val_loss: 0.5536 - val_accuracy: 0.9184
Epoch 953/1000
455/455 [==============================] - 0s 74us/step - loss: 1.3309 - accuracy: 0.7407 - val_loss: 0.5612 - val_accuracy: 0.9184
Epoch 954/1000
455/455 [==============================] - 0s 65us/step - loss: 1.4095 - accuracy: 0.7451 - val_loss: 0

In [86]:
acc_test_over7 = model1_over7.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over7*100))

196/196 [==============================] - 0s 71us/step
over-sampling test accuracy: 91.84%


In [87]:
hist1_over7 = model1_over7.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 78us/step - loss: 1.3750 - accuracy: 0.7363 - val_loss: 0.5663 - val_accuracy: 0.9184
Epoch 2/1000
455/455 [==============================] - 0s 71us/step - loss: 1.3675 - accuracy: 0.7451 - val_loss: 0.5585 - val_accuracy: 0.9184
Epoch 3/1000
455/455 [==============================] - 0s 73us/step - loss: 1.3416 - accuracy: 0.7385 - val_loss: 0.5364 - val_accuracy: 0.9184
Epoch 4/1000
455/455 [==============================] - 0s 155us/step - loss: 1.4131 - accuracy: 0.7516 - val_loss: 0.5666 - val_accuracy: 0.9235
Epoch 5/1000
455/455 [==============================] - 0s 205us/step - loss: 1.3272 - accuracy: 0.7253 - val_loss: 0.5364 - val_accuracy: 0.9184
Epoch 6/1000
455/455 [==============================] - 0s 92us/step - loss: 1.5675 - accuracy: 0.7385 - val_loss: 0.5420 - val_accuracy: 0.9235
Epoch 7/1000
455/455 [==============================] - 0s 64us/step - loss: 1.132

455/455 [==============================] - 0s 80us/step - loss: 1.2705 - accuracy: 0.7802 - val_loss: 0.5594 - val_accuracy: 0.9184
Epoch 113/1000
455/455 [==============================] - 0s 79us/step - loss: 1.1356 - accuracy: 0.7824 - val_loss: 0.5614 - val_accuracy: 0.9235
Epoch 114/1000
455/455 [==============================] - 0s 82us/step - loss: 1.1000 - accuracy: 0.7582 - val_loss: 0.5525 - val_accuracy: 0.9235
Epoch 115/1000
455/455 [==============================] - 0s 80us/step - loss: 1.4487 - accuracy: 0.7473 - val_loss: 0.5516 - val_accuracy: 0.9235
Epoch 116/1000
455/455 [==============================] - 0s 79us/step - loss: 1.1622 - accuracy: 0.7692 - val_loss: 0.5442 - val_accuracy: 0.9184
Epoch 117/1000
455/455 [==============================] - 0s 86us/step - loss: 1.3092 - accuracy: 0.7560 - val_loss: 0.5644 - val_accuracy: 0.9184
Epoch 118/1000
455/455 [==============================] - 0s 92us/step - loss: 1.2100 - accuracy: 0.7604 - val_loss: 0.5790 - val_acc

Epoch 168/1000
455/455 [==============================] - 0s 87us/step - loss: 1.1833 - accuracy: 0.7538 - val_loss: 0.5272 - val_accuracy: 0.9184
Epoch 169/1000
455/455 [==============================] - 0s 77us/step - loss: 1.2865 - accuracy: 0.7429 - val_loss: 0.5424 - val_accuracy: 0.9235
Epoch 170/1000
455/455 [==============================] - 0s 82us/step - loss: 1.2172 - accuracy: 0.7473 - val_loss: 0.5348 - val_accuracy: 0.9235
Epoch 171/1000
455/455 [==============================] - 0s 83us/step - loss: 1.0518 - accuracy: 0.8000 - val_loss: 0.5450 - val_accuracy: 0.9235
Epoch 172/1000
455/455 [==============================] - 0s 81us/step - loss: 1.3354 - accuracy: 0.7538 - val_loss: 0.5301 - val_accuracy: 0.9184
Epoch 173/1000
455/455 [==============================] - 0s 76us/step - loss: 1.1602 - accuracy: 0.7758 - val_loss: 0.5318 - val_accuracy: 0.9235
Epoch 174/1000
455/455 [==============================] - 0s 80us/step - loss: 1.1132 - accuracy: 0.7692 - val_loss: 0

Epoch 224/1000
455/455 [==============================] - 0s 75us/step - loss: 1.1522 - accuracy: 0.7385 - val_loss: 0.5621 - val_accuracy: 0.9235
Epoch 225/1000
455/455 [==============================] - 0s 84us/step - loss: 1.2420 - accuracy: 0.7297 - val_loss: 0.5642 - val_accuracy: 0.9184
Epoch 226/1000
455/455 [==============================] - 0s 93us/step - loss: 1.2743 - accuracy: 0.7407 - val_loss: 0.6292 - val_accuracy: 0.9235
Epoch 227/1000
455/455 [==============================] - 0s 92us/step - loss: 1.3141 - accuracy: 0.7297 - val_loss: 0.5420 - val_accuracy: 0.9184
Epoch 228/1000
455/455 [==============================] - 0s 78us/step - loss: 1.0993 - accuracy: 0.7846 - val_loss: 0.5308 - val_accuracy: 0.9184
Epoch 229/1000
455/455 [==============================] - 0s 69us/step - loss: 1.1420 - accuracy: 0.7451 - val_loss: 0.5272 - val_accuracy: 0.9184
Epoch 230/1000
455/455 [==============================] - 0s 75us/step - loss: 1.2624 - accuracy: 0.7692 - val_loss: 0

Epoch 280/1000
455/455 [==============================] - 0s 69us/step - loss: 1.3463 - accuracy: 0.7297 - val_loss: 0.5418 - val_accuracy: 0.9184
Epoch 281/1000
455/455 [==============================] - 0s 76us/step - loss: 1.1974 - accuracy: 0.7538 - val_loss: 0.5541 - val_accuracy: 0.9235
Epoch 282/1000
455/455 [==============================] - 0s 90us/step - loss: 1.3874 - accuracy: 0.7582 - val_loss: 0.5502 - val_accuracy: 0.9184
Epoch 283/1000
455/455 [==============================] - 0s 96us/step - loss: 1.3020 - accuracy: 0.7407 - val_loss: 0.5635 - val_accuracy: 0.9184
Epoch 284/1000
455/455 [==============================] - 0s 102us/step - loss: 1.3262 - accuracy: 0.7714 - val_loss: 0.5505 - val_accuracy: 0.9184
Epoch 285/1000
455/455 [==============================] - 0s 108us/step - loss: 1.2908 - accuracy: 0.7407 - val_loss: 0.5403 - val_accuracy: 0.9082
Epoch 286/1000
455/455 [==============================] - 0s 101us/step - loss: 1.2919 - accuracy: 0.7648 - val_loss

...

Epoch 613/1000
455/455 [==============================] - 0s 78us/step - loss: 1.1150 - accuracy: 0.7714 - val_loss: 0.5781 - val_accuracy: 0.9133
Epoch 614/1000
455/455 [==============================] - 0s 75us/step - loss: 1.0917 - accuracy: 0.7626 - val_loss: 0.5705 - val_accuracy: 0.9133
Epoch 615/1000
455/455 [==============================] - 0s 72us/step - loss: 1.3350 - accuracy: 0.7275 - val_loss: 0.5519 - val_accuracy: 0.9133
Epoch 616/1000
455/455 [==============================] - 0s 67us/step - loss: 1.3291 - accuracy: 0.7429 - val_loss: 0.5544 - val_accuracy: 0.9133
Epoch 617/1000
455/455 [==============================] - 0s 70us/step - loss: 1.0849 - accuracy: 0.7648 - val_loss: 0.5696 - val_accuracy: 0.9031
Epoch 618/1000
455/455 [==============================] - 0s 69us/step - loss: 0.9359 - accuracy: 0.8022 - val_loss: 0.5678 - val_accuracy: 0.9184
Epoch 619/1000
455/455 [==============================] - 0s 69us/step - loss: 1.1838 - accuracy: 0.7516 - val_loss: 0

Epoch 669/1000
455/455 [==============================] - 0s 104us/step - loss: 1.3895 - accuracy: 0.7319 - val_loss: 0.5504 - val_accuracy: 0.9184
Epoch 670/1000
455/455 [==============================] - 0s 112us/step - loss: 1.0974 - accuracy: 0.7626 - val_loss: 0.5507 - val_accuracy: 0.9133
Epoch 671/1000
455/455 [==============================] - 0s 110us/step - loss: 1.0884 - accuracy: 0.7714 - val_loss: 0.5378 - val_accuracy: 0.9133
Epoch 672/1000
455/455 [==============================] - 0s 85us/step - loss: 1.2991 - accuracy: 0.7495 - val_loss: 0.5401 - val_accuracy: 0.9133
Epoch 673/1000
455/455 [==============================] - 0s 94us/step - loss: 1.4378 - accuracy: 0.7033 - val_loss: 0.5615 - val_accuracy: 0.9184
Epoch 674/1000
455/455 [==============================] - 0s 79us/step - loss: 1.0991 - accuracy: 0.7670 - val_loss: 0.5648 - val_accuracy: 0.9133
Epoch 675/1000
455/455 [==============================] - 0s 68us/step - loss: 1.1912 - accuracy: 0.7429 - val_loss

Epoch 725/1000
455/455 [==============================] - 0s 67us/step - loss: 1.1834 - accuracy: 0.7560 - val_loss: 0.5854 - val_accuracy: 0.9133
Epoch 726/1000
455/455 [==============================] - 0s 60us/step - loss: 1.2531 - accuracy: 0.7516 - val_loss: 0.5883 - val_accuracy: 0.9133
Epoch 727/1000
455/455 [==============================] - 0s 58us/step - loss: 1.1489 - accuracy: 0.7802 - val_loss: 0.5877 - val_accuracy: 0.9286
Epoch 728/1000
455/455 [==============================] - 0s 65us/step - loss: 1.1677 - accuracy: 0.7736 - val_loss: 0.5520 - val_accuracy: 0.9184
Epoch 729/1000
455/455 [==============================] - 0s 73us/step - loss: 1.0597 - accuracy: 0.7604 - val_loss: 0.5446 - val_accuracy: 0.9133
Epoch 730/1000
455/455 [==============================] - 0s 62us/step - loss: 1.0726 - accuracy: 0.7538 - val_loss: 0.5788 - val_accuracy: 0.9133
Epoch 731/1000
455/455 [==============================] - 0s 60us/step - loss: 1.3174 - accuracy: 0.7495 - val_loss: 0

Epoch 836/1000
455/455 [==============================] - 0s 72us/step - loss: 1.1374 - accuracy: 0.7429 - val_loss: 0.5555 - val_accuracy: 0.9235
Epoch 837/1000
455/455 [==============================] - 0s 84us/step - loss: 1.2192 - accuracy: 0.7143 - val_loss: 0.5665 - val_accuracy: 0.9235
Epoch 838/1000
455/455 [==============================] - 0s 71us/step - loss: 1.2179 - accuracy: 0.7824 - val_loss: 0.5399 - val_accuracy: 0.9184
Epoch 839/1000
455/455 [==============================] - 0s 69us/step - loss: 0.9420 - accuracy: 0.7868 - val_loss: 0.5588 - val_accuracy: 0.9184
Epoch 840/1000
455/455 [==============================] - 0s 72us/step - loss: 1.0349 - accuracy: 0.7495 - val_loss: 0.5548 - val_accuracy: 0.9184
Epoch 841/1000
455/455 [==============================] - 0s 73us/step - loss: 1.2453 - accuracy: 0.7407 - val_loss: 0.5614 - val_accuracy: 0.9184
Epoch 842/1000
455/455 [==============================] - 0s 68us/step - loss: 0.9655 - accuracy: 0.7626 - val_loss: 0

Epoch 947/1000
455/455 [==============================] - 0s 172us/step - loss: 1.1357 - accuracy: 0.7407 - val_loss: 0.5530 - val_accuracy: 0.9184
Epoch 948/1000
455/455 [==============================] - 0s 185us/step - loss: 1.0008 - accuracy: 0.7824 - val_loss: 0.5544 - val_accuracy: 0.9184
Epoch 949/1000
455/455 [==============================] - 0s 133us/step - loss: 1.2644 - accuracy: 0.7429 - val_loss: 0.5743 - val_accuracy: 0.9184
Epoch 950/1000
455/455 [==============================] - 0s 91us/step - loss: 1.2866 - accuracy: 0.7341 - val_loss: 0.5757 - val_accuracy: 0.9184
Epoch 951/1000
455/455 [==============================] - 0s 78us/step - loss: 0.9934 - accuracy: 0.7824 - val_loss: 0.5597 - val_accuracy: 0.9184
Epoch 952/1000
455/455 [==============================] - 0s 73us/step - loss: 1.0800 - accuracy: 0.7341 - val_loss: 0.5482 - val_accuracy: 0.9184
Epoch 953/1000
455/455 [==============================] - 0s 68us/step - loss: 1.1740 - accuracy: 0.7824 - val_loss

In [88]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over7.history['accuracy'])*100))

over-sampling train accuracy: 75.51%


In [89]:
model1_over8 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(3, activation='softmax'),
    Dropout(0.2, ),
])

In [90]:
model1_over8.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [91]:
model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 438us/step - loss: 23.6711 - accuracy: 0.3560 - val_loss: 18.5902 - val_accuracy: 0.3214
Epoch 2/1000
455/455 [==============================] - 0s 64us/step - loss: 22.2112 - accuracy: 0.3560 - val_loss: 16.6142 - val_accuracy: 0.3878
Epoch 3/1000
455/455 [==============================] - 0s 68us/step - loss: 19.8013 - accuracy: 0.3912 - val_loss: 13.9720 - val_accuracy: 0.5153
Epoch 4/1000
455/455 [==============================] - 0s 70us/step - loss: 18.1511 - accuracy: 0.4396 - val_loss: 12.1857 - val_accuracy: 0.3827
Epoch 5/1000
455/455 [==============================] - 0s 74us/step - loss: 17.0067 - accuracy: 0.4615 - val_loss: 11.5730 - val_accuracy: 0.5510
Epoch 6/1000
455/455 [==============================] - 0s 70us/step - loss: 16.1185 - accuracy: 0.5165 - val_loss: 10.9204 - val_accuracy: 0.6582
Epoch 7/1000
455/455 [==============================] - 0s 70us/step - 

Epoch 112/1000
455/455 [==============================] - 0s 118us/step - loss: 2.9512 - accuracy: 0.6967 - val_loss: 0.7462 - val_accuracy: 0.8469
Epoch 113/1000
455/455 [==============================] - 0s 109us/step - loss: 3.1982 - accuracy: 0.6571 - val_loss: 0.8060 - val_accuracy: 0.8469
Epoch 114/1000
455/455 [==============================] - 0s 97us/step - loss: 3.1754 - accuracy: 0.6505 - val_loss: 0.7729 - val_accuracy: 0.8469
Epoch 115/1000
455/455 [==============================] - 0s 94us/step - loss: 2.6845 - accuracy: 0.7055 - val_loss: 0.8229 - val_accuracy: 0.8316
Epoch 116/1000
455/455 [==============================] - 0s 109us/step - loss: 2.8999 - accuracy: 0.6725 - val_loss: 0.8048 - val_accuracy: 0.8316
Epoch 117/1000
455/455 [==============================] - 0s 105us/step - loss: 3.2938 - accuracy: 0.6615 - val_loss: 0.8081 - val_accuracy: 0.8316
Epoch 118/1000
455/455 [==============================] - 0s 108us/step - loss: 2.7634 - accuracy: 0.6791 - val_lo

Epoch 223/1000
455/455 [==============================] - 0s 90us/step - loss: 2.1909 - accuracy: 0.7099 - val_loss: 0.7599 - val_accuracy: 0.9031
Epoch 224/1000
455/455 [==============================] - 0s 85us/step - loss: 2.2634 - accuracy: 0.7033 - val_loss: 0.7474 - val_accuracy: 0.9082
Epoch 225/1000
455/455 [==============================] - 0s 83us/step - loss: 2.1352 - accuracy: 0.7033 - val_loss: 0.7593 - val_accuracy: 0.8929
Epoch 226/1000
455/455 [==============================] - 0s 82us/step - loss: 1.9245 - accuracy: 0.7495 - val_loss: 0.7531 - val_accuracy: 0.9031
Epoch 227/1000
455/455 [==============================] - 0s 79us/step - loss: 2.1077 - accuracy: 0.7473 - val_loss: 0.7237 - val_accuracy: 0.9031
Epoch 228/1000
455/455 [==============================] - 0s 83us/step - loss: 2.0168 - accuracy: 0.7077 - val_loss: 0.7424 - val_accuracy: 0.9031
Epoch 229/1000
455/455 [==============================] - 0s 80us/step - loss: 2.0224 - accuracy: 0.7363 - val_loss: 0

Epoch 279/1000
455/455 [==============================] - 0s 130us/step - loss: 1.8783 - accuracy: 0.7604 - val_loss: 0.6895 - val_accuracy: 0.9133
Epoch 280/1000
455/455 [==============================] - 0s 86us/step - loss: 2.1582 - accuracy: 0.7187 - val_loss: 0.6954 - val_accuracy: 0.9133
Epoch 281/1000
455/455 [==============================] - 0s 83us/step - loss: 1.8225 - accuracy: 0.7341 - val_loss: 0.6876 - val_accuracy: 0.9133
Epoch 282/1000
455/455 [==============================] - 0s 84us/step - loss: 1.8099 - accuracy: 0.7582 - val_loss: 0.6975 - val_accuracy: 0.9082
Epoch 283/1000
455/455 [==============================] - 0s 105us/step - loss: 2.3907 - accuracy: 0.6857 - val_loss: 0.7182 - val_accuracy: 0.9082
Epoch 284/1000
455/455 [==============================] - 0s 101us/step - loss: 1.9964 - accuracy: 0.7385 - val_loss: 0.7205 - val_accuracy: 0.9082
Epoch 285/1000
455/455 [==============================] - 0s 105us/step - loss: 1.9724 - accuracy: 0.7429 - val_los

Epoch 390/1000
455/455 [==============================] - 0s 92us/step - loss: 1.5837 - accuracy: 0.7604 - val_loss: 0.6849 - val_accuracy: 0.9133
Epoch 391/1000
455/455 [==============================] - 0s 85us/step - loss: 1.8308 - accuracy: 0.7341 - val_loss: 0.6712 - val_accuracy: 0.9133
Epoch 392/1000
455/455 [==============================] - 0s 82us/step - loss: 1.8304 - accuracy: 0.7385 - val_loss: 0.6920 - val_accuracy: 0.9133
Epoch 393/1000
455/455 [==============================] - 0s 81us/step - loss: 1.8281 - accuracy: 0.7451 - val_loss: 0.6991 - val_accuracy: 0.9133
Epoch 394/1000
455/455 [==============================] - 0s 81us/step - loss: 1.7499 - accuracy: 0.7209 - val_loss: 0.6861 - val_accuracy: 0.9133
Epoch 395/1000
455/455 [==============================] - 0s 83us/step - loss: 1.7587 - accuracy: 0.7429 - val_loss: 0.6691 - val_accuracy: 0.9082
Epoch 396/1000
455/455 [==============================] - 0s 86us/step - loss: 1.9736 - accuracy: 0.7209 - val_loss: 0

...

Epoch 725/1000
455/455 [==============================] - 0s 88us/step - loss: 1.3713 - accuracy: 0.7495 - val_loss: 0.6978 - val_accuracy: 0.9133
Epoch 726/1000
455/455 [==============================] - 0s 83us/step - loss: 1.3428 - accuracy: 0.7560 - val_loss: 0.7231 - val_accuracy: 0.9133
Epoch 727/1000
455/455 [==============================] - 0s 83us/step - loss: 1.3921 - accuracy: 0.7231 - val_loss: 0.6877 - val_accuracy: 0.9133
Epoch 728/1000
455/455 [==============================] - 0s 82us/step - loss: 1.1584 - accuracy: 0.7824 - val_loss: 0.7068 - val_accuracy: 0.9133
Epoch 729/1000
455/455 [==============================] - 0s 83us/step - loss: 1.2433 - accuracy: 0.7385 - val_loss: 0.6397 - val_accuracy: 0.9133
Epoch 730/1000
455/455 [==============================] - 0s 84us/step - loss: 1.6364 - accuracy: 0.7560 - val_loss: 0.7545 - val_accuracy: 0.9184
Epoch 731/1000
455/455 [==============================] - 0s 88us/step - loss: 1.5562 - accuracy: 0.7385 - val_loss: 0

Epoch 781/1000
455/455 [==============================] - 0s 89us/step - loss: 1.3106 - accuracy: 0.7319 - val_loss: 0.6518 - val_accuracy: 0.9184
Epoch 782/1000
455/455 [==============================] - 0s 85us/step - loss: 1.2694 - accuracy: 0.7319 - val_loss: 0.6413 - val_accuracy: 0.9184
Epoch 783/1000
455/455 [==============================] - 0s 85us/step - loss: 1.4703 - accuracy: 0.7297 - val_loss: 0.6438 - val_accuracy: 0.9184
Epoch 784/1000
455/455 [==============================] - 0s 84us/step - loss: 1.2997 - accuracy: 0.7473 - val_loss: 0.6322 - val_accuracy: 0.9235
Epoch 785/1000
455/455 [==============================] - 0s 83us/step - loss: 1.2051 - accuracy: 0.7429 - val_loss: 0.6228 - val_accuracy: 0.9235
Epoch 786/1000
455/455 [==============================] - 0s 82us/step - loss: 1.4112 - accuracy: 0.7033 - val_loss: 0.6783 - val_accuracy: 0.9184
Epoch 787/1000
455/455 [==============================] - 0s 85us/step - loss: 1.2755 - accuracy: 0.7231 - val_loss: 0

Epoch 837/1000
455/455 [==============================] - 0s 95us/step - loss: 1.1983 - accuracy: 0.7582 - val_loss: 0.6516 - val_accuracy: 0.9184
Epoch 838/1000
455/455 [==============================] - 0s 83us/step - loss: 1.4498 - accuracy: 0.7099 - val_loss: 0.6723 - val_accuracy: 0.9235
Epoch 839/1000
455/455 [==============================] - 0s 85us/step - loss: 1.2851 - accuracy: 0.7451 - val_loss: 0.6700 - val_accuracy: 0.9235
Epoch 840/1000
455/455 [==============================] - 0s 82us/step - loss: 1.3191 - accuracy: 0.7714 - val_loss: 0.6832 - val_accuracy: 0.9184
Epoch 841/1000
455/455 [==============================] - 0s 82us/step - loss: 1.4432 - accuracy: 0.7275 - val_loss: 0.6339 - val_accuracy: 0.9133
Epoch 842/1000
455/455 [==============================] - 0s 85us/step - loss: 1.3630 - accuracy: 0.7231 - val_loss: 0.6188 - val_accuracy: 0.9184
Epoch 843/1000
455/455 [==============================] - 0s 85us/step - loss: 1.5323 - accuracy: 0.7407 - val_loss: 0

Epoch 893/1000
455/455 [==============================] - 0s 85us/step - loss: 1.2668 - accuracy: 0.7451 - val_loss: 0.5897 - val_accuracy: 0.9184
Epoch 894/1000
455/455 [==============================] - 0s 80us/step - loss: 1.3817 - accuracy: 0.7363 - val_loss: 0.6157 - val_accuracy: 0.9184
Epoch 895/1000
455/455 [==============================] - 0s 66us/step - loss: 1.3519 - accuracy: 0.7538 - val_loss: 0.6468 - val_accuracy: 0.9184
Epoch 896/1000
455/455 [==============================] - 0s 70us/step - loss: 1.1409 - accuracy: 0.7648 - val_loss: 0.6624 - val_accuracy: 0.9184
Epoch 897/1000
455/455 [==============================] - 0s 77us/step - loss: 1.4346 - accuracy: 0.7538 - val_loss: 0.6664 - val_accuracy: 0.9184
Epoch 898/1000
455/455 [==============================] - 0s 69us/step - loss: 1.3083 - accuracy: 0.7407 - val_loss: 0.6454 - val_accuracy: 0.9184
Epoch 899/1000
455/455 [==============================] - 0s 69us/step - loss: 1.3609 - accuracy: 0.7231 - val_loss: 0

Epoch 949/1000
455/455 [==============================] - 0s 51us/step - loss: 1.3714 - accuracy: 0.7407 - val_loss: 0.6557 - val_accuracy: 0.9235
Epoch 950/1000
455/455 [==============================] - 0s 56us/step - loss: 1.4001 - accuracy: 0.7121 - val_loss: 0.6255 - val_accuracy: 0.9184
Epoch 951/1000
455/455 [==============================] - 0s 54us/step - loss: 1.4223 - accuracy: 0.7187 - val_loss: 0.6174 - val_accuracy: 0.9082
Epoch 952/1000
455/455 [==============================] - 0s 58us/step - loss: 1.3031 - accuracy: 0.7516 - val_loss: 0.6673 - val_accuracy: 0.9235
Epoch 953/1000
455/455 [==============================] - 0s 55us/step - loss: 1.1187 - accuracy: 0.7824 - val_loss: 0.6516 - val_accuracy: 0.9235
Epoch 954/1000
455/455 [==============================] - 0s 55us/step - loss: 1.2392 - accuracy: 0.7341 - val_loss: 0.6372 - val_accuracy: 0.9235
Epoch 955/1000
455/455 [==============================] - 0s 59us/step - loss: 1.2980 - accuracy: 0.7253 - val_loss: 0

In [92]:
acc_test_over8 = model1_over8.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over8*100))

196/196 [==============================] - 0s 38us/step
over-sampling test accuracy: 91.84%


In [93]:
hist1_over8 = model1_over8.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 455 samples, validate on 196 samples
Epoch 1/1000
455/455 [==============================] - 0s 62us/step - loss: 1.3620 - accuracy: 0.7473 - val_loss: 0.6351 - val_accuracy: 0.9184
Epoch 2/1000
455/455 [==============================] - 0s 60us/step - loss: 1.3842 - accuracy: 0.7626 - val_loss: 0.6160 - val_accuracy: 0.9184
Epoch 3/1000
455/455 [==============================] - 0s 62us/step - loss: 1.3706 - accuracy: 0.7319 - val_loss: 0.6416 - val_accuracy: 0.9184
Epoch 4/1000
455/455 [==============================] - 0s 56us/step - loss: 1.1661 - accuracy: 0.7780 - val_loss: 0.6196 - val_accuracy: 0.9184
Epoch 5/1000
455/455 [==============================] - 0s 70us/step - loss: 1.4736 - accuracy: 0.7121 - val_loss: 0.6477 - val_accuracy: 0.9133
Epoch 6/1000
455/455 [==============================] - 0s 51us/step - loss: 1.2701 - accuracy: 0.7341 - val_loss: 0.7024 - val_accuracy: 0.9235
Epoch 7/1000
455/455 [==============================] - 0s 58us/step - loss: 1.1331 

Epoch 57/1000
455/455 [==============================] - 0s 52us/step - loss: 1.1800 - accuracy: 0.7604 - val_loss: 0.6762 - val_accuracy: 0.8980
Epoch 58/1000
455/455 [==============================] - 0s 56us/step - loss: 1.6238 - accuracy: 0.7451 - val_loss: 0.5637 - val_accuracy: 0.9184
Epoch 59/1000
455/455 [==============================] - 0s 53us/step - loss: 1.4075 - accuracy: 0.7341 - val_loss: 0.5501 - val_accuracy: 0.9184
Epoch 60/1000
455/455 [==============================] - 0s 60us/step - loss: 1.2298 - accuracy: 0.7516 - val_loss: 0.5594 - val_accuracy: 0.9184
Epoch 61/1000
455/455 [==============================] - 0s 53us/step - loss: 1.2960 - accuracy: 0.7385 - val_loss: 0.6271 - val_accuracy: 0.9184
Epoch 62/1000
455/455 [==============================] - 0s 59us/step - loss: 1.1900 - accuracy: 0.7407 - val_loss: 0.6086 - val_accuracy: 0.9235
Epoch 63/1000
455/455 [==============================] - 0s 52us/step - loss: 1.2003 - accuracy: 0.7780 - val_loss: 0.6118 -

455/455 [==============================] - 0s 58us/step - loss: 1.2386 - accuracy: 0.7451 - val_loss: 0.6164 - val_accuracy: 0.9184
Epoch 114/1000
455/455 [==============================] - 0s 51us/step - loss: 1.3571 - accuracy: 0.7736 - val_loss: 0.6522 - val_accuracy: 0.9184
Epoch 115/1000
455/455 [==============================] - 0s 60us/step - loss: 1.3553 - accuracy: 0.7451 - val_loss: 0.6627 - val_accuracy: 0.9184
Epoch 116/1000
455/455 [==============================] - 0s 53us/step - loss: 1.3591 - accuracy: 0.7341 - val_loss: 0.6427 - val_accuracy: 0.9235
Epoch 117/1000
455/455 [==============================] - 0s 57us/step - loss: 1.2241 - accuracy: 0.7670 - val_loss: 0.6069 - val_accuracy: 0.9184
Epoch 118/1000
455/455 [==============================] - 0s 50us/step - loss: 1.2140 - accuracy: 0.7473 - val_loss: 0.6014 - val_accuracy: 0.9184
Epoch 119/1000
455/455 [==============================] - 0s 60us/step - loss: 1.2915 - accuracy: 0.7429 - val_loss: 0.6155 - val_acc

Epoch 169/1000
455/455 [==============================] - 0s 53us/step - loss: 1.1672 - accuracy: 0.7758 - val_loss: 0.5950 - val_accuracy: 0.9235
Epoch 170/1000
455/455 [==============================] - 0s 59us/step - loss: 1.1521 - accuracy: 0.7670 - val_loss: 0.5713 - val_accuracy: 0.9235
Epoch 171/1000
455/455 [==============================] - 0s 55us/step - loss: 1.1129 - accuracy: 0.7604 - val_loss: 0.5793 - val_accuracy: 0.9286
Epoch 172/1000
455/455 [==============================] - 0s 58us/step - loss: 1.3079 - accuracy: 0.7385 - val_loss: 0.5919 - val_accuracy: 0.9286
Epoch 173/1000
455/455 [==============================] - 0s 55us/step - loss: 1.3006 - accuracy: 0.7011 - val_loss: 0.5814 - val_accuracy: 0.9235
Epoch 174/1000
455/455 [==============================] - 0s 55us/step - loss: 1.2983 - accuracy: 0.7604 - val_loss: 0.5445 - val_accuracy: 0.9184
Epoch 175/1000
455/455 [==============================] - 0s 61us/step - loss: 1.2062 - accuracy: 0.7429 - val_loss: 0

Epoch 225/1000
455/455 [==============================] - 0s 51us/step - loss: 1.0340 - accuracy: 0.7758 - val_loss: 0.5653 - val_accuracy: 0.9184
Epoch 226/1000
455/455 [==============================] - ETA: 0s - loss: 0.8914 - accuracy: 0.84 - 0s 61us/step - loss: 1.3790 - accuracy: 0.7538 - val_loss: 0.5969 - val_accuracy: 0.9082
Epoch 227/1000
455/455 [==============================] - 0s 59us/step - loss: 1.1478 - accuracy: 0.7451 - val_loss: 0.5921 - val_accuracy: 0.9235
Epoch 228/1000
455/455 [==============================] - 0s 56us/step - loss: 0.9397 - accuracy: 0.7736 - val_loss: 0.5524 - val_accuracy: 0.9235
Epoch 229/1000
455/455 [==============================] - 0s 64us/step - loss: 1.1527 - accuracy: 0.7604 - val_loss: 0.5769 - val_accuracy: 0.9184
Epoch 230/1000
455/455 [==============================] - 0s 54us/step - loss: 1.1358 - accuracy: 0.7714 - val_loss: 0.5959 - val_accuracy: 0.9184
Epoch 231/1000
455/455 [==============================] - 0s 64us/step - los

...

Epoch 728/1000
455/455 [==============================] - 0s 52us/step - loss: 1.1531 - accuracy: 0.7516 - val_loss: 0.5714 - val_accuracy: 0.9133
Epoch 729/1000
455/455 [==============================] - 0s 56us/step - loss: 1.2274 - accuracy: 0.7451 - val_loss: 0.5635 - val_accuracy: 0.9184
Epoch 730/1000
455/455 [==============================] - 0s 52us/step - loss: 1.0876 - accuracy: 0.7363 - val_loss: 0.5654 - val_accuracy: 0.9235
Epoch 731/1000
455/455 [==============================] - 0s 62us/step - loss: 0.9970 - accuracy: 0.7341 - val_loss: 0.5795 - val_accuracy: 0.9235
Epoch 732/1000
455/455 [==============================] - 0s 51us/step - loss: 1.0142 - accuracy: 0.7780 - val_loss: 0.5702 - val_accuracy: 0.9235
Epoch 733/1000
455/455 [==============================] - 0s 64us/step - loss: 0.9061 - accuracy: 0.7890 - val_loss: 0.5728 - val_accuracy: 0.9286
Epoch 734/1000
455/455 [==============================] - 0s 52us/step - loss: 1.0541 - accuracy: 0.7604 - val_loss: 0

Epoch 784/1000
455/455 [==============================] - 0s 58us/step - loss: 1.0823 - accuracy: 0.7670 - val_loss: 0.6514 - val_accuracy: 0.9235
Epoch 785/1000
455/455 [==============================] - 0s 51us/step - loss: 1.2731 - accuracy: 0.7802 - val_loss: 0.7055 - val_accuracy: 0.9286
Epoch 786/1000
455/455 [==============================] - 0s 57us/step - loss: 1.0969 - accuracy: 0.7429 - val_loss: 0.6545 - val_accuracy: 0.9235
Epoch 787/1000
455/455 [==============================] - 0s 56us/step - loss: 1.0071 - accuracy: 0.7714 - val_loss: 0.6703 - val_accuracy: 0.9235
Epoch 788/1000
455/455 [==============================] - 0s 54us/step - loss: 1.2786 - accuracy: 0.7495 - val_loss: 0.6618 - val_accuracy: 0.9235
Epoch 789/1000
455/455 [==============================] - 0s 58us/step - loss: 1.2274 - accuracy: 0.7407 - val_loss: 0.6418 - val_accuracy: 0.9235
Epoch 790/1000
455/455 [==============================] - 0s 52us/step - loss: 1.1142 - accuracy: 0.7473 - val_loss: 0

Epoch 840/1000
455/455 [==============================] - 0s 52us/step - loss: 1.0403 - accuracy: 0.7714 - val_loss: 0.5955 - val_accuracy: 0.9235
Epoch 841/1000
455/455 [==============================] - 0s 59us/step - loss: 1.3012 - accuracy: 0.7495 - val_loss: 0.5909 - val_accuracy: 0.9235
Epoch 842/1000
455/455 [==============================] - 0s 55us/step - loss: 1.2478 - accuracy: 0.7275 - val_loss: 0.5698 - val_accuracy: 0.9133
Epoch 843/1000
455/455 [==============================] - 0s 58us/step - loss: 1.3367 - accuracy: 0.7275 - val_loss: 0.5704 - val_accuracy: 0.9133
Epoch 844/1000
455/455 [==============================] - 0s 51us/step - loss: 1.1776 - accuracy: 0.7253 - val_loss: 0.5614 - val_accuracy: 0.9235
Epoch 845/1000
455/455 [==============================] - 0s 59us/step - loss: 1.1022 - accuracy: 0.7692 - val_loss: 0.5958 - val_accuracy: 0.9286
Epoch 846/1000
455/455 [==============================] - 0s 57us/step - loss: 1.2003 - accuracy: 0.7495 - val_loss: 0

Epoch 896/1000
455/455 [==============================] - 0s 59us/step - loss: 1.1917 - accuracy: 0.7495 - val_loss: 0.6357 - val_accuracy: 0.9337
Epoch 897/1000
455/455 [==============================] - 0s 52us/step - loss: 1.2044 - accuracy: 0.7385 - val_loss: 0.6337 - val_accuracy: 0.9337
Epoch 898/1000
455/455 [==============================] - 0s 61us/step - loss: 1.1977 - accuracy: 0.7648 - val_loss: 0.6067 - val_accuracy: 0.9286
Epoch 899/1000
455/455 [==============================] - 0s 56us/step - loss: 0.8978 - accuracy: 0.7538 - val_loss: 0.5762 - val_accuracy: 0.9337
Epoch 900/1000
455/455 [==============================] - 0s 56us/step - loss: 1.1188 - accuracy: 0.7692 - val_loss: 0.5511 - val_accuracy: 0.9286
Epoch 901/1000
455/455 [==============================] - 0s 56us/step - loss: 1.0708 - accuracy: 0.7451 - val_loss: 0.5258 - val_accuracy: 0.9337
Epoch 902/1000
455/455 [==============================] - 0s 54us/step - loss: 0.9614 - accuracy: 0.7560 - val_loss: 0

Epoch 952/1000
455/455 [==============================] - 0s 52us/step - loss: 1.1426 - accuracy: 0.7626 - val_loss: 0.5994 - val_accuracy: 0.9235
Epoch 953/1000
455/455 [==============================] - 0s 61us/step - loss: 0.9835 - accuracy: 0.7626 - val_loss: 0.5903 - val_accuracy: 0.9286
Epoch 954/1000
455/455 [==============================] - 0s 53us/step - loss: 1.0865 - accuracy: 0.7692 - val_loss: 0.6105 - val_accuracy: 0.9286
Epoch 955/1000
455/455 [==============================] - 0s 61us/step - loss: 1.1065 - accuracy: 0.7692 - val_loss: 0.6033 - val_accuracy: 0.9286
Epoch 956/1000
455/455 [==============================] - 0s 55us/step - loss: 1.2426 - accuracy: 0.7670 - val_loss: 0.5786 - val_accuracy: 0.9286
Epoch 957/1000
455/455 [==============================] - 0s 60us/step - loss: 1.1191 - accuracy: 0.7714 - val_loss: 0.5987 - val_accuracy: 0.9235
Epoch 958/1000
455/455 [==============================] - 0s 64us/step - loss: 1.0653 - accuracy: 0.7451 - val_loss: 0

In [94]:
print('over-sampling train accuracy: %.2f%%' % (np.mean(hist1_over8.history['accuracy'])*100))

over-sampling train accuracy: 75.24%


In [95]:
accs_reg = [acc_test_over5, acc_test_over6, acc_test_over7, acc_test_over8]

In [96]:
mean_reg = np.mean(accs_reg)
print('over-sampling test accuracy regularization mean: %.2f%%' % (mean_reg*100))

over-sampling test accuracy regularization mean: 91.96%


In [97]:
std_reg = np.std(accs_reg)
print('over-sampling test accuracy regularization standard deviation:', std_reg)

over-sampling test accuracy regularization standard deviation: 0.0022092474255901687


In [98]:
accs_train_reg = [np.mean(hist1_over5.history['accuracy']), np.mean(hist1_over6.history['accuracy']), np.mean(hist1_over7.history['accuracy']),
             np.mean(hist1_over8.history['accuracy'])]

In [99]:
mean_train_reg = np.mean(accs_train_reg)
print('over-sampling train accuracy regularization mean: %.2f%%' % (mean_train_reg*100))

over-sampling train accuracy regularization mean: 75.40%


In [100]:
std_train_reg = np.std(accs_train_reg)
print('over-sampling train accuracy regularization standard deviation:', std_train_reg)

over-sampling train accuracy regularization standard deviation: 0.0024529449
